### ADD DEBUG 

In [1]:
# import pandas as pd
# import plotly.graph_objects as go
# from datetime import datetime, date
# import numpy as np
# import ipywidgets as widgets
# from IPython.display import display, Markdown
# import pprint
# import io

# import pandas as pd
# import numpy as np

# from datetime import datetime, date
# from IPython.display import display, Markdown
# from pathlib import Path

In [2]:
# # ==============================================================================
# # FINAL PROJECT FUNCTIONS (Consolidated - Run this cell once)
# # ==============================================================================

# import pandas as pd
# import numpy as np
# import plotly.graph_objects as go
# import ipywidgets as widgets
# import pprint
# import re

# from datetime import datetime, date
# from IPython.display import display, Markdown
# from tqdm.auto import tqdm # For progress bars
# from pathlib import Path

# # --- A. HELPER FUNCTIONS ---

# def calculate_gain(price_series: pd.Series):
#     if price_series.dropna().shape[0] < 2: return np.nan
#     return (price_series.ffill().iloc[-1] / price_series.bfill().iloc[0]) - 1

# def calculate_sharpe(return_series: pd.Series):
#     if return_series.dropna().shape[0] < 2: return np.nan
#     std_dev = return_series.std()
#     if std_dev > 0 and std_dev != np.inf:
#         return (return_series.mean() / std_dev) * np.sqrt(252)
#     return np.nan

# def parse_period_string(period_str: str) -> pd.DateOffset:
#     """
#     Parses a string like '90D', '13W', '6M', or '1Y' into a pandas DateOffset.
    
#     Args:
#         period_str (str): The period string to parse.
    
#     Returns:
#         pd.DateOffset: The corresponding DateOffset object.
        
#     Raises:
#         ValueError: If the string format is invalid.
#     """
#     # <<< NEW: This is the new helper function for decoupling.
#     match = re.match(r"(\d+)([DWMY])", period_str.strip().upper())
#     if not match:
#         raise ValueError(f"Invalid period format: '{period_str}'. Use format like '90D', '6M', '1Y'.")
    
#     quantity, unit = int(match.group(1)), match.group(2)
    
#     if unit == 'D':
#         return pd.DateOffset(days=quantity)
#     elif unit == 'W':
#         return pd.DateOffset(weeks=quantity)
#     elif unit == 'M':
#         return pd.DateOffset(months=quantity)
#     elif unit == 'Y':
#         return pd.DateOffset(years=quantity)
#     return None # Should not be reached

# # # --- B. THE CORE CALCULATION ENGINE ---

# def run_walk_forward_step(df_close_full, df_high_full, df_low_full,
#                           start_date, calc_period, fwd_period,
#                           metric, rank_start, rank_end, benchmark_ticker,
#                           debug=False):
#     debug_data = {} if debug else None
#     min_date_available = df_close_full.index.min()
#     max_date_available = df_close_full.index.max()
#     safe_start_date = max(start_date, min_date_available)
#     safe_calc_end_date = min(start_date + calc_period, max_date_available)
#     safe_viz_end_date = min(safe_calc_end_date + fwd_period, max_date_available)
#     if safe_start_date >= safe_calc_end_date:
#         return ({'error': "Invalid date range."}, None)
    
#     calc_close_raw = df_close_full.loc[safe_start_date:safe_calc_end_date]
#     calc_close = calc_close_raw.dropna(axis=1, how='all')
#     if calc_close.shape[1] == 0 or len(calc_close) < 2:
#         return ({'error': "Not enough data in calc period."}, None)

#     first_prices = calc_close.bfill().iloc[0]
#     last_prices = calc_close.ffill().iloc[-1]
#     daily_returns = calc_close.bfill().ffill().pct_change()
#     mean_returns = daily_returns.mean()
#     std_returns = daily_returns.std()
    
#     valid_tickers = calc_close.columns
#     calc_high = df_high_full[valid_tickers].loc[safe_start_date:safe_calc_end_date]
#     calc_low = df_low_full[valid_tickers].loc[safe_start_date:safe_calc_end_date]
#     tr = np.maximum(calc_high - calc_low, abs(calc_high - df_close_full[valid_tickers].shift(1)), abs(calc_low - df_close_full[valid_tickers].shift(1)))
#     atr = tr.ewm(alpha=1/14, adjust=False).mean()
#     atrp = (atr / calc_close).mean()

#     metric_values = {}
#     metric_values['Price'] = (last_prices / first_prices).dropna()
#     metric_values['Sharpe'] = (mean_returns / std_returns * np.sqrt(252)).fillna(0)
#     metric_values['Sharpe (ATR)'] = (mean_returns / atrp).fillna(0)

#     if debug:
#         df_ranking = pd.DataFrame({
#             'FirstPrice': first_prices, 'LastPrice': last_prices, 'MeanDailyReturn': mean_returns,
#             'StdDevDailyReturn': std_returns, 'MeanATRP': atrp, 'Metric_Price': metric_values['Price'],
#             'Metric_Sharpe': metric_values['Sharpe'], 'Metric_Sharpe (ATR)': metric_values['Sharpe (ATR)']
#         })
#         df_ranking.index.name = 'Ticker'
#         debug_data['ranking_metrics'] = df_ranking.sort_values(f'Metric_{metric}', ascending=False)

#     sorted_tickers = metric_values[metric].sort_values(ascending=False)
#     tickers_to_display = sorted_tickers.index[rank_start-1:rank_end].tolist()
#     if not tickers_to_display:
#         return ({'error': "No tickers found for the selected rank."}, None)

#     normalized_plot_data = df_close_full[tickers_to_display].loc[safe_start_date:safe_viz_end_date]
#     normalized_plot_data = normalized_plot_data.div(normalized_plot_data.bfill().iloc[0])
#     actual_calc_end_ts = calc_close.index.max()
#     portfolio_series = normalized_plot_data.mean(axis=1)
#     portfolio_return_series = portfolio_series.pct_change()
#     benchmark_price_series = df_close_full.get(benchmark_ticker)
#     benchmark_return_series = benchmark_price_series.loc[safe_start_date:safe_viz_end_date].bfill().ffill().pct_change() if benchmark_price_series is not None else pd.Series(dtype='float64')

#     # --- THIS SECTION IS NOW CORRECT AND COMPLETE ---
#     perf_data = {}
#     perf_data['calc_p_gain'] = calculate_gain(portfolio_series.loc[:actual_calc_end_ts])
#     perf_data['fwd_p_gain'] = calculate_gain(portfolio_series.loc[actual_calc_end_ts:])
#     perf_data['full_p_gain'] = calculate_gain(portfolio_series)
#     perf_data['calc_p_sharpe'] = calculate_sharpe(portfolio_return_series.loc[:actual_calc_end_ts])
#     perf_data['fwd_p_sharpe'] = calculate_sharpe(portfolio_return_series.loc[actual_calc_end_ts:])
#     perf_data['full_p_sharpe'] = calculate_sharpe(portfolio_return_series)
#     perf_data['calc_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:actual_calc_end_ts]) if benchmark_price_series is not None else np.nan
#     perf_data['fwd_b_gain'] = calculate_gain(benchmark_price_series.loc[actual_calc_end_ts:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
#     perf_data['full_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
#     perf_data['calc_b_sharpe'] = calculate_sharpe(benchmark_return_series.loc[:actual_calc_end_ts])
#     perf_data['fwd_b_sharpe'] = calculate_sharpe(benchmark_return_series.loc[actual_calc_end_ts:])
#     perf_data['full_b_sharpe'] = calculate_sharpe(benchmark_return_series)
#     # --- END OF CORRECTED SECTION ---

#     calc_end_prices = calc_close.ffill().iloc[-1]
#     fwd_close_slice = df_close_full.loc[actual_calc_end_ts:safe_viz_end_date]
#     viz_end_prices = fwd_close_slice.ffill().iloc[-1] if not fwd_close_slice.empty and len(fwd_close_slice) >= 2 else calc_end_prices
#     calc_gains = (calc_end_prices / calc_close.bfill().iloc[0]) - 1
#     fwd_gains = (viz_end_prices / calc_end_prices) - 1
#     results_df = pd.DataFrame({'Rank': range(rank_start, rank_start + len(tickers_to_display)), 'Metric': metric, 'MetricValue': sorted_tickers.loc[tickers_to_display].values, 'CalcPrice': calc_end_prices.loc[tickers_to_display], 'CalcGain': calc_gains.loc[tickers_to_display], 'FwdGain': fwd_gains.loc[tickers_to_display]}, index=pd.Index(tickers_to_display, name='Ticker'))
#     if benchmark_price_series is not None and benchmark_ticker in calc_close.columns:
#         benchmark_df_row = pd.DataFrame({'Rank': np.nan, 'Metric': metric, 'MetricValue': metric_values[metric].get(benchmark_ticker, np.nan), 'CalcPrice': calc_end_prices[benchmark_ticker], 'CalcGain': calc_gains[benchmark_ticker], 'FwdGain': fwd_gains[benchmark_ticker]}, index=pd.Index([f"{benchmark_ticker} (BM)"], name='Ticker'))
#         results_df = pd.concat([results_df, benchmark_df_row])
    
#     if debug:
#         df_trace = normalized_plot_data.copy()
#         df_trace.columns = [f'Norm_Price_{c}' for c in df_trace.columns]
#         df_trace['Norm_Price_Portfolio'] = portfolio_series
#         if benchmark_price_series is not None and not benchmark_price_series.loc[safe_start_date:safe_viz_end_date].dropna().empty:
#             norm_bm = benchmark_price_series.loc[safe_start_date:safe_viz_end_date] / benchmark_price_series.loc[safe_start_date:].bfill().iloc[0]
#             df_trace[f'Norm_Price_Benchmark_{benchmark_ticker}'] = norm_bm
#         for col in df_trace.columns:
#             if 'Norm_Price' in col:
#                 df_trace[col.replace('Norm_Price', 'Return')] = df_trace[col].pct_change()
#         debug_data['portfolio_trace'] = df_trace

#     final_results = {
#         'tickers_to_display': tickers_to_display,
#         'normalized_plot_data': normalized_plot_data,
#         'portfolio_series': portfolio_series,
#         'benchmark_price_series': benchmark_price_series,
#         'performance_data': perf_data,
#         'results_df': results_df,
#         'actual_calc_end_ts': actual_calc_end_ts,
#         'safe_start_date': pd.to_datetime(df_close_full.loc[safe_start_date:safe_viz_end_date].index.min()),
#         'safe_viz_end_date': pd.to_datetime(df_close_full.loc[safe_start_date:safe_viz_end_date].index.max()),
#         'error': None
#     }
    
#     return (final_results, debug_data)

# def plot_walk_forward_analyzer(df_ohlcv, 
#                                default_start_date=None, default_calc_period='3M', default_fwd_period='1M',
#                                default_metric='Sharpe (ATR)', default_rank_start=1, default_rank_end=10,
#                                default_benchmark_ticker='VOO',
#                                quality_thresholds={'min_median_dollar_volume': 1_000_000, 'max_stale_pct': 0.05, 'max_same_vol_count': 10},
#                                debug=False):
#     print("Initializing Walk-Forward Analyzer (with Dynamic Universe Filtering)...")
#     if not isinstance(df_ohlcv.index, pd.MultiIndex): raise ValueError("Input DataFrame must have a (Ticker, Date) MultiIndex.")
#     df_ohlcv = df_ohlcv.sort_index()
#     print("Pre-calculating data quality metrics...")
#     quality_metrics_df = calculate_rolling_quality_metrics(df_ohlcv, window=252)
#     print("Pre-processing data (unstacking)...")
#     df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
#     df_high_full = df_ohlcv['Adj High'].unstack(level=0)
#     df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
    
#     start_date_picker = widgets.DatePicker(description='Start Date:', value=pd.to_datetime(default_start_date), disabled=False)
    
#     # <<< CHANGED: Replaced Dropdowns with Text widgets for flexible period input.
#     calc_period_input = widgets.Text(value=default_calc_period, description='Calc Period:', placeholder="e.g., '6M', '90D'")
#     fwd_period_input = widgets.Text(value=default_fwd_period, description='Fwd Period:', placeholder="e.g., '3M', '26W'")

#     metrics = ['Price', 'Sharpe', 'Sharpe (ATR)']
#     metric_dropdown = widgets.Dropdown(options=metrics, value=default_metric, description='Metric:')
    
#     # <<< CHANGED: Replaced Dropdowns with IntText widgets for flexible rank input.
#     rank_start_input = widgets.IntText(value=default_rank_start, description='Rank Start:')
#     rank_end_input = widgets.IntText(value=default_rank_end, description='Rank End:')
    
#     benchmark_ticker_input = widgets.Text(value=default_benchmark_ticker, description='Benchmark:', placeholder='Enter Ticker')
#     update_button = widgets.Button(description="Update Chart", button_style='primary')
#     ticker_list_output = widgets.Output()
#     results_container, debug_data_container = [None], [None]

#     fig = go.FigureWidget()
#     max_traces = 50
#     for i in range(max_traces): fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'placeholder_{i}', visible=False, showlegend=False))
#     fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Benchmark', visible=True, showlegend=True, line=dict(color='black', width=3, dash='dash')))
#     fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Group Portfolio', visible=True, showlegend=True, line=dict(color='green', width=3)))

#     def update_plot(button_click):
#         ticker_list_output.clear_output()
#         start_date = pd.to_datetime(start_date_picker.value)
        
#         # <<< CHANGED: Parse period strings and handle potential errors.
#         try:
#             calc_period = parse_period_string(calc_period_input.value)
#             fwd_period = parse_period_string(fwd_period_input.value)
#         except ValueError as e:
#             with ticker_list_output:
#                 print(f"Error: {e}")
#             return

#         metric = metric_dropdown.value
#         # <<< CHANGED: Read values from IntText widgets.
#         rank_start, rank_end = rank_start_input.value, rank_end_input.value
#         benchmark_ticker = benchmark_ticker_input.value.strip().upper()
#         if rank_start > rank_end:
#             with ticker_list_output: print("Error: 'Rank Start' must be <= 'Rank End'."); return
#         if rank_start < 1:
#             with ticker_list_output: print("Error: 'Rank Start' must be >= 1."); return

#         eligible_tickers = get_eligible_universe(quality_metrics_df, start_date, quality_thresholds)
#         if not eligible_tickers:
#             with ticker_list_output: print(f"Error: No eligible tickers found on {start_date.date()} with the current quality filters."); return
#         df_close_step = df_close_full[eligible_tickers]; df_high_step = df_high_full[eligible_tickers]; df_low_step = df_low_full[eligible_tickers]

#         results, debug_output = run_walk_forward_step(
#             df_close_step, df_high_step, df_low_step, start_date, calc_period, fwd_period, 
#             metric, rank_start, rank_end, benchmark_ticker, debug=debug
#         )
#         if results['error']:
#             with ticker_list_output: print(f"Error: {results['error']}"); return

#         with fig.batch_update():
#             for i in range(max_traces):
#                 trace = fig.data[i]
#                 if i < len(results['tickers_to_display']):
#                     ticker = results['tickers_to_display'][i]; plot_data_series = results['normalized_plot_data'][ticker]
#                     trace.x, trace.y, trace.name, trace.visible, trace.showlegend = plot_data_series.index, plot_data_series.values, ticker, True, True
#                 else: trace.visible, trace.showlegend = False, False
#             benchmark_trace = fig.data[max_traces]
#             if results['benchmark_price_series'] is not None and not results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']].dropna().empty:
#                 normalized_benchmark = results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']] / results['benchmark_price_series'].loc[results['safe_start_date']:].bfill().iloc[0]
#                 benchmark_trace.x, benchmark_trace.y, benchmark_trace.name, benchmark_trace.visible = normalized_benchmark.index, normalized_benchmark, f"Benchmark ({benchmark_ticker})", True
#             else: benchmark_trace.visible = False
#             portfolio_trace = fig.data[max_traces + 1]
#             portfolio_trace.x, portfolio_trace.y, portfolio_trace.name, portfolio_trace.visible = results['portfolio_series'].index, results['portfolio_series'], 'Group Portfolio', True
#             fig.layout.shapes = []; fig.add_shape(type="line", x0=results['actual_calc_end_ts'], y0=0, x1=results['actual_calc_end_ts'], y1=1, xref='x', yref='paper', line=dict(color="grey", width=2, dash="dash"))
            
#         results_container[0] = results; debug_data_container[0] = debug_output
        
#         with ticker_list_output:
#             print(f"Analysis Period: {results['safe_start_date'].date()} to {results['safe_viz_end_date'].date()}.")
#             pprint.pprint(results['tickers_to_display'])
#             p = results['performance_data']
#             rows = []
#             rows.append({'Metric': 'Group Portfolio Gain', 'Full': p['full_p_gain'], 'Calc': p['calc_p_gain'], 'Fwd': p['fwd_p_gain']})
#             if not np.isnan(p['full_b_gain']):
#                 rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Gain', 'Full': p['full_b_gain'], 'Calc': p['calc_b_gain'], 'Fwd': p['fwd_b_gain']})
#                 rows.append({'Metric': 'Gain Delta (vs Bm)', 'Full': p['full_p_gain'] - p['full_b_gain'], 'Calc': p['calc_p_gain'] - p['calc_b_gain'], 'Fwd': p['fwd_p_gain'] - p['fwd_b_gain']})
#             rows.append({'Metric': 'Group Portfolio Sharpe', 'Full': p['full_p_sharpe'], 'Calc': p['calc_p_sharpe'], 'Fwd': p['fwd_p_sharpe']})
#             if not np.isnan(p['full_b_sharpe']):
#                 rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Sharpe', 'Full': p['full_b_sharpe'], 'Calc': p['calc_b_sharpe'], 'Fwd': p['fwd_b_sharpe']})
#                 rows.append({'Metric': 'Sharpe Delta (vs Bm)', 'Full': p['full_p_sharpe'] - p['full_b_sharpe'], 'Calc': p['calc_p_sharpe'] - p['calc_b_sharpe'], 'Fwd': p['fwd_p_sharpe'] - p['fwd_b_sharpe']})
#             report_df = pd.DataFrame(rows).set_index('Metric')
#             gain_rows = [row for row in report_df.index if 'Gain' in row or 'Delta' in row]
#             sharpe_rows = [row for row in report_df.index if 'Sharpe' in row]
#             styled_df = report_df.style.format('{:+.2%}', na_rep='N/A', subset=(gain_rows, report_df.columns)).format('{:+.2f}', na_rep='N/A', subset=(sharpe_rows, report_df.columns)).set_properties(**{'text-align': 'right', 'width': '100px'}).set_table_styles([{'selector': 'th.col_heading', 'props': [('text-align', 'right')]}, {'selector': 'th.row_heading', 'props': [('text-align', 'left')]}])
#             print("\n--- Strategy Performance Summary ---")
#             display(styled_df)
            
#     fig.update_layout(title_text='Walk-Forward Performance Analysis', xaxis_title='Date', yaxis_title='Normalized Price (Start = 1)', hovermode='x unified', legend_title_text='Tickers (Ranked)', height=600, margin=dict(t=50))
#     fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="grey")
#     update_button.on_click(update_plot)
#     # <<< CHANGED: Updated HBox to use the new input widgets.
#     controls_row1 = widgets.HBox([start_date_picker, calc_period_input, fwd_period_input])
#     controls_row2 = widgets.HBox([metric_dropdown, rank_start_input, rank_end_input, benchmark_ticker_input, update_button])
#     ui_container = widgets.VBox([controls_row1, controls_row2, ticker_list_output], layout=widgets.Layout(margin='10px 0 20px 0'))
#     display(ui_container, fig)
#     update_plot(None)
#     return (results_container, debug_data_container)

# # --- C. DYNAMIC DATA QUALITY FILTER FUNCTIONS ---

# def calculate_rolling_quality_metrics(df_ohlcv, window=252, min_periods=126, debug=False):
#     print(f"--- Calculating Rolling Quality Metrics (Window: {window} days) ---")
#     df = df_ohlcv.copy()
#     if not df.index.is_monotonic_increasing:
#         df.sort_index(inplace=True)
#     df['IsStale'] = np.where((df['Volume'] == 0) | (df['Adj High'] == df['Adj Low']), 1, 0)
#     df['DollarVolume'] = df['Adj Close'] * df['Volume']
#     df['HasSameVolumeAsPrevDay'] = (df.groupby(level='Ticker')['Volume'].diff() == 0).astype(int)
#     grouped = df.groupby(level='Ticker')
#     stale_pct = grouped['IsStale'].rolling(window=window, min_periods=min_periods).mean()
#     median_vol = grouped['DollarVolume'].rolling(window=window, min_periods=min_periods).median()
#     same_vol_count = grouped['HasSameVolumeAsPrevDay'].rolling(window=window, min_periods=min_periods).sum()
#     quality_df = pd.concat([stale_pct, median_vol, same_vol_count], axis=1)
#     quality_df.columns = ['RollingStalePct', 'RollingMedianVolume', 'RollingSameVolCount']
#     quality_df.index = quality_df.index.droplevel(0)
#     print("✅ Rolling metrics calculation complete.")
#     return quality_df

# def get_eligible_universe(quality_metrics_df, filter_date, thresholds):
#     filter_date_ts = pd.to_datetime(filter_date)
#     date_index = quality_metrics_df.index.get_level_values('Date').unique().sort_values()
#     if filter_date_ts < date_index[0]:
#         print(f"Warning: Filter date {filter_date_ts.date()} is before the earliest data point. Returning empty universe.")
#         return []
#     valid_prior_dates = date_index[date_index <= filter_date_ts]
#     if valid_prior_dates.empty:
#         print(f"Warning: No available data found on or before {filter_date_ts.date()}. Returning empty universe.")
#         return []
#     actual_date_to_use = valid_prior_dates[-1]
#     if actual_date_to_use.date() != filter_date_ts.date():
#         print(f"ℹ️ Info: Filter date {filter_date_ts.date()} not found. Using previous available date {actual_date_to_use.date()}.")
#     metrics_on_date = quality_metrics_df.xs(actual_date_to_use, level='Date')
#     mask = ((metrics_on_date['RollingMedianVolume'] >= thresholds['min_median_dollar_volume']) &
#             (metrics_on_date['RollingStalePct'] <= thresholds['max_stale_pct']) &
#             (metrics_on_date['RollingSameVolCount'] <= thresholds['max_same_vol_count']))
#     eligible_tickers = metrics_on_date[mask].index.tolist()
#     all_tickers = metrics_on_date.index.tolist()
#     print(f"Dynamic Filter ({filter_date_ts.date()}): Kept {len(eligible_tickers)} of {len(all_tickers)} tickers.")
#     return eligible_tickers    

# # --- D. INTERACTIVE ANALYSIS TOOLS ---

# def plot_walk_forward_analyzer(df_ohlcv, 
#                                default_start_date=None, default_calc_period='3M', default_fwd_period='1M',
#                                default_metric='Sharpe (ATR)', default_rank_start=1, default_rank_end=10,
#                                default_benchmark_ticker='VOO',
#                                quality_thresholds={'min_median_dollar_volume': 1_000_000, 'max_stale_pct': 0.05, 'max_same_vol_count': 10},
#                                debug=False):
    
#     print("Initializing Walk-Forward Analyzer (with Dynamic Universe Filtering)...")
#     if not isinstance(df_ohlcv.index, pd.MultiIndex): raise ValueError("Input DataFrame must have a (Ticker, Date) MultiIndex.")
#     df_ohlcv = df_ohlcv.sort_index()
    
#     print("Pre-calculating data quality metrics...")
#     quality_metrics_df = calculate_rolling_quality_metrics(df_ohlcv, window=252)
    
#     print("Pre-processing data (unstacking)...")
#     df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
#     df_high_full = df_ohlcv['Adj High'].unstack(level=0)
#     df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
    
#     start_date_picker = widgets.DatePicker(description='Start Date:', value=pd.to_datetime(default_start_date), disabled=False)
#     calc_period_options = {'3M': pd.DateOffset(months=3), '6M': pd.DateOffset(months=6), '1Y': pd.DateOffset(years=1)}
#     fwd_period_options = {'1M': pd.DateOffset(months=1), '3M': pd.DateOffset(months=3)}
#     calc_period_dropdown = widgets.Dropdown(options=calc_period_options.keys(), value=default_calc_period, description='Calc Period:')
#     fwd_period_dropdown = widgets.Dropdown(options=fwd_period_options.keys(), value=default_fwd_period, description='Fwd Period:')
#     metrics = ['Price', 'Sharpe', 'Sharpe (ATR)']
#     metric_dropdown = widgets.Dropdown(options=metrics, value=default_metric, description='Metric:')
#     rank_options = [1, 5, 10, 20, 30, 40, 50, 75, 100]
#     rank_start_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_start, description='Rank Start:')
#     rank_end_dropdown = widgets.Dropdown(options=rank_options, value=default_rank_end, description='Rank End:')
#     benchmark_ticker_input = widgets.Text(value=default_benchmark_ticker, description='Benchmark:', placeholder='Enter Ticker')
#     update_button = widgets.Button(description="Update Chart", button_style='primary')
#     ticker_list_output = widgets.Output()
    
#     results_container = [None]
#     debug_data_container = [None]

#     fig = go.FigureWidget()
#     max_traces = 50
#     for i in range(max_traces): fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'placeholder_{i}', visible=False, showlegend=False))
#     fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Benchmark', visible=True, showlegend=True, line=dict(color='black', width=3, dash='dash')))
#     fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Group Portfolio', visible=True, showlegend=True, line=dict(color='green', width=3)))

#     def update_plot(button_click):
#         ticker_list_output.clear_output()
#         start_date = pd.to_datetime(start_date_picker.value)
#         calc_period = calc_period_options[calc_period_dropdown.value]; fwd_period = fwd_period_options[fwd_period_dropdown.value]
#         metric = metric_dropdown.value; rank_start, rank_end = rank_start_dropdown.value, rank_end_dropdown.value
#         benchmark_ticker = benchmark_ticker_input.value.strip().upper()
#         if rank_start > rank_end:
#             with ticker_list_output: print("Error: 'Rank Start' must be <= 'Rank End'."); return

#         eligible_tickers = get_eligible_universe(
#             quality_metrics_df, filter_date=start_date, thresholds=quality_thresholds
#         )
#         if not eligible_tickers:
#             with ticker_list_output: print(f"Error: No eligible tickers found on {start_date.date()} with the current quality filters.")
#             return
            
#         df_close_step = df_close_full[eligible_tickers]
#         df_high_step = df_high_full[eligible_tickers]
#         df_low_step = df_low_full[eligible_tickers]

#         results, debug_output = run_walk_forward_step(
#             df_close_step, df_high_step, df_low_step,
#             start_date, calc_period, fwd_period, 
#             metric, rank_start, rank_end, benchmark_ticker,
#             debug=debug
#         )
        
#         if results['error']:
#             with ticker_list_output: print(f"Error: {results['error']}")
#             return

#         with fig.batch_update():
#             for i in range(max_traces):
#                 trace = fig.data[i]
#                 if i < len(results['tickers_to_display']):
#                     ticker = results['tickers_to_display'][i]
#                     plot_data_series = results['normalized_plot_data'][ticker]
#                     trace.x, trace.y, trace.name = plot_data_series.index, plot_data_series.values, ticker
#                     trace.visible, trace.showlegend = True, True
#                 else: trace.visible, trace.showlegend = False, False
#             benchmark_trace = fig.data[max_traces]
#             if results['benchmark_price_series'] is not None and not results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']].dropna().empty:
#                 normalized_benchmark = results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']] / results['benchmark_price_series'].loc[results['safe_start_date']:].bfill().iloc[0]
#                 benchmark_trace.x, benchmark_trace.y = normalized_benchmark.index, normalized_benchmark
#                 benchmark_trace.name = f"Benchmark ({benchmark_ticker})"; benchmark_trace.visible = True
#             else: benchmark_trace.visible = False
#             portfolio_trace = fig.data[max_traces + 1]
#             portfolio_trace.x, portfolio_trace.y = results['portfolio_series'].index, results['portfolio_series']
#             portfolio_trace.name = 'Group Portfolio'; portfolio_trace.visible = True
#             fig.layout.shapes = []; fig.add_shape(type="line", x0=results['actual_calc_end_ts'], y0=0, x1=results['actual_calc_end_ts'], y1=1, xref='x', yref='paper', line=dict(color="grey", width=2, dash="dash"))
            
#         results_container[0] = results
#         debug_data_container[0] = debug_output
        
#         with ticker_list_output:
#             print(f"Analysis Period: {results['safe_start_date'].date()} to {results['safe_viz_end_date'].date()}.")
#             pprint.pprint(results['tickers_to_display'])
            
#             # --- THIS IS THE CORRECTED, COMPLETE SECTION ---
#             p = results['performance_data']
#             rows = []
#             rows.append({'Metric': 'Group Portfolio Gain', 'Full': p['full_p_gain'], 'Calc': p['calc_p_gain'], 'Fwd': p['fwd_p_gain']})
#             if not np.isnan(p['full_b_gain']):
#                 rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Gain', 'Full': p['full_b_gain'], 'Calc': p['calc_b_gain'], 'Fwd': p['fwd_b_gain']})
#                 rows.append({'Metric': 'Gain Delta (vs Bm)', 'Full': p['full_p_gain'] - p['full_b_gain'], 'Calc': p['calc_p_gain'] - p['calc_b_gain'], 'Fwd': p['fwd_p_gain'] - p['fwd_b_gain']})
#             rows.append({'Metric': 'Group Portfolio Sharpe', 'Full': p['full_p_sharpe'], 'Calc': p['calc_p_sharpe'], 'Fwd': p['fwd_p_sharpe']})
#             if not np.isnan(p['full_b_sharpe']):
#                 rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Sharpe', 'Full': p['full_b_sharpe'], 'Calc': p['calc_b_sharpe'], 'Fwd': p['fwd_b_sharpe']})
#                 rows.append({'Metric': 'Sharpe Delta (vs Bm)', 'Full': p['full_p_sharpe'] - p['full_b_sharpe'], 'Calc': p['calc_p_sharpe'] - p['calc_b_sharpe'], 'Fwd': p['fwd_p_sharpe'] - p['fwd_b_sharpe']})
#             report_df = pd.DataFrame(rows).set_index('Metric')
#             gain_rows = [row for row in report_df.index if 'Gain' in row or 'Delta' in row]
#             sharpe_rows = [row for row in report_df.index if 'Sharpe' in row]
#             styled_df = report_df.style.format('{:+.2%}', na_rep='N/A', subset=(gain_rows, report_df.columns)).format('{:+.2f}', na_rep='N/A', subset=(sharpe_rows, report_df.columns)).set_properties(**{'text-align': 'right', 'width': '100px'}).set_table_styles([{'selector': 'th.col_heading', 'props': [('text-align', 'right')]}, {'selector': 'th.row_heading', 'props': [('text-align', 'left')]}])
#             print("\n--- Strategy Performance Summary ---")
#             display(styled_df)
#             # --- END OF CORRECTED SECTION ---
            
#     fig.update_layout(title_text='Walk-Forward Performance Analysis', xaxis_title='Date', yaxis_title='Normalized Price (Start = 1)', hovermode='x unified', legend_title_text='Tickers (Ranked)', height=600, margin=dict(t=50))
#     fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="grey")
#     update_button.on_click(update_plot)
#     controls_row1 = widgets.HBox([start_date_picker, calc_period_dropdown, fwd_period_dropdown])
#     controls_row2 = widgets.HBox([metric_dropdown, rank_start_dropdown, rank_end_dropdown, benchmark_ticker_input, update_button])
#     ui_container = widgets.VBox([controls_row1, controls_row2, ticker_list_output], layout=widgets.Layout(margin='10px 0 20px 0'))
#     display(ui_container, fig)
#     update_plot(None)
    
#     return (results_container, debug_data_container)

# # def run_full_backtest(df_ohlcv, strategy_params, quality_thresholds):
# #     print(f"--- Running Full Forensic Backtest ---")
# #     start_date, end_date = strategy_params['start_date'], strategy_params['end_date']
# #     calc_period_str, fwd_period_str = strategy_params['calc_period'], strategy_params['fwd_period']
# #     metric, rank_start, rank_end = strategy_params['metric'], strategy_params['rank_start'], strategy_params['rank_end']
# #     benchmark_ticker = strategy_params['benchmark_ticker']
    
# #     quality_metrics_df = calculate_rolling_quality_metrics(df_ohlcv, window=252)
# #     df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
# #     df_high_full = df_ohlcv['Adj High'].unstack(level=0)
# #     df_low_full = df_ohlcv['Adj Low'].unstack(level=0)

# #     period_options = {'3M': pd.DateOffset(months=3), '6M': pd.DateOffset(months=6), '1Y': pd.DateOffset(years=1)}
# #     calc_period, fwd_period = period_options[calc_period_str], period_options[fwd_period_str]
    
# #     step_dates = pd.date_range(start=start_date, end=end_date, freq=f'{fwd_period.months}ME')
# #     all_fwd_gains, period_by_period_debug = [], {}

# #     print(f"Simulating {len(step_dates)} periods...")
# #     for step_date in tqdm(step_dates, desc="Backtest Progress"):
# #         eligible_tickers = get_eligible_universe(quality_metrics_df, step_date, quality_thresholds)
# #         if not eligible_tickers: continue
        
# #         df_close_step, df_high_step, df_low_step = df_close_full[eligible_tickers], df_high_full[eligible_tickers], df_low_full[eligible_tickers]
        
# #         results, debug_output = run_walk_forward_step(
# #             df_close_step, df_high_step, df_low_step, step_date, calc_period, fwd_period,
# #             metric, rank_start, rank_end, benchmark_ticker, debug=True
# #         )
        
# #         if results['error'] is None:
# #             fwd_series = results['portfolio_series'].loc[results['actual_calc_end_ts']:]
# #             all_fwd_gains.append(fwd_series.pct_change().dropna())
# #             period_by_period_debug[step_date.date().isoformat()] = debug_output
            
# #     if not all_fwd_gains:
# #         print("Error: No valid periods were simulated."); return None

# #     strategy_returns = pd.concat(all_fwd_gains)
# #     strategy_equity_curve = (1 + strategy_returns).cumprod()
# #     benchmark_returns = df_close_full[benchmark_ticker].pct_change().loc[strategy_equity_curve.index]
# #     benchmark_equity_curve = (1 + benchmark_returns).cumprod()
    
# #     cumulative_equity_df = pd.DataFrame({'Strategy_Equity': strategy_equity_curve, 'Benchmark_Equity': benchmark_equity_curve})
    
# #     fig = go.Figure()
# #     fig.add_trace(go.Scatter(x=cumulative_equity_df.index, y=cumulative_equity_df['Strategy_Equity'], name='Strategy', line=dict(color='green')))
# #     fig.add_trace(go.Scatter(x=cumulative_equity_df.index, y=cumulative_equity_df['Benchmark_Equity'], name=f'Benchmark ({benchmark_ticker})', line=dict(color='black', dash='dash')))
# #     fig.update_layout(title=f"Cumulative Performance: '{metric}' Strategy (Top {rank_start}-{rank_end})", xaxis_title="Date", yaxis_title="Cumulative Growth")
# #     fig.show()

# #     final_backtest_results = {'cumulative_performance': cumulative_equity_df, 'period_by_period_debug': period_by_period_debug}
# #     print("\n✅ Full backtest complete.")
# #     return final_backtest_results

# def run_full_backtest(df_ohlcv, strategy_params, quality_thresholds):
#     print(f"--- Running Full Forensic Backtest for Strategy: {strategy_params['metric']} (Top {strategy_params['rank_start']}-{strategy_params['rank_end']}) ---")
#     start_date, end_date = strategy_params['start_date'], strategy_params['end_date']
#     calc_period_str, fwd_period_str = strategy_params['calc_period'], strategy_params['fwd_period']
#     metric, rank_start, rank_end = strategy_params['metric'], strategy_params['rank_start'], strategy_params['rank_end']
#     benchmark_ticker = strategy_params['benchmark_ticker']
#     quality_metrics_df = calculate_rolling_quality_metrics(df_ohlcv, window=252)
#     df_close_full = df_ohlcv['Adj Close'].unstack(level=0); df_high_full = df_ohlcv['Adj High'].unstack(level=0); df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
    
#     # <<< CHANGED: Now using the parse_period_string helper.
#     try:
#         calc_period = parse_period_string(calc_period_str)
#         fwd_period = parse_period_string(fwd_period_str)
#     except ValueError as e:
#         print(f"Error in strategy_params: {e}"); return None

#     # <<< CHANGED: More robust frequency determination for date range generation.
#     match = re.match(r"(\d+)([DWMY])", fwd_period_str.strip().upper())
#     quantity, unit = int(match.group(1)), match.group(2)
#     freq_map = {'D': 'D', 'W': 'W', 'M': 'ME', 'Y': 'YE'} # ME for Month-End
#     freq_str = f"{quantity}{freq_map[unit]}"
#     step_dates = pd.date_range(start=start_date, end=end_date, freq=freq_str)

#     all_fwd_gains, period_by_period_debug = [], {}

#     print(f"Simulating {len(step_dates)} periods from {step_dates[0].date()} to {step_dates[-1].date()}...")
#     for step_date in tqdm(step_dates, desc="Backtest Progress"):
#         eligible_tickers = get_eligible_universe(quality_metrics_df, step_date, quality_thresholds)
#         if not eligible_tickers: continue
#         df_close_step = df_close_full[eligible_tickers]; df_high_step = df_high_full[eligible_tickers]; df_low_step = df_low_full[eligible_tickers]
#         results, debug_output = run_walk_forward_step(
#             df_close_step, df_high_step, df_low_step, step_date, calc_period, fwd_period,
#             metric, rank_start, rank_end, benchmark_ticker, debug=True
#         )
#         if results['error'] is None:
#             fwd_series = results['portfolio_series'].loc[results['actual_calc_end_ts']:]
#             all_fwd_gains.append(fwd_series.pct_change().dropna())
#             period_by_period_debug[step_date.date().isoformat()] = debug_output
            
#     if not all_fwd_gains:
#         print("Error: No valid periods were simulated."); return None

#     strategy_returns = pd.concat(all_fwd_gains); strategy_equity_curve = (1 + strategy_returns).cumprod()
#     benchmark_returns = df_close_full[benchmark_ticker].pct_change().loc[strategy_equity_curve.index]; benchmark_equity_curve = (1 + benchmark_returns).cumprod()
#     cumulative_equity_df = pd.DataFrame({'Strategy_Equity': strategy_equity_curve, 'Benchmark_Equity': benchmark_equity_curve})
    
#     fig = go.Figure()
#     fig.add_trace(go.Scatter(x=cumulative_equity_df.index, y=cumulative_equity_df['Strategy_Equity'], name='Strategy', line=dict(color='green')))
#     fig.add_trace(go.Scatter(x=cumulative_equity_df.index, y=cumulative_equity_df['Benchmark_Equity'], name=f'Benchmark ({benchmark_ticker})', line=dict(color='black', dash='dash')))
#     fig.update_layout(title=f"Cumulative Performance: '{metric}' Strategy (Top {rank_start}-{rank_end})", xaxis_title="Date", yaxis_title="Cumulative Growth")
#     fig.show()

#     final_backtest_results = {'cumulative_performance': cumulative_equity_df, 'period_by_period_debug': period_by_period_debug}
#     print("\n✅ Full backtest complete. Results object is ready for forensic analysis.")
#     return final_backtest_results

# # --- E. VERIFICATION TOOLS ---

# # def verify_group_tickers_walk_forward_calculation(df_ohlcv, tickers_to_verify, benchmark_ticker,
# #                                                   start_date, calc_period, fwd_period, export_csv=False):
# #     # This function is unchanged
# #     display(Markdown(f"## Verification Report for Portfolio vs. Benchmark"))
# #     display(Markdown(f"**Portfolio Tickers:** `{tickers_to_verify}`"))
# #     display(Markdown(f"**Benchmark Ticker:** `{benchmark_ticker}`"))
# #     period_options = { '1M': pd.DateOffset(months=1), '3M': pd.DateOffset(months=3), '6M': pd.DateOffset(months=6), '1Y': pd.DateOffset(years=1), '0D': pd.DateOffset(days=0), '1W': pd.DateOffset(weeks=1), '2W': pd.DateOffset(weeks=2) }
# #     df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
# #     start_date_ts = pd.to_datetime(start_date)
# #     calc_offset = period_options[calc_period]; fwd_offset = period_options[fwd_period]
# #     calc_end_date_ts_theoretical = start_date_ts + calc_offset
# #     fwd_end_date_ts_theoretical = calc_end_date_ts_theoretical + fwd_offset
# #     actual_calc_end_ts = df_close_full.loc[start_date_ts:calc_end_date_ts_theoretical].index.max()
# #     display(Markdown(f"**Analysis Start Date:** `{start_date_ts.date()}`"))
# #     display(Markdown(f"**Calculation Period End Date:** `{actual_calc_end_ts.date()}`"))
# #     display(Markdown(f"**Forward Period End Date:** `{fwd_end_date_ts_theoretical.date()}`"))
# #     portfolio_prices_raw_slice = df_close_full[tickers_to_verify].loc[start_date_ts:fwd_end_date_ts_theoretical]
# #     normalized_portfolio_prices = portfolio_prices_raw_slice.div(portfolio_prices_raw_slice.bfill().iloc[0])
# #     portfolio_value_series = normalized_portfolio_prices.mean(axis=1)
# #     try: benchmark_price_series = df_close_full[benchmark_ticker]
# #     except KeyError as e: print(f"---! ERROR: Ticker {e} not found !---"); return

# #     def print_verification_steps(title, price_series):
# #         display(Markdown(f"#### Verification for: `{title}`"))
# #         if price_series.dropna().shape[0] < 2: print("  - Not enough data points."); return {'gain': np.nan, 'sharpe': np.nan}
# #         start_price = price_series.bfill().iloc[0]; end_price = price_series.ffill().iloc[-1]
# #         gain = (end_price / start_price) - 1
# #         print(f"  - Start Value (on {price_series.first_valid_index().date()}): {start_price:,.4f}\n  - End Value   (on {price_series.last_valid_index().date()}): {end_price:,.4f}\n  - Gain = ({end_price:,.4f} / {start_price:,.4f}) - 1 = {gain:.2%}")
# #         returns = price_series.pct_change()
# #         mean_return = returns.mean(); std_return = returns.std()
# #         sharpe = (mean_return / std_return * np.sqrt(252)) if std_return > 0 and std_return != np.inf else np.nan
# #         print(f"\n  - Mean Daily Return: {mean_return:.6f}\n  - Std Dev of Daily Return: {std_return:.6f}\n  - Sharpe = ({mean_return:.6f} / {std_return:.6f}) * sqrt(252) = {sharpe:.2f}")
# #         return {'gain': gain, 'sharpe': sharpe}

# #     display(Markdown("### A. Calculation Period Analysis ('In-Sample')"))
# #     perf_calc_p = print_verification_steps("Group Portfolio", portfolio_value_series.loc[start_date_ts:actual_calc_end_ts])
# #     perf_calc_b = print_verification_steps(f"Benchmark ({benchmark_ticker})", benchmark_price_series.loc[start_date_ts:actual_calc_end_ts])
# #     display(Markdown("\n### B. Forward Period Analysis ('Moment of Truth')"))
# #     perf_fwd_p = print_verification_steps("Group Portfolio", portfolio_value_series.loc[actual_calc_end_ts:fwd_end_date_ts_theoretical])
# #     perf_fwd_b = print_verification_steps(f"Benchmark ({benchmark_ticker})", benchmark_price_series.loc[actual_calc_end_ts:fwd_end_date_ts_theoretical])
# #     display(Markdown("\n### C. Full Period Analysis (Total)"))
# #     perf_full_p = print_verification_steps("Group Portfolio", portfolio_value_series)
# #     perf_full_b = print_verification_steps(f"Benchmark ({benchmark_ticker})", benchmark_price_series.loc[start_date_ts:fwd_end_date_ts_theoretical])
# #     display(Markdown("\n### D. Final Summary Table (matches analyzer output)"))
# #     rows = []
# #     rows.append({'Metric': 'Group Portfolio Gain', 'Full': perf_full_p['gain'], 'Calc': perf_calc_p['gain'], 'Fwd': perf_fwd_p['gain']})
# #     rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Gain', 'Full': perf_full_b['gain'], 'Calc': perf_calc_b['gain'], 'Fwd': perf_fwd_b['gain']})
# #     rows.append({'Metric': 'Gain Delta (vs Bm)', 'Full': perf_full_p['gain'] - perf_full_b['gain'], 'Calc': perf_calc_p['gain'] - perf_calc_b['gain'], 'Fwd': perf_fwd_p['gain'] - perf_fwd_b['gain']})
# #     rows.append({'Metric': 'Group Portfolio Sharpe', 'Full': perf_full_p['sharpe'], 'Calc': perf_calc_p['sharpe'], 'Fwd': perf_fwd_p['sharpe']})
# #     rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Sharpe', 'Full': perf_full_b['sharpe'], 'Calc': perf_calc_b['sharpe'], 'Fwd': perf_fwd_b['sharpe']})
# #     rows.append({'Metric': 'Sharpe Delta (vs Bm)', 'Full': perf_full_p['sharpe'] - perf_full_b['sharpe'], 'Calc': perf_calc_p['sharpe'] - perf_calc_b['sharpe'], 'Fwd': perf_fwd_p['sharpe'] - perf_fwd_b['sharpe']})
# #     report_df = pd.DataFrame(rows).set_index('Metric')
# #     gain_rows = [row for row in report_df.index if 'Gain' in row or 'Delta' in row]
# #     sharpe_rows = [row for row in report_df.index if 'Sharpe' in row]
# #     styled_df = report_df.style.format('{:+.2%}', na_rep='N/A', subset=(gain_rows, report_df.columns)).format('{:+.2f}', na_rep='N/A', subset=(sharpe_rows, report_df.columns)).set_properties(**{'text-align': 'right', 'width': '100px'}).set_table_styles([{'selector': 'th.col_heading', 'props': [('text-align', 'right')]}, {'selector': 'th.row_heading', 'props': [('text-align', 'left')]}])
# #     display(styled_df)
# #     if export_csv:
# #         export_df = pd.DataFrame({'Portfolio_Value_Normalized': portfolio_value_series, 'Portfolio_Return': portfolio_value_series.pct_change(), f'Benchmark_Price_{benchmark_ticker}': benchmark_price_series})
# #         filename = f"verification_group_tickers_{start_date_ts.strftime('%Y%m%d')}.csv"
# #         export_df.to_csv(filename, float_format='%.6f')
# #         print(f"\n✅ Detailed group verification data exported to '{filename}'")

# # def verify_ticker_ranking_metrics(df_ohlcv, 
# #                                   ticker, 
# #                                   start_date, 
# #                                   calc_period, 
# #                                   fwd_period, 
# #                                   export_csv=False):
# #     display(Markdown(f"## Verification Report for Ticker Ranking: `{ticker}`"))
# #     period_options = { '1M': pd.DateOffset(months=1), '3M': pd.DateOffset(months=3), '6M': pd.DateOffset(months=6), '1Y': pd.DateOffset(years=1), '0D': pd.DateOffset(days=0), '1W': pd.DateOffset(weeks=1), '2W': pd.DateOffset(weeks=2) }
# #     try: df_ticker = df_ohlcv.loc[ticker].sort_index()
# #     except KeyError: print(f"---! ERROR: Ticker '{ticker}' not found !---"); return
# #     start_date_ts = pd.to_datetime(start_date)
# #     calc_offset = period_options[calc_period]; fwd_offset = period_options[fwd_period]
# #     calc_end_date_ts = start_date_ts + calc_offset; fwd_end_date_ts = calc_end_date_ts + fwd_offset
# #     display(Markdown(f"**Analysis Start Date:** `{start_date_ts.date()}`"))
# #     display(Markdown(f"**Requested Calculation Period:** `{start_date_ts.date()}` to `{calc_end_date_ts.date()}`"))
# #     display(Markdown(f"**Requested Forward Period:**   `{calc_end_date_ts.date()}` to `{fwd_end_date_ts.date()}`"))
# #     display(Markdown("### A. Calculation Period Analysis (for Ranking Metrics)"))
# #     calc_df = df_ticker.loc[start_date_ts:calc_end_date_ts].copy()
# #     if calc_df['Adj Close'].notna().sum() < 2: print("\n---! ERROR: Not enough data points !---"); return
# #     actual_calc_end_date = calc_df.index.max().date()
# #     display(Markdown(f"**Actual Dates Used:** `{calc_df.index.min().date()}` to `{actual_calc_end_date}`"))
# #     calc_gain = calculate_gain(calc_df['Adj Close'])
# #     calc_start_price = calc_df['Adj Close'].bfill().iloc[0]
# #     calc_end_price = calc_df['Adj Close'].ffill().iloc[-1]
# #     display(Markdown("#### `CalcGain` Verification:"))
# #     print(f"  - Calc Start Price: ${calc_start_price:.2f}\n  - Calc End Price:   ${calc_end_price:.2f}  <-- 'CalcPrice'\n  - CalcGain = {calc_gain:.2%}")
# #     display(Markdown("#### `MetricValue` Verification:"))
# #     price_metric = (calc_end_price / calc_start_price)
# #     print(f"\n1. Price Metric:\n   - Formula: Last Price / First Price = {price_metric:.4f}")
# #     daily_returns = calc_df['Adj Close'].bfill().ffill().pct_change()
# #     sharpe_ratio = calculate_sharpe(daily_returns)
# #     print(f"\n2. Sharpe Metric:\n   - Mean Daily Return: {daily_returns.mean():.6f}\n   - Std Dev Daily Return: {daily_returns.std():.6f}\n   - Annualized Sharpe = {sharpe_ratio:.4f}")
# #     print(f"\n3. Sharpe (ATR) Metric:")
# #     tr = np.maximum(calc_df['Adj High'] - calc_df['Adj Low'], abs(calc_df['Adj High'] - calc_df['Adj Close'].shift(1)), abs(calc_df['Adj Low'] - calc_df['Adj Close'].shift(1)))
# #     atr = tr.ewm(alpha=1/14, adjust=False).mean()
# #     atrp_series = atr / calc_df['Adj Close']
# #     atrp_mean = atrp_series.mean()
# #     sharpe_atr = (daily_returns.mean() / atrp_mean) if atrp_mean > 0 else 0
# #     print(f"   - Mean Daily Return: {daily_returns.mean():.6f} (same as above)\n   - Average ATR Percent (ATRP): {atrp_mean:.6f}\n   - Sharpe (ATR) = {sharpe_atr:.4f}")
# #     display(Markdown("\n### B. Forward Period Analysis (`FwdGain`)"))
# #     fwd_df = df_ticker.loc[actual_calc_end_date:fwd_end_date_ts].copy()
# #     fwd_gain = calculate_gain(fwd_df['Adj Close'])
# #     fwd_end_price = fwd_df['Adj Close'].ffill().iloc[-1] if fwd_gain is not np.nan else calc_end_price
# #     print(f"  - Fwd Start Price (Calc End Price): ${calc_end_price:.2f}\n  - Fwd End Price: ${fwd_end_price:.2f}\n  - FwdGain = {fwd_gain:.2%}")
# #     display(Markdown("\n### C. Final Summary Table"))
# #     summary_data = {'Metric': ['Price', 'Sharpe', 'Sharpe (ATR)'], 'Calculated Value': [f"{price_metric:.4f}", f"{sharpe_ratio:.4f}", f"{sharpe_atr:.4f}"], 'Corresponds To': ['`MetricValue`', '`MetricValue`', '`MetricValue`'], '---': ['---','---','---'], 'Gain Metric': ['Calc Period Gain', 'Forward Period Gain'], 'Gain Value': [f"{calc_gain:.2%}", f"{fwd_gain:.2%}"], 'Gain Corresponds To': ['`CalcGain`', '`FwdGain`']}
# #     summary_df = pd.DataFrame(summary_data)
# #     display(summary_df.style.hide(axis="index"))
    
# #     if export_csv:
# #         calc_df['Period'] = 'Calculation'; calc_df['Daily_Return'] = daily_returns; calc_df['True_Range'] = tr; calc_df['ATR_14'] = atr; calc_df['ATRP'] = atrp_series
# #         fwd_df['Period'] = 'Forward'
# #         combined_df = pd.concat([calc_df, fwd_df.iloc[1:]])
# #         filename = f"verification_ticker_{ticker}_{start_date_ts.strftime('%Y%m%d')}.csv"
# #         combined_df.to_csv(filename, float_format='%.6f')
# #         print(f"\n✅ Detailed ticker data exported to '{filename}'")


# # --- E. VERIFICATION TOOLS (User Requested) ---

# def verify_group_tickers_walk_forward_calculation(df_ohlcv, tickers_to_verify, benchmark_ticker,
#                                                   start_date, calc_period, fwd_period, export_csv=False):
#     display(Markdown(f"## Verification Report for Portfolio vs. Benchmark"))
#     display(Markdown(f"**Portfolio Tickers:** `{tickers_to_verify}`\n**Benchmark Ticker:** `{benchmark_ticker}`"))
    
#     # <<< CHANGED: Removed hardcoded period_options dict, now using the parser.
#     try:
#         calc_offset = parse_period_string(calc_period)
#         fwd_offset = parse_period_string(fwd_period)
#     except ValueError as e:
#         print(f"Error: {e}"); return
        
#     df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
#     start_date_ts = pd.to_datetime(start_date)
#     calc_end_date_ts_theoretical = start_date_ts + calc_offset
#     fwd_end_date_ts_theoretical = calc_end_date_ts_theoretical + fwd_offset
#     actual_calc_end_ts = df_close_full.loc[start_date_ts:calc_end_date_ts_theoretical].index.max()
#     display(Markdown(f"**Analysis Start:** `{start_date_ts.date()}`\n**Calc End:** `{actual_calc_end_ts.date()}`\n**Fwd End:** `{fwd_end_date_ts_theoretical.date()}`"))
    
#     portfolio_prices_raw_slice = df_close_full[tickers_to_verify].loc[start_date_ts:fwd_end_date_ts_theoretical]
#     portfolio_value_series = portfolio_prices_raw_slice.div(portfolio_prices_raw_slice.bfill().iloc[0]).mean(axis=1)
#     benchmark_price_series = df_close_full.get(benchmark_ticker)

#     def print_verification_steps(title, price_series):
#         display(Markdown(f"#### Verification for: `{title}`"))
#         if price_series.dropna().shape[0] < 2: print("  - Not enough data points."); return {'gain': np.nan, 'sharpe': np.nan}
#         start_price, end_price = price_series.bfill().iloc[0], price_series.ffill().iloc[-1]
#         gain = (end_price / start_price) - 1
#         print(f"Start Value ({price_series.first_valid_index().date()}): {start_price:,.4f}\nEnd Value   ({price_series.last_valid_index().date()}): {end_price:,.4f}\nGain = {gain:.2%}")
#         returns = price_series.pct_change()
#         sharpe = calculate_sharpe(returns)
#         print(f"Mean Daily Return: {returns.mean():.6f}\nStd Dev: {returns.std():.6f}\nSharpe = {sharpe:.2f}")
#         return {'gain': gain, 'sharpe': sharpe}

#     display(Markdown("### A. Calculation Period"))
#     perf_calc_p = print_verification_steps("Group Portfolio", portfolio_value_series.loc[start_date_ts:actual_calc_end_ts])
#     perf_calc_b = print_verification_steps(f"Benchmark", benchmark_price_series.loc[start_date_ts:actual_calc_end_ts])
#     display(Markdown("### B. Forward Period"))
#     perf_fwd_p = print_verification_steps("Group Portfolio", portfolio_value_series.loc[actual_calc_end_ts:fwd_end_date_ts_theoretical])
#     perf_fwd_b = print_verification_steps(f"Benchmark", benchmark_price_series.loc[actual_calc_end_ts:fwd_end_date_ts_theoretical])
    
# def verify_ticker_ranking_metrics(df_ohlcv, ticker, start_date, calc_period, fwd_period, export_csv=False):
#     display(Markdown(f"## Verification Report for Ticker Ranking: `{ticker}`"))
    
#     # <<< CHANGED: Removed hardcoded period_options dict, now using the parser.
#     try:
#         calc_offset = parse_period_string(calc_period)
#         fwd_offset = parse_period_string(fwd_period)
#     except ValueError as e:
#         print(f"Error: {e}"); return
        
#     df_ticker = df_ohlcv.loc[ticker].sort_index()
#     start_date_ts = pd.to_datetime(start_date)
#     calc_end_date_ts = start_date_ts + calc_offset; fwd_end_date_ts = calc_end_date_ts + fwd_offset
#     calc_df = df_ticker.loc[start_date_ts:calc_end_date_ts].copy()
#     if calc_df.empty: print("No data in calc period."); return

#     display(Markdown("### A. Calculation Period (for Ranking Metrics)"))
#     calc_gain = calculate_gain(calc_df['Adj Close'])
#     calc_start_price, calc_end_price = calc_df['Adj Close'].bfill().iloc[0], calc_df['Adj Close'].ffill().iloc[-1]
#     print(f"CalcGain = {calc_gain:.2%}")
#     display(Markdown("#### `MetricValue` Verification:"))
#     price_metric = (calc_end_price / calc_start_price)
#     print(f"1. Price Metric: {price_metric:.4f}")
#     daily_returns = calc_df['Adj Close'].bfill().ffill().pct_change()
#     sharpe_ratio = calculate_sharpe(daily_returns)
#     print(f"2. Sharpe Metric: {sharpe_ratio:.4f}")
#     tr = np.maximum(calc_df['Adj High'] - calc_df['Adj Low'], abs(calc_df['Adj High'] - calc_df['Adj Close'].shift(1)), abs(calc_df['Adj Low'] - calc_df['Adj Close'].shift(1)))
#     atr = tr.ewm(alpha=1/14, adjust=False).mean()
#     atrp_mean = (atr / calc_df['Adj Close']).mean()
#     sharpe_atr = (daily_returns.mean() / atrp_mean) if atrp_mean > 0 else 0
#     print(f"3. Sharpe (ATR) Metric: {sharpe_atr:.4f}")



In [3]:
# # ==============================================================================
# # FINAL PROJECT FUNCTIONS (Golden Copy - Updated for Flexibility)
# # ==============================================================================

# import pandas as pd
# import numpy as np
# import plotly.graph_objects as go
# from datetime import datetime, date
# import ipywidgets as widgets
# from IPython.display import display, Markdown
# import pprint
# from tqdm.auto import tqdm
# import re # <-- Added for the new parser function
# from pathlib import Path

# # --- A. HELPER FUNCTIONS ---

# def parse_period_string(period_str: str) -> pd.DateOffset:
#     """
#     Parses a string like '90D', '13W', '6M', or '1Y' into a pandas DateOffset.
#     """
#     match = re.match(r"(\d+)([DWMY])", period_str.strip().upper())
#     if not match:
#         raise ValueError(f"Invalid period format: '{period_str}'. Use format like '90D', '6M', '1Y'.")
    
#     quantity, unit = int(match.group(1)), match.group(2)
    
#     if unit == 'D':
#         return pd.DateOffset(days=quantity)
#     elif unit == 'W':
#         return pd.DateOffset(weeks=quantity)
#     elif unit == 'M':
#         return pd.DateOffset(months=quantity)
#     elif unit == 'Y':
#         return pd.DateOffset(years=quantity)
#     return None

# def calculate_gain(price_series: pd.Series):
#     if price_series.dropna().shape[0] < 2: return np.nan
#     return (price_series.ffill().iloc[-1] / price_series.bfill().iloc[0]) - 1

# def calculate_sharpe(return_series: pd.Series):
#     if return_series.dropna().shape[0] < 2: return np.nan
#     std_dev = return_series.std()
#     if std_dev > 0 and std_dev != np.inf:
#         return (return_series.mean() / std_dev) * np.sqrt(252)
#     return np.nan

# # --- B. THE CORE CALCULATION ENGINE ---

# def run_walk_forward_step(df_close_full, df_high_full, df_low_full,
#                           start_date, calc_period, fwd_period,
#                           metric, rank_start, rank_end, benchmark_ticker,
#                           debug=False):
#     debug_data = {} if debug else None
#     min_date_available = df_close_full.index.min()
#     max_date_available = df_close_full.index.max()
#     safe_start_date = max(start_date, min_date_available)
#     safe_calc_end_date = min(start_date + calc_period, max_date_available)
#     safe_viz_end_date = min(safe_calc_end_date + fwd_period, max_date_available)
#     if safe_start_date >= safe_calc_end_date:
#         return ({'error': "Invalid date range."}, None)
    
#     calc_close_raw = df_close_full.loc[safe_start_date:safe_calc_end_date]
#     calc_close = calc_close_raw.dropna(axis=1, how='all')
#     if calc_close.shape[1] == 0 or len(calc_close) < 2:
#         return ({'error': "Not enough data in calc period."}, None)

#     first_prices = calc_close.bfill().iloc[0]
#     last_prices = calc_close.ffill().iloc[-1]
#     daily_returns = calc_close.bfill().ffill().pct_change()
#     mean_returns = daily_returns.mean()
#     std_returns = daily_returns.std()
    
#     valid_tickers = calc_close.columns
#     calc_high = df_high_full[valid_tickers].loc[safe_start_date:safe_calc_end_date]
#     calc_low = df_low_full[valid_tickers].loc[safe_start_date:safe_calc_end_date]
#     tr = np.maximum(calc_high - calc_low, abs(calc_high - df_close_full[valid_tickers].shift(1)), abs(calc_low - df_close_full[valid_tickers].shift(1)))
#     atr = tr.ewm(alpha=1/14, adjust=False).mean()
#     atrp = (atr / calc_close).mean()

#     metric_values = {}
#     metric_values['Price'] = (last_prices / first_prices).dropna()
#     metric_values['Sharpe'] = (mean_returns / std_returns * np.sqrt(252)).fillna(0)
#     metric_values['Sharpe (ATR)'] = (mean_returns / atrp).fillna(0)

#     if debug:
#         df_ranking = pd.DataFrame({
#             'FirstPrice': first_prices, 'LastPrice': last_prices, 'MeanDailyReturn': mean_returns,
#             'StdDevDailyReturn': std_returns, 'MeanATRP': atrp, 'Metric_Price': metric_values['Price'],
#             'Metric_Sharpe': metric_values['Sharpe'], 'Metric_Sharpe (ATR)': metric_values['Sharpe (ATR)']
#         })
#         df_ranking.index.name = 'Ticker'
#         debug_data['ranking_metrics'] = df_ranking.sort_values(f'Metric_{metric}', ascending=False)

#     sorted_tickers = metric_values[metric].sort_values(ascending=False)
#     tickers_to_display = sorted_tickers.index[rank_start-1:rank_end].tolist()
#     if not tickers_to_display:
#         return ({'error': "No tickers found for the selected rank."}, None)

#     normalized_plot_data = df_close_full[tickers_to_display].loc[safe_start_date:safe_viz_end_date]
#     normalized_plot_data = normalized_plot_data.div(normalized_plot_data.bfill().iloc[0])
#     actual_calc_end_ts = calc_close.index.max()
#     portfolio_series = normalized_plot_data.mean(axis=1)
#     portfolio_return_series = portfolio_series.pct_change()
#     benchmark_price_series = df_close_full.get(benchmark_ticker)
#     benchmark_return_series = benchmark_price_series.loc[safe_start_date:safe_viz_end_date].bfill().ffill().pct_change() if benchmark_price_series is not None else pd.Series(dtype='float64')

#     perf_data = {}
#     perf_data['calc_p_gain'] = calculate_gain(portfolio_series.loc[:actual_calc_end_ts])
#     perf_data['fwd_p_gain'] = calculate_gain(portfolio_series.loc[actual_calc_end_ts:])
#     perf_data['full_p_gain'] = calculate_gain(portfolio_series)
#     perf_data['calc_p_sharpe'] = calculate_sharpe(portfolio_return_series.loc[:actual_calc_end_ts])
#     perf_data['fwd_p_sharpe'] = calculate_sharpe(portfolio_return_series.loc[actual_calc_end_ts:])
#     perf_data['full_p_sharpe'] = calculate_sharpe(portfolio_return_series)
#     perf_data['calc_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:actual_calc_end_ts]) if benchmark_price_series is not None else np.nan
#     perf_data['fwd_b_gain'] = calculate_gain(benchmark_price_series.loc[actual_calc_end_ts:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
#     perf_data['full_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
#     perf_data['calc_b_sharpe'] = calculate_sharpe(benchmark_return_series.loc[:actual_calc_end_ts])
#     perf_data['fwd_b_sharpe'] = calculate_sharpe(benchmark_return_series.loc[actual_calc_end_ts:])
#     perf_data['full_b_sharpe'] = calculate_sharpe(benchmark_return_series)

#     calc_end_prices = calc_close.ffill().iloc[-1]
#     fwd_close_slice = df_close_full.loc[actual_calc_end_ts:safe_viz_end_date]
#     viz_end_prices = fwd_close_slice.ffill().iloc[-1] if not fwd_close_slice.empty and len(fwd_close_slice) >= 2 else calc_end_prices
#     calc_gains = (calc_end_prices / calc_close.bfill().iloc[0]) - 1
#     fwd_gains = (viz_end_prices / calc_end_prices) - 1
#     results_df = pd.DataFrame({'Rank': range(rank_start, rank_start + len(tickers_to_display)), 'Metric': metric, 'MetricValue': sorted_tickers.loc[tickers_to_display].values, 'CalcPrice': calc_end_prices.loc[tickers_to_display], 'CalcGain': calc_gains.loc[tickers_to_display], 'FwdGain': fwd_gains.loc[tickers_to_display]}, index=pd.Index(tickers_to_display, name='Ticker'))
#     if benchmark_price_series is not None and benchmark_ticker in calc_close.columns:
#         benchmark_df_row = pd.DataFrame({'Rank': np.nan, 'Metric': metric, 'MetricValue': metric_values[metric].get(benchmark_ticker, np.nan), 'CalcPrice': calc_end_prices[benchmark_ticker], 'CalcGain': calc_gains[benchmark_ticker], 'FwdGain': fwd_gains[benchmark_ticker]}, index=pd.Index([f"{benchmark_ticker} (BM)"], name='Ticker'))
#         results_df = pd.concat([results_df, benchmark_df_row])
    
#     if debug:
#         df_trace = normalized_plot_data.copy()
#         df_trace.columns = [f'Norm_Price_{c}' for c in df_trace.columns]
#         df_trace['Norm_Price_Portfolio'] = portfolio_series
#         if benchmark_price_series is not None and not benchmark_price_series.loc[safe_start_date:safe_viz_end_date].dropna().empty:
#             norm_bm = benchmark_price_series.loc[safe_start_date:safe_viz_end_date] / benchmark_price_series.loc[safe_start_date:].bfill().iloc[0]
#             df_trace[f'Norm_Price_Benchmark_{benchmark_ticker}'] = norm_bm
#         for col in df_trace.columns:
#             if 'Norm_Price' in col:
#                 df_trace[col.replace('Norm_Price', 'Return')] = df_trace[col].pct_change()
#         debug_data['portfolio_trace'] = df_trace

#     final_results = {
#         'tickers_to_display': tickers_to_display, 'normalized_plot_data': normalized_plot_data,
#         'portfolio_series': portfolio_series, 'benchmark_price_series': benchmark_price_series,
#         'performance_data': perf_data, 'results_df': results_df, 'actual_calc_end_ts': actual_calc_end_ts,
#         'safe_start_date': pd.to_datetime(df_close_full.loc[safe_start_date:safe_viz_end_date].index.min()),
#         'safe_viz_end_date': pd.to_datetime(df_close_full.loc[safe_start_date:safe_viz_end_date].index.max()),
#         'error': None
#     }
#     return (final_results, debug_data)

# # --- C. DYNAMIC DATA QUALITY FILTER FUNCTIONS ---

# def calculate_rolling_quality_metrics(df_ohlcv, window=252, min_periods=126, debug=False):
#     print(f"--- Calculating Rolling Quality Metrics (Window: {window} days) ---")
#     df = df_ohlcv.copy()
#     if not df.index.is_monotonic_increasing:
#         df.sort_index(inplace=True)
#     df['IsStale'] = np.where((df['Volume'] == 0) | (df['Adj High'] == df['Adj Low']), 1, 0)
#     df['DollarVolume'] = df['Adj Close'] * df['Volume']
#     df['HasSameVolumeAsPrevDay'] = (df.groupby(level='Ticker')['Volume'].diff() == 0).astype(int)
#     grouped = df.groupby(level='Ticker')
#     stale_pct = grouped['IsStale'].rolling(window=window, min_periods=min_periods).mean()
#     median_vol = grouped['DollarVolume'].rolling(window=window, min_periods=min_periods).median()
#     same_vol_count = grouped['HasSameVolumeAsPrevDay'].rolling(window=window, min_periods=min_periods).sum()
#     quality_df = pd.concat([stale_pct, median_vol, same_vol_count], axis=1)
#     quality_df.columns = ['RollingStalePct', 'RollingMedianVolume', 'RollingSameVolCount']
#     quality_df.index = quality_df.index.droplevel(0)
#     print("✅ Rolling metrics calculation complete.")
#     return quality_df

# def get_eligible_universe(quality_metrics_df, filter_date, thresholds):
#     filter_date_ts = pd.to_datetime(filter_date)
#     date_index = quality_metrics_df.index.get_level_values('Date').unique().sort_values()
#     if filter_date_ts < date_index[0]:
#         print(f"Warning: Filter date {filter_date_ts.date()} is before the earliest data point. Returning empty universe.")
#         return []
#     valid_prior_dates = date_index[date_index <= filter_date_ts]
#     if valid_prior_dates.empty:
#         print(f"Warning: No available data found on or before {filter_date_ts.date()}. Returning empty universe.")
#         return []
#     actual_date_to_use = valid_prior_dates[-1]
#     if actual_date_to_use.date() != filter_date_ts.date():
#         print(f"ℹ️ Info: Filter date {filter_date_ts.date()} not found. Using previous available date {actual_date_to_use.date()}.")
#     metrics_on_date = quality_metrics_df.xs(actual_date_to_use, level='Date')
#     mask = ((metrics_on_date['RollingMedianVolume'] >= thresholds['min_median_dollar_volume']) &
#             (metrics_on_date['RollingStalePct'] <= thresholds['max_stale_pct']) &
#             (metrics_on_date['RollingSameVolCount'] <= thresholds['max_same_vol_count']))
#     eligible_tickers = metrics_on_date[mask].index.tolist()
#     all_tickers = metrics_on_date.index.tolist()
#     print(f"Dynamic Filter ({filter_date_ts.date()}): Kept {len(eligible_tickers)} of {len(all_tickers)} tickers.")
#     return eligible_tickers    

# # --- D. INTERACTIVE ANALYSIS & BACKTESTING TOOLS ---

# def plot_walk_forward_analyzer(df_ohlcv, 
#                                default_start_date=None, default_calc_period='3M', default_fwd_period='1M',
#                                default_metric='Sharpe (ATR)', default_rank_start=1, default_rank_end=10,
#                                default_benchmark_ticker='VOO',
#                                quality_thresholds={'min_median_dollar_volume': 1_000_000, 'max_stale_pct': 0.05, 'max_same_vol_count': 10},
#                                debug=False):
#     print("Initializing Walk-Forward Analyzer (with Dynamic Universe Filtering)...")
#     if not isinstance(df_ohlcv.index, pd.MultiIndex): raise ValueError("Input DataFrame must have a (Ticker, Date) MultiIndex.")
#     df_ohlcv = df_ohlcv.sort_index()
#     print("Pre-calculating data quality metrics...")
#     quality_metrics_df = calculate_rolling_quality_metrics(df_ohlcv, window=252)
#     print("Pre-processing data (unstacking)...")
#     df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
#     df_high_full = df_ohlcv['Adj High'].unstack(level=0)
#     df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
    
#     start_date_picker = widgets.DatePicker(description='Start Date:', value=pd.to_datetime(default_start_date), disabled=False)
#     calc_period_input = widgets.Text(value=default_calc_period, description='Calc Period:', placeholder="e.g., '6M', '90D'")
#     fwd_period_input = widgets.Text(value=default_fwd_period, description='Fwd Period:', placeholder="e.g., '3M', '26W'")
#     metrics = ['Price', 'Sharpe', 'Sharpe (ATR)']
#     metric_dropdown = widgets.Dropdown(options=metrics, value=default_metric, description='Metric:')
#     rank_start_input = widgets.IntText(value=default_rank_start, description='Rank Start:')
#     rank_end_input = widgets.IntText(value=default_rank_end, description='Rank End:')
#     benchmark_ticker_input = widgets.Text(value=default_benchmark_ticker, description='Benchmark:', placeholder='Enter Ticker')
#     update_button = widgets.Button(description="Update Chart", button_style='primary')
#     ticker_list_output = widgets.Output()
#     results_container, debug_data_container = [None], [None]

#     fig = go.FigureWidget()
#     max_traces = 50
#     for i in range(max_traces): fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'placeholder_{i}', visible=False, showlegend=False))
#     fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Benchmark', visible=True, showlegend=True, line=dict(color='black', width=3, dash='dash')))
#     fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Group Portfolio', visible=True, showlegend=True, line=dict(color='green', width=3)))

#     def update_plot(button_click):
#         ticker_list_output.clear_output()
#         start_date = pd.to_datetime(start_date_picker.value)
#         try:
#             calc_period = parse_period_string(calc_period_input.value)
#             fwd_period = parse_period_string(fwd_period_input.value)
#         except ValueError as e:
#             with ticker_list_output:
#                 print(f"Error: {e}")
#             return
#         metric = metric_dropdown.value
#         rank_start, rank_end = rank_start_input.value, rank_end_input.value
#         benchmark_ticker = benchmark_ticker_input.value.strip().upper()
#         if rank_start > rank_end:
#             with ticker_list_output: print("Error: 'Rank Start' must be <= 'Rank End'."); return
#         if rank_start < 1:
#             with ticker_list_output: print("Error: 'Rank Start' must be >= 1."); return

#         eligible_tickers = get_eligible_universe(quality_metrics_df, start_date, quality_thresholds)
#         if not eligible_tickers:
#             with ticker_list_output: print(f"Error: No eligible tickers found on {start_date.date()} with the current quality filters."); return
#         df_close_step = df_close_full[eligible_tickers]; df_high_step = df_high_full[eligible_tickers]; df_low_step = df_low_full[eligible_tickers]

#         results, debug_output = run_walk_forward_step(
#             df_close_step, df_high_step, df_low_step, start_date, calc_period, fwd_period, 
#             metric, rank_start, rank_end, benchmark_ticker, debug=debug
#         )
#         if results['error']:
#             with ticker_list_output: print(f"Error: {results['error']}"); return

#         with fig.batch_update():
#             for i in range(max_traces):
#                 trace = fig.data[i]
#                 if i < len(results['tickers_to_display']):
#                     ticker = results['tickers_to_display'][i]; plot_data_series = results['normalized_plot_data'][ticker]
#                     trace.x, trace.y, trace.name, trace.visible, trace.showlegend = plot_data_series.index, plot_data_series.values, ticker, True, True
#                 else: trace.visible, trace.showlegend = False, False
#             benchmark_trace = fig.data[max_traces]
#             if results['benchmark_price_series'] is not None and not results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']].dropna().empty:
#                 normalized_benchmark = results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']] / results['benchmark_price_series'].loc[results['safe_start_date']:].bfill().iloc[0]
#                 benchmark_trace.x, benchmark_trace.y, benchmark_trace.name, benchmark_trace.visible = normalized_benchmark.index, normalized_benchmark, f"Benchmark ({benchmark_ticker})", True
#             else: benchmark_trace.visible = False
#             portfolio_trace = fig.data[max_traces + 1]
#             portfolio_trace.x, portfolio_trace.y, portfolio_trace.name, portfolio_trace.visible = results['portfolio_series'].index, results['portfolio_series'], 'Group Portfolio', True
#             fig.layout.shapes = []; fig.add_shape(type="line", x0=results['actual_calc_end_ts'], y0=0, x1=results['actual_calc_end_ts'], y1=1, xref='x', yref='paper', line=dict(color="grey", width=2, dash="dash"))
            
#         results_container[0] = results; debug_data_container[0] = debug_output
        
#         with ticker_list_output:
#             print(f"Analysis Period: {results['safe_start_date'].date()} to {results['safe_viz_end_date'].date()}.")
#             pprint.pprint(results['tickers_to_display'])
#             p = results['performance_data']
#             rows = []
#             rows.append({'Metric': 'Group Portfolio Gain', 'Full': p['full_p_gain'], 'Calc': p['calc_p_gain'], 'Fwd': p['fwd_p_gain']})
#             if not np.isnan(p['full_b_gain']):
#                 rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Gain', 'Full': p['full_b_gain'], 'Calc': p['calc_b_gain'], 'Fwd': p['fwd_b_gain']})
#                 rows.append({'Metric': 'Gain Delta (vs Bm)', 'Full': p['full_p_gain'] - p['full_b_gain'], 'Calc': p['calc_p_gain'] - p['calc_b_gain'], 'Fwd': p['fwd_p_gain'] - p['fwd_b_gain']})
#             rows.append({'Metric': 'Group Portfolio Sharpe', 'Full': p['full_p_sharpe'], 'Calc': p['calc_p_sharpe'], 'Fwd': p['fwd_p_sharpe']})
#             if not np.isnan(p['full_b_sharpe']):
#                 rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Sharpe', 'Full': p['full_b_sharpe'], 'Calc': p['calc_b_sharpe'], 'Fwd': p['fwd_b_sharpe']})
#                 rows.append({'Metric': 'Sharpe Delta (vs Bm)', 'Full': p['full_p_sharpe'] - p['full_b_sharpe'], 'Calc': p['calc_p_sharpe'] - p['calc_b_sharpe'], 'Fwd': p['fwd_p_sharpe'] - p['fwd_b_sharpe']})
#             report_df = pd.DataFrame(rows).set_index('Metric')
#             gain_rows = [row for row in report_df.index if 'Gain' in row or 'Delta' in row]
#             sharpe_rows = [row for row in report_df.index if 'Sharpe' in row]
#             styled_df = report_df.style.format('{:+.2%}', na_rep='N/A', subset=(gain_rows, report_df.columns)).format('{:+.2f}', na_rep='N/A', subset=(sharpe_rows, report_df.columns)).set_properties(**{'text-align': 'right', 'width': '100px'}).set_table_styles([{'selector': 'th.col_heading', 'props': [('text-align', 'right')]}, {'selector': 'th.row_heading', 'props': [('text-align', 'left')]}])
#             print("\n--- Strategy Performance Summary ---")
#             display(styled_df)
            
#     fig.update_layout(title_text='Walk-Forward Performance Analysis', xaxis_title='Date', yaxis_title='Normalized Price (Start = 1)', hovermode='x unified', legend_title_text='Tickers (Ranked)', height=600, margin=dict(t=50))
#     fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="grey")
#     update_button.on_click(update_plot)
#     controls_row1 = widgets.HBox([start_date_picker, calc_period_input, fwd_period_input])
#     controls_row2 = widgets.HBox([metric_dropdown, rank_start_input, rank_end_input, benchmark_ticker_input, update_button])
#     ui_container = widgets.VBox([controls_row1, controls_row2, ticker_list_output], layout=widgets.Layout(margin='10px 0 20px 0'))
#     display(ui_container, fig)
#     update_plot(None)
#     return (results_container, debug_data_container)

# def run_full_backtest(df_ohlcv, strategy_params, quality_thresholds):
#     print(f"--- Running Full Forensic Backtest for Strategy: {strategy_params['metric']} (Top {strategy_params['rank_start']}-{strategy_params['rank_end']}) ---")
#     start_date, end_date = strategy_params['start_date'], strategy_params['end_date']
#     calc_period_str, fwd_period_str = strategy_params['calc_period'], strategy_params['fwd_period']
#     metric, rank_start, rank_end = strategy_params['metric'], strategy_params['rank_start'], strategy_params['rank_end']
#     benchmark_ticker = strategy_params['benchmark_ticker']
#     quality_metrics_df = calculate_rolling_quality_metrics(df_ohlcv, window=252)
#     df_close_full = df_ohlcv['Adj Close'].unstack(level=0); df_high_full = df_ohlcv['Adj High'].unstack(level=0); df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
#     try:
#         calc_period = parse_period_string(calc_period_str)
#         fwd_period = parse_period_string(fwd_period_str)
#     except ValueError as e:
#         print(f"Error in strategy_params: {e}"); return None
#     match = re.match(r"(\d+)([DWMY])", fwd_period_str.strip().upper())
#     quantity, unit = int(match.group(1)), match.group(2)
#     freq_map = {'D': 'D', 'W': 'W', 'M': 'ME', 'Y': 'YE'}
#     freq_str = f"{quantity}{freq_map[unit]}"
#     step_dates = pd.date_range(start=start_date, end=end_date, freq=freq_str)
#     all_fwd_gains, period_by_period_debug = [], {}

#     print(f"Simulating {len(step_dates)} periods from {step_dates[0].date()} to {step_dates[-1].date()}...")
#     for step_date in tqdm(step_dates, desc="Backtest Progress"):
#         eligible_tickers = get_eligible_universe(quality_metrics_df, step_date, quality_thresholds)
#         if not eligible_tickers: continue
#         df_close_step = df_close_full[eligible_tickers]; df_high_step = df_high_full[eligible_tickers]; df_low_step = df_low_full[eligible_tickers]
#         results, debug_output = run_walk_forward_step(
#             df_close_step, df_high_step, df_low_step, step_date, calc_period, fwd_period,
#             metric, rank_start, rank_end, benchmark_ticker, debug=True
#         )
#         if results['error'] is None:
#             fwd_series = results['portfolio_series'].loc[results['actual_calc_end_ts']:]
#             all_fwd_gains.append(fwd_series.pct_change().dropna())
#             period_by_period_debug[step_date.date().isoformat()] = debug_output
            
#     if not all_fwd_gains:
#         print("Error: No valid periods were simulated."); return None

#     strategy_returns = pd.concat(all_fwd_gains); strategy_equity_curve = (1 + strategy_returns).cumprod()
#     benchmark_returns = df_close_full[benchmark_ticker].pct_change().loc[strategy_equity_curve.index]; benchmark_equity_curve = (1 + benchmark_returns).cumprod()
#     cumulative_equity_df = pd.DataFrame({'Strategy_Equity': strategy_equity_curve, 'Benchmark_Equity': benchmark_equity_curve})
    
#     fig = go.Figure()
#     fig.add_trace(go.Scatter(x=cumulative_equity_df.index, y=cumulative_equity_df['Strategy_Equity'], name='Strategy', line=dict(color='green')))
#     fig.add_trace(go.Scatter(x=cumulative_equity_df.index, y=cumulative_equity_df['Benchmark_Equity'], name=f'Benchmark ({benchmark_ticker})', line=dict(color='black', dash='dash')))
#     fig.update_layout(title=f"Cumulative Performance: '{metric}' Strategy (Top {rank_start}-{rank_end})", xaxis_title="Date", yaxis_title="Cumulative Growth")
#     fig.show()

#     final_backtest_results = {'cumulative_performance': cumulative_equity_df, 'period_by_period_debug': period_by_period_debug}
#     print("\n✅ Full backtest complete. Results object is ready for forensic analysis.")
#     return final_backtest_results

# # --- E. VERIFICATION TOOLS (User Requested) ---

# def verify_group_tickers_walk_forward_calculation(df_ohlcv, tickers_to_verify, benchmark_ticker,
#                                                   start_date, calc_period, fwd_period, export_csv=False):
#     display(Markdown(f"## Verification Report for Portfolio vs. Benchmark"))
#     display(Markdown(f"**Portfolio Tickers:** `{tickers_to_verify}`\n**Benchmark Ticker:** `{benchmark_ticker}`"))
#     try:
#         calc_offset = parse_period_string(calc_period)
#         fwd_offset = parse_period_string(fwd_period)
#     except ValueError as e:
#         print(f"Error: {e}"); return
#     df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
#     start_date_ts = pd.to_datetime(start_date)
#     calc_end_date_ts_theoretical = start_date_ts + calc_offset
#     fwd_end_date_ts_theoretical = calc_end_date_ts_theoretical + fwd_offset
#     actual_calc_end_ts = df_close_full.loc[start_date_ts:calc_end_date_ts_theoretical].index.max()
#     display(Markdown(f"**Analysis Start:** `{start_date_ts.date()}`\n**Calc End:** `{actual_calc_end_ts.date()}`\n**Fwd End:** `{fwd_end_date_ts_theoretical.date()}`"))
    
#     portfolio_prices_raw_slice = df_close_full[tickers_to_verify].loc[start_date_ts:fwd_end_date_ts_theoretical]
#     portfolio_value_series = portfolio_prices_raw_slice.div(portfolio_prices_raw_slice.bfill().iloc[0]).mean(axis=1)
#     benchmark_price_series = df_close_full.get(benchmark_ticker)

#     def print_verification_steps(title, price_series):
#         display(Markdown(f"#### Verification for: `{title}`"))
#         if price_series.dropna().shape[0] < 2: print("  - Not enough data points."); return {'gain': np.nan, 'sharpe': np.nan}
#         start_price, end_price = price_series.bfill().iloc[0], price_series.ffill().iloc[-1]
#         gain = (end_price / start_price) - 1
#         print(f"Start Value ({price_series.first_valid_index().date()}): {start_price:,.4f}\nEnd Value   ({price_series.last_valid_index().date()}): {end_price:,.4f}\nGain = {gain:.2%}")
#         returns = price_series.pct_change()
#         sharpe = calculate_sharpe(returns)
#         print(f"Mean Daily Return: {returns.mean():.6f}\nStd Dev: {returns.std():.6f}\nSharpe = {sharpe:.2f}")
#         return {'gain': gain, 'sharpe': sharpe}

#     display(Markdown("### A. Calculation Period"))
#     perf_calc_p = print_verification_steps("Group Portfolio", portfolio_value_series.loc[start_date_ts:actual_calc_end_ts])
#     perf_calc_b = print_verification_steps(f"Benchmark", benchmark_price_series.loc[start_date_ts:actual_calc_end_ts])
#     display(Markdown("### B. Forward Period"))
#     perf_fwd_p = print_verification_steps("Group Portfolio", portfolio_value_series.loc[actual_calc_end_ts:fwd_end_date_ts_theoretical])
#     perf_fwd_b = print_verification_steps(f"Benchmark", benchmark_price_series.loc[actual_calc_end_ts:fwd_end_date_ts_theoretical])
    
# def verify_ticker_ranking_metrics(df_ohlcv, ticker, start_date, calc_period, fwd_period, export_csv=False):
#     display(Markdown(f"## Verification Report for Ticker Ranking: `{ticker}`"))
#     try:
#         calc_offset = parse_period_string(calc_period)
#         fwd_offset = parse_period_string(fwd_period)
#     except ValueError as e:
#         print(f"Error: {e}"); return
#     df_ticker = df_ohlcv.loc[ticker].sort_index()
#     start_date_ts = pd.to_datetime(start_date)
#     calc_end_date_ts = start_date_ts + calc_offset; fwd_end_date_ts = calc_end_date_ts + fwd_offset
#     calc_df = df_ticker.loc[start_date_ts:calc_end_date_ts].copy()
#     if calc_df.empty: print("No data in calc period."); return

#     display(Markdown("### A. Calculation Period (for Ranking Metrics)"))
#     calc_gain = calculate_gain(calc_df['Adj Close'])
#     calc_start_price, calc_end_price = calc_df['Adj Close'].bfill().iloc[0], calc_df['Adj Close'].ffill().iloc[-1]
#     print(f"CalcGain = {calc_gain:.2%}")
#     display(Markdown("#### `MetricValue` Verification:"))
#     price_metric = (calc_end_price / calc_start_price)
#     print(f"1. Price Metric: {price_metric:.4f}")
#     daily_returns = calc_df['Adj Close'].bfill().ffill().pct_change()
#     sharpe_ratio = calculate_sharpe(daily_returns)
#     print(f"2. Sharpe Metric: {sharpe_ratio:.4f}")
#     tr = np.maximum(calc_df['Adj High'] - calc_df['Adj Low'], abs(calc_df['Adj High'] - calc_df['Adj Close'].shift(1)), abs(calc_df['Adj Low'] - calc_df['Adj Close'].shift(1)))
#     atr = tr.ewm(alpha=1/14, adjust=False).mean()
#     atrp_mean = (atr / calc_df['Adj Close']).mean()
#     sharpe_atr = (daily_returns.mean() / atrp_mean) if atrp_mean > 0 else 0
#     print(f"3. Sharpe (ATR) Metric: {sharpe_atr:.4f}")

In [4]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
import pprint
import os 

from datetime import datetime, date
from IPython.display import display, Markdown
from tqdm.auto import tqdm
from pathlib import Path

In [5]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 3000)

In [6]:
# # ==============================================================================
# # FINAL PROJECT FUNCTIONS (Golden Copy - Updated for Trading Day Logic)
# # ==============================================================================

# # --- A. HELPER FUNCTIONS ---

# def calculate_gain(price_series: pd.Series):
#     if price_series.dropna().shape[0] < 2: return np.nan
#     return (price_series.ffill().iloc[-1] / price_series.bfill().iloc[0]) - 1

# def calculate_sharpe(return_series: pd.Series):
#     if return_series.dropna().shape[0] < 2: return np.nan
#     std_dev = return_series.std()
#     if std_dev > 0 and std_dev != np.inf:
#         return (return_series.mean() / std_dev) * np.sqrt(252)
#     return np.nan

# # def calculate_true_range(high: pd.Series, low: pd.Series, close: pd.Series) -> pd.Series:
# #     """
# #     Calculates the True Range for a given set of price series.
# #     This function contains the robust, corrected logic.

# #     Args:
# #         high (pd.Series): Series of high prices.
# #         low (pd.Series): Series of low prices.
# #         close (pd.Series): Series of close prices. NOTE: This should be the full
# #                            series before slicing to allow for a correct shift(-1).

# #     Returns:
# #         pd.Series: A series containing the True Range values.
# #     """
# #     # <<< NEW: This is the centralized, robust True Range calculation function.
# #     prev_close = close.shift(1)
    
# #     tr_df = pd.DataFrame({
# #         'h_l': high - low,          # High - Low
# #         'h_pc': abs(high - prev_close), # abs(High - Previous Close)
# #         'l_pc': abs(low - prev_close)   # abs(Low - Previous Close)
# #     })
    
# #     return tr_df.max(axis=1)

# def calculate_true_range(high: pd.Series, low: pd.Series, close: pd.Series) -> pd.Series:
#     """
#     Calculates the True Range for a given set of price series.
#     This version is robust against index misalignments.

#     Args:
#         high (pd.Series): Series/DataFrame of high prices for the target period.
#         low (pd.Series): Series/DataFrame of low prices for the target period.
#         close (pd.Series): Series/DataFrame of close prices. Must contain data for at least
#                            one day prior to the start of the 'high'/'low' series.

#     Returns:
#         pd.Series: A series containing the True Range values, indexed identically to 'high'.
#     """
#     # <<< BUG FIX: Make the function robust by explicitly aligning indices.
    
#     # Calculate previous close using the potentially longer 'close' series
#     prev_close = close.shift(1)
    
#     # Explicitly reindex prev_close to match the exact index of the 'high' series.
#     # This is the key step to prevent alignment errors.
#     prev_close_aligned = prev_close.reindex(high.index)

#     # Now all three components are guaranteed to have the same index.
#     tr_df = pd.DataFrame({
#         'h_l': high - low,                      # High - Low
#         'h_pc': abs(high - prev_close_aligned), # abs(High - Previous Close)
#         'l_pc': abs(low - prev_close_aligned)   # abs(Low - Previous Close)
#     })
    
#     return tr_df.max(axis=1)


# # --- B. THE CORE CALCULATION ENGINE ---

# # def run_walk_forward_step(df_close_full, df_high_full, df_low_full,
# #                           master_trading_days,
# #                           start_date, calc_period, fwd_period,
# #                           metric, rank_start, rank_end, benchmark_ticker,
# #                           debug=False):
# #     debug_data = {} if debug else None
    
# #     # <<< CHANGED: All date calculations are now based on the master_trading_days index.
# #     try:
# #         start_idx = master_trading_days.get_loc(start_date)
# #     except KeyError:
# #         return ({'error': f"Start date {start_date.date()} is not a valid trading day."}, None)
        
# #     # Calculate end dates based on index locations
# #     calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
# #     viz_end_idx = min(calc_end_idx + fwd_period, len(master_trading_days) - 1)

# #     safe_start_date = master_trading_days[start_idx]
# #     safe_calc_end_date = master_trading_days[calc_end_idx]
# #     safe_viz_end_date = master_trading_days[viz_end_idx]
    
# #     if safe_start_date >= safe_calc_end_date:
# #         return ({'error': "Invalid date range (calc period has zero or negative length)."}, None)

# #     calc_close_raw = df_close_full.loc[safe_start_date:safe_calc_end_date]
# #     calc_close = calc_close_raw.dropna(axis=1, how='all')
# #     if calc_close.shape[1] == 0 or len(calc_close) < 2:
# #         return ({'error': "Not enough data in calc period."}, None)

# #     # --- The rest of the calculation logic remains largely the same ---
# #     first_prices = calc_close.bfill().iloc[0]
# #     last_prices = calc_close.ffill().iloc[-1]
# #     daily_returns = calc_close.bfill().ffill().pct_change()
# #     mean_returns = daily_returns.mean()
# #     std_returns = daily_returns.std()
    
# #     valid_tickers = calc_close.columns
# #     calc_high = df_high_full[valid_tickers].loc[safe_start_date:safe_calc_end_date]
# #     calc_low = df_low_full[valid_tickers].loc[safe_start_date:safe_calc_end_date]
# #     tr = np.maximum(calc_high - calc_low, abs(calc_high - df_close_full[valid_tickers].shift(1)), abs(calc_low - df_close_full[valid_tickers].shift(1)))
# #     atr = tr.ewm(alpha=1/14, adjust=False).mean()
# #     atrp = (atr / calc_close).mean()

# #     metric_values = {}
# #     metric_values['Price'] = (last_prices / first_prices).dropna()
# #     metric_values['Sharpe'] = (mean_returns / std_returns * np.sqrt(252)).fillna(0)
# #     metric_values['Sharpe (ATR)'] = (mean_returns / atrp).fillna(0)

# #     if debug:
# #         df_ranking = pd.DataFrame({
# #             'FirstPrice': first_prices, 'LastPrice': last_prices, 'MeanDailyReturn': mean_returns,
# #             'StdDevDailyReturn': std_returns, 'MeanATRP': atrp, 'Metric_Price': metric_values['Price'],
# #             'Metric_Sharpe': metric_values['Sharpe'], 'Metric_Sharpe (ATR)': metric_values['Sharpe (ATR)']
# #         })
# #         df_ranking.index.name = 'Ticker'
# #         debug_data['ranking_metrics'] = df_ranking.sort_values(f'Metric_{metric}', ascending=False)

# #     sorted_tickers = metric_values[metric].sort_values(ascending=False)
# #     tickers_to_display = sorted_tickers.index[rank_start-1:rank_end].tolist()
# #     if not tickers_to_display:
# #         return ({'error': "No tickers found for the selected rank."}, None)

# #     normalized_plot_data = df_close_full[tickers_to_display].loc[safe_start_date:safe_viz_end_date]
# #     normalized_plot_data = normalized_plot_data.div(normalized_plot_data.bfill().iloc[0])
# #     actual_calc_end_ts = calc_close.index.max()
# #     portfolio_series = normalized_plot_data.mean(axis=1)
# #     portfolio_return_series = portfolio_series.pct_change()
# #     benchmark_price_series = df_close_full.get(benchmark_ticker)
# #     benchmark_return_series = benchmark_price_series.loc[safe_start_date:safe_viz_end_date].bfill().ffill().pct_change() if benchmark_price_series is not None else pd.Series(dtype='float64')

# #     perf_data = {}
# #     perf_data['calc_p_gain'] = calculate_gain(portfolio_series.loc[:actual_calc_end_ts])
# #     perf_data['fwd_p_gain'] = calculate_gain(portfolio_series.loc[actual_calc_end_ts:])
# #     perf_data['full_p_gain'] = calculate_gain(portfolio_series)
# #     perf_data['calc_p_sharpe'] = calculate_sharpe(portfolio_return_series.loc[:actual_calc_end_ts])
# #     perf_data['fwd_p_sharpe'] = calculate_sharpe(portfolio_return_series.loc[actual_calc_end_ts:])
# #     perf_data['full_p_sharpe'] = calculate_sharpe(portfolio_return_series)
# #     perf_data['calc_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:actual_calc_end_ts]) if benchmark_price_series is not None else np.nan
# #     perf_data['fwd_b_gain'] = calculate_gain(benchmark_price_series.loc[actual_calc_end_ts:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
# #     perf_data['full_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
# #     perf_data['calc_b_sharpe'] = calculate_sharpe(benchmark_return_series.loc[:actual_calc_end_ts])
# #     perf_data['fwd_b_sharpe'] = calculate_sharpe(benchmark_return_series.loc[actual_calc_end_ts:])
# #     perf_data['full_b_sharpe'] = calculate_sharpe(benchmark_return_series)

# #     calc_end_prices = calc_close.ffill().iloc[-1]
# #     fwd_close_slice = df_close_full.loc[actual_calc_end_ts:safe_viz_end_date]
# #     viz_end_prices = fwd_close_slice.ffill().iloc[-1] if not fwd_close_slice.empty and len(fwd_close_slice) >= 2 else calc_end_prices
# #     calc_gains = (calc_end_prices / calc_close.bfill().iloc[0]) - 1
# #     fwd_gains = (viz_end_prices / calc_end_prices) - 1
# #     results_df = pd.DataFrame({'Rank': range(rank_start, rank_start + len(tickers_to_display)), 'Metric': metric, 'MetricValue': sorted_tickers.loc[tickers_to_display].values, 'CalcPrice': calc_end_prices.loc[tickers_to_display], 'CalcGain': calc_gains.loc[tickers_to_display], 'FwdGain': fwd_gains.loc[tickers_to_display]}, index=pd.Index(tickers_to_display, name='Ticker'))
# #     if benchmark_price_series is not None and benchmark_ticker in calc_close.columns:
# #         benchmark_df_row = pd.DataFrame({'Rank': np.nan, 'Metric': metric, 'MetricValue': metric_values[metric].get(benchmark_ticker, np.nan), 'CalcPrice': calc_end_prices[benchmark_ticker], 'CalcGain': calc_gains[benchmark_ticker], 'FwdGain': fwd_gains[benchmark_ticker]}, index=pd.Index([f"{benchmark_ticker} (BM)"], name='Ticker'))
# #         results_df = pd.concat([results_df, benchmark_df_row])
    
# #     if debug:
# #         df_trace = normalized_plot_data.copy()
# #         df_trace.columns = [f'Norm_Price_{c}' for c in df_trace.columns]
# #         df_trace['Norm_Price_Portfolio'] = portfolio_series
# #         if benchmark_price_series is not None and not benchmark_price_series.loc[safe_start_date:safe_viz_end_date].dropna().empty:
# #             norm_bm = benchmark_price_series.loc[safe_start_date:safe_viz_end_date] / benchmark_price_series.loc[safe_start_date:].bfill().iloc[0]
# #             df_trace[f'Norm_Price_Benchmark_{benchmark_ticker}'] = norm_bm
# #         for col in df_trace.columns:
# #             if 'Norm_Price' in col:
# #                 df_trace[col.replace('Norm_Price', 'Return')] = df_trace[col].pct_change()
# #         debug_data['portfolio_trace'] = df_trace

# #     final_results = {
# #         'tickers_to_display': tickers_to_display, 'normalized_plot_data': normalized_plot_data,
# #         'portfolio_series': portfolio_series, 'benchmark_price_series': benchmark_price_series,
# #         'performance_data': perf_data, 'results_df': results_df, 'actual_calc_end_ts': actual_calc_end_ts,
# #         'safe_start_date': safe_start_date, 'safe_viz_end_date': safe_viz_end_date,
# #         'error': None
# #     }
# #     return (final_results, debug_data)

# # def run_walk_forward_step(df_close_full, df_high_full, df_low_full,
# #                           master_trading_days,
# #                           start_date, calc_period, fwd_period,
# #                           metric, rank_start, rank_end, benchmark_ticker,
# #                           debug=False):
# #     debug_data = {} if debug else None
    
# #     try:
# #         start_idx = master_trading_days.get_loc(start_date)
# #     except KeyError:
# #         return ({'error': f"Start date {start_date.date()} is not a valid trading day."}, None)
        
# #     calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
# #     viz_end_idx = min(calc_end_idx + fwd_period, len(master_trading_days) - 1)

# #     safe_start_date = master_trading_days[start_idx]
# #     safe_calc_end_date = master_trading_days[calc_end_idx]
# #     safe_viz_end_date = master_trading_days[viz_end_idx]
    
# #     if safe_start_date >= safe_calc_end_date:
# #         return ({'error': "Invalid date range (calc period has zero or negative length)."}, None)

# #     calc_close_raw = df_close_full.loc[safe_start_date:safe_calc_end_date]
# #     calc_close = calc_close_raw.dropna(axis=1, how='all')
# #     if calc_close.shape[1] == 0 or len(calc_close) < 2:
# #         return ({'error': "Not enough data in calc period."}, None)

# #     first_prices = calc_close.bfill().iloc[0]
# #     last_prices = calc_close.ffill().iloc[-1]
# #     daily_returns = calc_close.bfill().ffill().pct_change()
# #     mean_returns = daily_returns.mean()
# #     std_returns = daily_returns.std()
    
# #     valid_tickers = calc_close.columns
# #     calc_high = df_high_full[valid_tickers].loc[safe_start_date:safe_calc_end_date]
# #     calc_low = df_low_full[valid_tickers].loc[safe_start_date:safe_calc_end_date]
    
# #     # <<< REFACTORED: Replaced the block of TR logic with a single call to the new helper function.
# #     tr = calculate_true_range(calc_high, calc_low, df_close_full[valid_tickers])
# #     atr = tr.ewm(alpha=1/14, adjust=False).mean()
# #     atrp = (atr / calc_close).mean()

# #     metric_values = {}
# #     metric_values['Price'] = (last_prices / first_prices).dropna()
# #     metric_values['Sharpe'] = (mean_returns / std_returns * np.sqrt(252)).fillna(0)
# #     metric_values['Sharpe (ATR)'] = (mean_returns / atrp).fillna(0)

# #     if debug:
# #         df_ranking = pd.DataFrame({
# #             'FirstPrice': first_prices, 'LastPrice': last_prices, 'MeanDailyReturn': mean_returns,
# #             'StdDevDailyReturn': std_returns, 'MeanATRP': atrp, 'Metric_Price': metric_values['Price'],
# #             'Metric_Sharpe': metric_values['Sharpe'], 'Metric_Sharpe (ATR)': metric_values['Sharpe (ATR)']
# #         })
# #         df_ranking.index.name = 'Ticker'
# #         debug_data['ranking_metrics'] = df_ranking.sort_values(f'Metric_{metric}', ascending=False)

# #     sorted_tickers = metric_values[metric].sort_values(ascending=False)
# #     tickers_to_display = sorted_tickers.index[rank_start-1:rank_end].tolist()
# #     if not tickers_to_display:
# #         return ({'error': "No tickers found for the selected rank."}, None)

# #     normalized_plot_data = df_close_full[tickers_to_display].loc[safe_start_date:safe_viz_end_date]
# #     normalized_plot_data = normalized_plot_data.div(normalized_plot_data.bfill().iloc[0])
# #     actual_calc_end_ts = calc_close.index.max()
# #     portfolio_series = normalized_plot_data.mean(axis=1)
# #     portfolio_return_series = portfolio_series.pct_change()
# #     benchmark_price_series = df_close_full.get(benchmark_ticker)
# #     benchmark_return_series = benchmark_price_series.loc[safe_start_date:safe_viz_end_date].bfill().ffill().pct_change() if benchmark_price_series is not None else pd.Series(dtype='float64')

# #     # <<< BUG FIX: Slicing the return series correctly to prevent overlap.
# #     try:
# #         # Find the integer location of the calculation end date
# #         boundary_loc = portfolio_return_series.index.get_loc(actual_calc_end_ts)
        
# #         # Calc period includes the boundary date
# #         calc_portfolio_returns = portfolio_return_series.iloc[:boundary_loc + 1]
# #         # Fwd period starts the day AFTER the boundary date
# #         fwd_portfolio_returns = portfolio_return_series.iloc[boundary_loc + 1:]

# #         # Do the same for the benchmark
# #         bm_boundary_loc = benchmark_return_series.index.get_loc(actual_calc_end_ts)
# #         calc_benchmark_returns = benchmark_return_series.iloc[:bm_boundary_loc + 1]
# #         fwd_benchmark_returns = benchmark_return_series.iloc[bm_boundary_loc + 1:]

# #     except (KeyError, IndexError):
# #         # Fallback for cases where the index might not align perfectly
# #         calc_portfolio_returns = portfolio_return_series.loc[:actual_calc_end_ts]
# #         fwd_portfolio_returns = portfolio_return_series.loc[actual_calc_end_ts:].iloc[1:]
# #         calc_benchmark_returns = benchmark_return_series.loc[:actual_calc_end_ts]
# #         fwd_benchmark_returns = benchmark_return_series.loc[actual_calc_end_ts:].iloc[1:]


# #     perf_data = {}
# #     perf_data['calc_p_gain'] = calculate_gain(portfolio_series.loc[:actual_calc_end_ts])
# #     perf_data['fwd_p_gain'] = calculate_gain(portfolio_series.loc[actual_calc_end_ts:])
# #     perf_data['full_p_gain'] = calculate_gain(portfolio_series)
    
# #     # <<< BUG FIX: Use the corrected, non-overlapping return series for Sharpe calculations.
# #     perf_data['calc_p_sharpe'] = calculate_sharpe(calc_portfolio_returns)
# #     perf_data['fwd_p_sharpe'] = calculate_sharpe(fwd_portfolio_returns)
# #     perf_data['full_p_sharpe'] = calculate_sharpe(portfolio_return_series)
    
# #     perf_data['calc_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:actual_calc_end_ts]) if benchmark_price_series is not None else np.nan
# #     perf_data['fwd_b_gain'] = calculate_gain(benchmark_price_series.loc[actual_calc_end_ts:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
# #     perf_data['full_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:safe_viz_end_date]) if benchmark_price_series is not None else np.nan

# #     perf_data['calc_b_sharpe'] = calculate_sharpe(calc_benchmark_returns)
# #     perf_data['fwd_b_sharpe'] = calculate_sharpe(fwd_benchmark_returns)
# #     perf_data['full_b_sharpe'] = calculate_sharpe(benchmark_return_series)

# #     calc_end_prices = calc_close.ffill().iloc[-1]
# #     fwd_close_slice = df_close_full.loc[actual_calc_end_ts:safe_viz_end_date]
# #     viz_end_prices = fwd_close_slice.ffill().iloc[-1] if not fwd_close_slice.empty and len(fwd_close_slice) >= 2 else calc_end_prices
# #     calc_gains = (calc_end_prices / calc_close.bfill().iloc[0]) - 1
# #     fwd_gains = (viz_end_prices / calc_end_prices) - 1
# #     results_df = pd.DataFrame({'Rank': range(rank_start, rank_start + len(tickers_to_display)), 'Metric': metric, 'MetricValue': sorted_tickers.loc[tickers_to_display].values, 'CalcPrice': calc_end_prices.loc[tickers_to_display], 'CalcGain': calc_gains.loc[tickers_to_display], 'FwdGain': fwd_gains.loc[tickers_to_display]}, index=pd.Index(tickers_to_display, name='Ticker'))
# #     if benchmark_price_series is not None and benchmark_ticker in calc_close.columns:
# #         benchmark_df_row = pd.DataFrame({'Rank': np.nan, 'Metric': metric, 'MetricValue': metric_values[metric].get(benchmark_ticker, np.nan), 'CalcPrice': calc_end_prices[benchmark_ticker], 'CalcGain': calc_gains[benchmark_ticker], 'FwdGain': fwd_gains[benchmark_ticker]}, index=pd.Index([f"{benchmark_ticker} (BM)"], name='Ticker'))
# #         results_df = pd.concat([results_df, benchmark_df_row])
    
# #     if debug:
# #         df_trace = normalized_plot_data.copy()
# #         df_trace.columns = [f'Norm_Price_{c}' for c in df_trace.columns]
# #         df_trace['Norm_Price_Portfolio'] = portfolio_series
# #         if benchmark_price_series is not None and not benchmark_price_series.loc[safe_start_date:safe_viz_end_date].dropna().empty:
# #             norm_bm = benchmark_price_series.loc[safe_start_date:safe_viz_end_date] / benchmark_price_series.loc[safe_start_date:].bfill().iloc[0]
# #             df_trace[f'Norm_Price_Benchmark_{benchmark_ticker}'] = norm_bm
# #         for col in df_trace.columns:
# #             if 'Norm_Price' in col:
# #                 df_trace[col.replace('Norm_Price', 'Return')] = df_trace[col].pct_change()
# #         debug_data['portfolio_trace'] = df_trace

# #     final_results = {
# #         'tickers_to_display': tickers_to_display, 'normalized_plot_data': normalized_plot_data,
# #         'portfolio_series': portfolio_series, 'benchmark_price_series': benchmark_price_series,
# #         'performance_data': perf_data, 'results_df': results_df, 'actual_calc_end_ts': actual_calc_end_ts,
# #         'safe_start_date': safe_start_date, 'safe_viz_end_date': safe_viz_end_date,
# #         'error': None
# #     }
# #     return (final_results, debug_data)

# def run_walk_forward_step(df_close_full, df_high_full, df_low_full,
#                           master_trading_days,
#                           start_date, calc_period, fwd_period,
#                           metric, rank_start, rank_end, benchmark_ticker,
#                           debug=False):
#     debug_data = {} if debug else None
    
#     try:
#         start_idx = master_trading_days.get_loc(start_date)
#     except KeyError:
#         return ({'error': f"Start date {start_date.date()} is not a valid trading day."}, None)
        
#     calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
#     viz_end_idx = min(calc_end_idx + fwd_period, len(master_trading_days) - 1)

#     safe_start_date = master_trading_days[start_idx]
#     safe_calc_end_date = master_trading_days[calc_end_idx]
#     safe_viz_end_date = master_trading_days[viz_end_idx]
    
#     if safe_start_date >= safe_calc_end_date:
#         return ({'error': "Invalid date range (calc period has zero or negative length)."}, None)

#     calc_close_raw = df_close_full.loc[safe_start_date:safe_calc_end_date]
#     calc_close = calc_close_raw.dropna(axis=1, how='all')
#     if calc_close.shape[1] == 0 or len(calc_close) < 2:
#         return ({'error': "Not enough data in calc period."}, None)

#     first_prices = calc_close.bfill().iloc[0]
#     last_prices = calc_close.ffill().iloc[-1]
#     daily_returns = calc_close.bfill().ffill().pct_change()
#     mean_returns = daily_returns.mean()
#     std_returns = daily_returns.std()
    
#     valid_tickers = calc_close.columns
#     calc_high = df_high_full[valid_tickers].loc[safe_start_date:safe_calc_end_date]
#     calc_low = df_low_full[valid_tickers].loc[safe_start_date:safe_calc_end_date]
    
#     # <<< BUG FIX: Prepare a correctly sliced close series for the TR calculation.
#     # We need to include one extra day at the start for the shift() to work.
#     extended_start_idx = max(0, start_idx - 1)
#     extended_start_date = master_trading_days[extended_start_idx]
#     calc_close_for_tr = df_close_full[valid_tickers].loc[extended_start_date:safe_calc_end_date]
    
#     # Now, call the helper with correctly aligned data.
#     tr = calculate_true_range(calc_high, calc_low, calc_close_for_tr)
    
#     atr = tr.ewm(alpha=1/14, adjust=False).mean()
#     atrp = (atr / calc_close).mean()

#     metric_values = {}
#     metric_values['Price'] = (last_prices / first_prices).dropna()
#     metric_values['Sharpe'] = (mean_returns / std_returns * np.sqrt(252)).fillna(0)
#     metric_values['Sharpe (ATR)'] = (mean_returns / atrp).fillna(0)
    
#     # ... (the rest of the function is unchanged) ...
#     if debug:
#         df_ranking = pd.DataFrame({
#             'FirstPrice': first_prices, 'LastPrice': last_prices, 'MeanDailyReturn': mean_returns,
#             'StdDevDailyReturn': std_returns, 'MeanATRP': atrp, 'Metric_Price': metric_values['Price'],
#             'Metric_Sharpe': metric_values['Sharpe'], 'Metric_Sharpe (ATR)': metric_values['Sharpe (ATR)']
#         })
#         df_ranking.index.name = 'Ticker'
#         debug_data['ranking_metrics'] = df_ranking.sort_values(f'Metric_{metric}', ascending=False)

#     sorted_tickers = metric_values[metric].sort_values(ascending=False)
#     tickers_to_display = sorted_tickers.index[rank_start-1:rank_end].tolist()
#     if not tickers_to_display:
#         return ({'error': "No tickers found for the selected rank."}, None)

#     normalized_plot_data = df_close_full[tickers_to_display].loc[safe_start_date:safe_viz_end_date]
#     normalized_plot_data = normalized_plot_data.div(normalized_plot_data.bfill().iloc[0])
#     actual_calc_end_ts = calc_close.index.max()
#     portfolio_series = normalized_plot_data.mean(axis=1)
#     portfolio_return_series = portfolio_series.pct_change()
#     benchmark_price_series = df_close_full.get( benchmark_ticker)
#     benchmark_return_series = benchmark_price_series.loc[safe_start_date:safe_viz_end_date].bfill().ffill().pct_change() if benchmark_price_series is not None else pd.Series(dtype='float64')

#     try:
#         boundary_loc = portfolio_return_series.index.get_loc(actual_calc_end_ts)
#         calc_portfolio_returns = portfolio_return_series.iloc[:boundary_loc + 1]
#         fwd_portfolio_returns = portfolio_return_series.iloc[boundary_loc + 1:]
#         bm_boundary_loc = benchmark_return_series.index.get_loc(actual_calc_end_ts)
#         calc_benchmark_returns = benchmark_return_series.iloc[:bm_boundary_loc + 1]
#         fwd_benchmark_returns = benchmark_return_series.iloc[bm_boundary_loc + 1:]
#     except (KeyError, IndexError):
#         calc_portfolio_returns = portfolio_return_series.loc[:actual_calc_end_ts]
#         fwd_portfolio_returns = portfolio_return_series.loc[actual_calc_end_ts:].iloc[1:]
#         calc_benchmark_returns = benchmark_return_series.loc[:actual_calc_end_ts]
#         fwd_benchmark_returns = benchmark_return_series.loc[actual_calc_end_ts:].iloc[1:]

#     perf_data = {}
#     perf_data['calc_p_gain'] = calculate_gain(portfolio_series.loc[:actual_calc_end_ts])
#     perf_data['fwd_p_gain'] = calculate_gain(portfolio_series.loc[actual_calc_end_ts:])
#     perf_data['full_p_gain'] = calculate_gain(portfolio_series)
#     perf_data['calc_p_sharpe'] = calculate_sharpe(calc_portfolio_returns)
#     perf_data['fwd_p_sharpe'] = calculate_sharpe(fwd_portfolio_returns)
#     perf_data['full_p_sharpe'] = calculate_sharpe(portfolio_return_series)
#     perf_data['calc_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:actual_calc_end_ts]) if benchmark_price_series is not None else np.nan
#     perf_data['fwd_b_gain'] = calculate_gain(benchmark_price_series.loc[actual_calc_end_ts:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
#     perf_data['full_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
#     perf_data['calc_b_sharpe'] = calculate_sharpe(calc_benchmark_returns)
#     perf_data['fwd_b_sharpe'] = calculate_sharpe(fwd_benchmark_returns)
#     perf_data['full_b_sharpe'] = calculate_sharpe(benchmark_return_series)

#     calc_end_prices = calc_close.ffill().iloc[-1]
#     fwd_close_slice = df_close_full.loc[actual_calc_end_ts:safe_viz_end_date]
#     viz_end_prices = fwd_close_slice.ffill().iloc[-1] if not fwd_close_slice.empty and len(fwd_close_slice) >= 2 else calc_end_prices
#     calc_gains = (calc_end_prices / calc_close.bfill().iloc[0]) - 1
#     fwd_gains = (viz_end_prices / calc_end_prices) - 1
#     results_df = pd.DataFrame({'Rank': range(rank_start, rank_start + len(tickers_to_display)), 'Metric': metric, 'MetricValue': sorted_tickers.loc[tickers_to_display].values, 'CalcPrice': calc_end_prices.loc[tickers_to_display], 'CalcGain': calc_gains.loc[tickers_to_display], 'FwdGain': fwd_gains.loc[tickers_to_display]}, index=pd.Index(tickers_to_display, name='Ticker'))
#     if benchmark_price_series is not None and benchmark_ticker in calc_close.columns:
#         benchmark_df_row = pd.DataFrame({'Rank': np.nan, 'Metric': metric, 'MetricValue': metric_values[metric].get(benchmark_ticker, np.nan), 'CalcPrice': calc_end_prices[benchmark_ticker], 'CalcGain': calc_gains[benchmark_ticker], 'FwdGain': fwd_gains[benchmark_ticker]}, index=pd.Index([f"{benchmark_ticker} (BM)"], name='Ticker'))
#         results_df = pd.concat([results_df, benchmark_df_row])
    
#     if debug:
#         df_trace = normalized_plot_data.copy()
#         df_trace.columns = [f'Norm_Price_{c}' for c in df_trace.columns]
#         df_trace['Norm_Price_Portfolio'] = portfolio_series
#         if benchmark_price_series is not None and not benchmark_price_series.loc[safe_start_date:safe_viz_end_date].dropna().empty:
#             norm_bm = benchmark_price_series.loc[safe_start_date:safe_viz_end_date] / benchmark_price_series.loc[safe_start_date:].bfill().iloc[0]
#             df_trace[f'Norm_Price_Benchmark_{benchmark_ticker}'] = norm_bm
#         for col in df_trace.columns:
#             if 'Norm_Price' in col:
#                 df_trace[col.replace('Norm_Price', 'Return')] = df_trace[col].pct_change()
#         debug_data['portfolio_trace'] = df_trace

#     final_results = {
#         'tickers_to_display': tickers_to_display, 'normalized_plot_data': normalized_plot_data,
#         'portfolio_series': portfolio_series, 'benchmark_price_series': benchmark_price_series,
#         'performance_data': perf_data, 'results_df': results_df, 'actual_calc_end_ts': actual_calc_end_ts,
#         'safe_start_date': safe_start_date, 'safe_viz_end_date': safe_viz_end_date,
#         'error': None
#     }
#     return (final_results, debug_data)

# # --- C. DYNAMIC DATA QUALITY FILTER FUNCTIONS ---

# def calculate_rolling_quality_metrics(df_ohlcv, window=252, min_periods=126, debug=False):
#     # This function does not need changes, it is independent of the period logic.
#     print(f"--- Calculating Rolling Quality Metrics (Window: {window} days) ---")
#     df = df_ohlcv.copy()
#     if not df.index.is_monotonic_increasing:
#         df.sort_index(inplace=True)
#     df['IsStale'] = np.where((df['Volume'] == 0) | (df['Adj High'] == df['Adj Low']), 1, 0)
#     df['DollarVolume'] = df['Adj Close'] * df['Volume']
#     df['HasSameVolumeAsPrevDay'] = (df.groupby(level='Ticker')['Volume'].diff() == 0).astype(int)
#     grouped = df.groupby(level='Ticker')
#     stale_pct = grouped['IsStale'].rolling(window=window, min_periods=min_periods).mean()
#     median_vol = grouped['DollarVolume'].rolling(window=window, min_periods=min_periods).median()
#     same_vol_count = grouped['HasSameVolumeAsPrevDay'].rolling(window=window, min_periods=min_periods).sum()
#     quality_df = pd.concat([stale_pct, median_vol, same_vol_count], axis=1)
#     quality_df.columns = ['RollingStalePct', 'RollingMedianVolume', 'RollingSameVolCount']
#     quality_df.index = quality_df.index.droplevel(0)
#     print("✅ Rolling metrics calculation complete.")
#     return quality_df

# def get_eligible_universe(quality_metrics_df, filter_date, thresholds):
#     # This function does not need changes.
#     filter_date_ts = pd.to_datetime(filter_date)
#     date_index = quality_metrics_df.index.get_level_values('Date').unique().sort_values()
#     if filter_date_ts < date_index[0]:
#         print(f"Warning: Filter date {filter_date_ts.date()} is before the earliest data point. Returning empty universe.")
#         return []
#     valid_prior_dates = date_index[date_index <= filter_date_ts]
#     if valid_prior_dates.empty:
#         print(f"Warning: No available data found on or before {filter_date_ts.date()}. Returning empty universe.")
#         return []
#     actual_date_to_use = valid_prior_dates[-1]
#     if actual_date_to_use.date() != filter_date_ts.date():
#         print(f"ℹ️ Info: Filter date {filter_date_ts.date()} not found. Using previous available date {actual_date_to_use.date()}.")
#     metrics_on_date = quality_metrics_df.xs(actual_date_to_use, level='Date')
#     mask = ((metrics_on_date['RollingMedianVolume'] >= thresholds['min_median_dollar_volume']) &
#             (metrics_on_date['RollingStalePct'] <= thresholds['max_stale_pct']) &
#             (metrics_on_date['RollingSameVolCount'] <= thresholds['max_same_vol_count']))
#     eligible_tickers = metrics_on_date[mask].index.tolist()
#     all_tickers = metrics_on_date.index.tolist()
#     print(f"Dynamic Filter ({filter_date_ts.date()}): Kept {len(eligible_tickers)} of {len(all_tickers)} tickers.")
#     return eligible_tickers    

# # --- D. INTERACTIVE ANALYSIS & BACKTESTING TOOLS ---

# def plot_walk_forward_analyzer(df_ohlcv, 
#                                default_start_date=None, default_calc_period=126, default_fwd_period=63,
#                                default_metric='Sharpe (ATR)', default_rank_start=1, default_rank_end=10,
#                                default_benchmark_ticker='VOO', master_calendar_ticker='VOO',
#                                quality_thresholds={'min_median_dollar_volume': 1_000_000, 'max_stale_pct': 0.05, 'max_same_vol_count': 10},
#                                debug=False):
#     print("Initializing Walk-Forward Analyzer (using Trading Day Logic)...")
#     if not isinstance(df_ohlcv.index, pd.MultiIndex): raise ValueError("Input DataFrame must have a (Ticker, Date) MultiIndex.")
#     df_ohlcv = df_ohlcv.sort_index()

#     # <<< NEW: Create the master trading day calendar.
#     if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
#         raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
#     master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()
#     print(f"Master trading day calendar created from '{master_calendar_ticker}' ({len(master_trading_days)} days).")

#     print("Pre-calculating data quality metrics...")
#     quality_metrics_df = calculate_rolling_quality_metrics(df_ohlcv, window=252)
#     print("Pre-processing data (unstacking)...")
#     df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
#     df_high_full = df_ohlcv['Adj High'].unstack(level=0)
#     df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
    
#     start_date_picker = widgets.DatePicker(description='Start Date:', value=pd.to_datetime(default_start_date), disabled=False)
    
#     # <<< CHANGED: Using IntText for precise trading day counts.
#     calc_period_input = widgets.IntText(value=default_calc_period, description='Calc Period (days):')
#     fwd_period_input = widgets.IntText(value=default_fwd_period, description='Fwd Period (days):')

#     metrics = ['Price', 'Sharpe', 'Sharpe (ATR)']
#     metric_dropdown = widgets.Dropdown(options=metrics, value=default_metric, description='Metric:')
#     rank_start_input = widgets.IntText(value=default_rank_start, description='Rank Start:')
#     rank_end_input = widgets.IntText(value=default_rank_end, description='Rank End:')
#     benchmark_ticker_input = widgets.Text(value=default_benchmark_ticker, description='Benchmark:', placeholder='Enter Ticker')
#     update_button = widgets.Button(description="Update Chart", button_style='primary')
#     ticker_list_output = widgets.Output()
#     results_container, debug_data_container = [None], [None]

#     fig = go.FigureWidget()
#     max_traces = 50
#     for i in range(max_traces): fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'placeholder_{i}', visible=False, showlegend=False))
#     fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Benchmark', visible=True, showlegend=True, line=dict(color='black', width=3, dash='dash')))
#     fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Group Portfolio', visible=True, showlegend=True, line=dict(color='green', width=3)))

#     def update_plot(button_click):
#         ticker_list_output.clear_output()
        
#         # <<< CHANGED: "Snap-forward" logic for the start date.
#         start_date_raw = pd.to_datetime(start_date_picker.value)
#         start_date_idx = master_trading_days.searchsorted(start_date_raw)
#         if start_date_idx >= len(master_trading_days):
#             with ticker_list_output: print(f"Error: Start date is after the last available trading day."); return
#         actual_start_date = master_trading_days[start_date_idx]
#         with ticker_list_output: 
#             if start_date_raw.date() != actual_start_date.date():
#                 print(f"ℹ️ Info: Start date {start_date_raw.date()} is not a trading day. Snapping forward to {actual_start_date.date()}.")

#         calc_period, fwd_period = calc_period_input.value, fwd_period_input.value
#         metric = metric_dropdown.value
#         rank_start, rank_end = rank_start_input.value, rank_end_input.value
#         benchmark_ticker = benchmark_ticker_input.value.strip().upper()
        
#         # Input validation
#         if rank_start > rank_end:
#             with ticker_list_output: print("Error: 'Rank Start' must be <= 'Rank End'."); return
#         if rank_start < 1 or calc_period < 2 or fwd_period < 1:
#             with ticker_list_output: print("Error: Ranks must be >= 1, Calc Period >= 2, Fwd Period >= 1."); return

#         eligible_tickers = get_eligible_universe(quality_metrics_df, actual_start_date, quality_thresholds)
#         if not eligible_tickers:
#             with ticker_list_output: print(f"Error: No eligible tickers found on {actual_start_date.date()} with the current quality filters."); return
#         df_close_step = df_close_full[eligible_tickers]; df_high_step = df_high_full[eligible_tickers]; df_low_step = df_low_full[eligible_tickers]

#         # <<< CHANGED: Passing master_trading_days to the core engine.
#         results, debug_output = run_walk_forward_step(
#             df_close_step, df_high_step, df_low_step, master_trading_days,
#             actual_start_date, calc_period, fwd_period, 
#             metric, rank_start, rank_end, benchmark_ticker, debug=debug
#         )
#         if results['error']:
#             with ticker_list_output: print(f"Error: {results['error']}"); return

#         # --- The rest of the plotting/reporting logic remains the same ---
#         with fig.batch_update():
#             for i in range(max_traces):
#                 trace = fig.data[i]
#                 if i < len(results['tickers_to_display']):
#                     ticker = results['tickers_to_display'][i]; plot_data_series = results['normalized_plot_data'][ticker]
#                     trace.x, trace.y, trace.name, trace.visible, trace.showlegend = plot_data_series.index, plot_data_series.values, ticker, True, True
#                 else: trace.visible, trace.showlegend = False, False
#             benchmark_trace = fig.data[max_traces]
#             if results['benchmark_price_series'] is not None and not results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']].dropna().empty:
#                 normalized_benchmark = results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']] / results['benchmark_price_series'].loc[results['safe_start_date']:].bfill().iloc[0]
#                 benchmark_trace.x, benchmark_trace.y, benchmark_trace.name, benchmark_trace.visible = normalized_benchmark.index, normalized_benchmark, f"Benchmark ({benchmark_ticker})", True
#             else: benchmark_trace.visible = False
#             portfolio_trace = fig.data[max_traces + 1]
#             portfolio_trace.x, portfolio_trace.y, portfolio_trace.name, portfolio_trace.visible = results['portfolio_series'].index, results['portfolio_series'], 'Group Portfolio', True
#             fig.layout.shapes = []; fig.add_shape(type="line", x0=results['actual_calc_end_ts'], y0=0, x1=results['actual_calc_end_ts'], y1=1, xref='x', yref='paper', line=dict(color="grey", width=2, dash="dash"))
            
#         results_container[0] = results; debug_data_container[0] = debug_output
        
#         with ticker_list_output:
#             print(f"Analysis Period: {results['safe_start_date'].date()} to {results['safe_viz_end_date'].date()}.")
#             pprint.pprint(results['tickers_to_display'])
#             p = results['performance_data']
#             rows = []
#             rows.append({'Metric': 'Group Portfolio Gain', 'Full': p['full_p_gain'], 'Calc': p['calc_p_gain'], 'Fwd': p['fwd_p_gain']})
#             if not np.isnan(p['full_b_gain']):
#                 rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Gain', 'Full': p['full_b_gain'], 'Calc': p['calc_b_gain'], 'Fwd': p['fwd_b_gain']})
#                 rows.append({'Metric': 'Gain Delta (vs Bm)', 'Full': p['full_p_gain'] - p['full_b_gain'], 'Calc': p['calc_p_gain'] - p['calc_b_gain'], 'Fwd': p['fwd_p_gain'] - p['fwd_b_gain']})
#             rows.append({'Metric': 'Group Portfolio Sharpe', 'Full': p['full_p_sharpe'], 'Calc': p['calc_p_sharpe'], 'Fwd': p['fwd_p_sharpe']})
#             if not np.isnan(p['full_b_sharpe']):
#                 rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Sharpe', 'Full': p['full_b_sharpe'], 'Calc': p['calc_b_sharpe'], 'Fwd': p['fwd_b_sharpe']})
#                 rows.append({'Metric': 'Sharpe Delta (vs Bm)', 'Full': p['full_p_sharpe'] - p['full_b_sharpe'], 'Calc': p['calc_p_sharpe'] - p['calc_b_sharpe'], 'Fwd': p['fwd_p_sharpe'] - p['fwd_b_sharpe']})
#             report_df = pd.DataFrame(rows).set_index('Metric')
#             gain_rows = [row for row in report_df.index if 'Gain' in row or 'Delta' in row]
#             sharpe_rows = [row for row in report_df.index if 'Sharpe' in row]
#             styled_df = report_df.style.format('{:+.2%}', na_rep='N/A', subset=(gain_rows, report_df.columns)).format('{:+.2f}', na_rep='N/A', subset=(sharpe_rows, report_df.columns)).set_properties(**{'text-align': 'right', 'width': '100px'}).set_table_styles([{'selector': 'th.col_heading', 'props': [('text-align', 'right')]}, {'selector': 'th.row_heading', 'props': [('text-align', 'left')]}])
#             print("\n--- Strategy Performance Summary ---")
#             display(styled_df)
            
#     fig.update_layout(title_text='Walk-Forward Performance Analysis', xaxis_title='Date', yaxis_title='Normalized Price (Start = 1)', hovermode='x unified', legend_title_text='Tickers (Ranked)', height=600, margin=dict(t=50))
#     fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="grey")
#     update_button.on_click(update_plot)
#     controls_row1 = widgets.HBox([start_date_picker, calc_period_input, fwd_period_input])
#     controls_row2 = widgets.HBox([metric_dropdown, rank_start_input, rank_end_input, benchmark_ticker_input, update_button])
#     ui_container = widgets.VBox([controls_row1, controls_row2, ticker_list_output], layout=widgets.Layout(margin='10px 0 20px 0'))
#     display(ui_container, fig)
#     update_plot(None)
#     return (results_container, debug_data_container)


# def run_full_backtest(df_ohlcv, strategy_params, quality_thresholds):
#     print(f"--- Running Full Forensic Backtest for Strategy: {strategy_params['metric']} (Top {strategy_params['rank_start']}-{strategy_params['rank_end']}) ---")
    
#     # <<< CHANGED: Parameters are now integer trading days.
#     start_date, end_date = pd.to_datetime(strategy_params['start_date']), pd.to_datetime(strategy_params['end_date'])
#     calc_period, fwd_period = strategy_params['calc_period'], strategy_params['fwd_period']
#     metric, rank_start, rank_end = strategy_params['metric'], strategy_params['rank_start'], strategy_params['rank_end']
#     benchmark_ticker = strategy_params['benchmark_ticker']
#     master_calendar_ticker = strategy_params.get('master_calendar_ticker', 'VOO')
    
#     # <<< NEW: Create master calendar and find start/end indices for the whole backtest.
#     if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
#         raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
#     master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()
    
#     start_idx = master_trading_days.searchsorted(start_date)
#     end_idx = master_trading_days.searchsorted(end_date, side='right')
    
#     quality_metrics_df = calculate_rolling_quality_metrics(df_ohlcv, window=252)
#     df_close_full = df_ohlcv['Adj Close'].unstack(level=0); df_high_full = df_ohlcv['Adj High'].unstack(level=0); df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
    
#     # <<< CHANGED: The main loop now iterates through indices of the master calendar.
#     step_indices = range(start_idx, end_idx, fwd_period)
#     all_fwd_gains, period_by_period_debug = [], {}

#     print(f"Simulating {len(step_indices)} periods from {master_trading_days[step_indices[0]].date()} to {master_trading_days[step_indices[-1]].date()}...")
#     for current_idx in tqdm(step_indices, desc="Backtest Progress"):
#         step_date = master_trading_days[current_idx]
#         eligible_tickers = get_eligible_universe(quality_metrics_df, step_date, quality_thresholds)
#         if not eligible_tickers: continue
        
#         df_close_step = df_close_full[eligible_tickers]; df_high_step = df_high_full[eligible_tickers]; df_low_step = df_low_full[eligible_tickers]
        
#         results, debug_output = run_walk_forward_step(
#             df_close_step, df_high_step, df_low_step, master_trading_days,
#             step_date, calc_period, fwd_period,
#             metric, rank_start, rank_end, benchmark_ticker, debug=True
#         )
#         if results['error'] is None:
#             fwd_series = results['portfolio_series'].loc[results['actual_calc_end_ts']:]
#             all_fwd_gains.append(fwd_series.pct_change().dropna())
#             period_by_period_debug[step_date.date().isoformat()] = debug_output
            
#     if not all_fwd_gains:
#         print("Error: No valid periods were simulated."); return None

#     strategy_returns = pd.concat(all_fwd_gains); strategy_equity_curve = (1 + strategy_returns).cumprod()
#     benchmark_returns = df_close_full[benchmark_ticker].pct_change().loc[strategy_equity_curve.index]; benchmark_equity_curve = (1 + benchmark_returns).cumprod()
#     cumulative_equity_df = pd.DataFrame({'Strategy_Equity': strategy_equity_curve, 'Benchmark_Equity': benchmark_equity_curve})
    
#     fig = go.Figure()
#     fig.add_trace(go.Scatter(x=cumulative_equity_df.index, y=cumulative_equity_df['Strategy_Equity'], name='Strategy', line=dict(color='green')))
#     fig.add_trace(go.Scatter(x=cumulative_equity_df.index, y=cumulative_equity_df['Benchmark_Equity'], name=f'Benchmark ({benchmark_ticker})', line=dict(color='black', dash='dash')))
#     fig.update_layout(title=f"Cumulative Performance: '{metric}' Strategy (Top {rank_start}-{rank_end})", xaxis_title="Date", yaxis_title="Cumulative Growth")
#     fig.show()

#     final_backtest_results = {'cumulative_performance': cumulative_equity_df, 'period_by_period_debug': period_by_period_debug}
#     print("\n✅ Full backtest complete. Results object is ready for forensic analysis.")
#     return final_backtest_results


# # --- E. VERIFICATION TOOLS (Updated for Trading Day Logic & Detailed Metrics) ---
# ### Updated `E. VERIFICATION TOOLS` with CSV Export

# # def verify_group_tickers_walk_forward_calculation(df_ohlcv, tickers_to_verify, benchmark_ticker,
# #                                                   start_date, calc_period, fwd_period,
# #                                                   master_calendar_ticker='VOO', export_csv=False):
# #     """
# #     Verifies portfolio and benchmark performance using precise trading day periods.
# #     """
# #     display(Markdown(f"## Verification Report for Portfolio vs. Benchmark"))
# #     display(Markdown(f"**Portfolio Tickers:** `{tickers_to_verify}`\n**Benchmark Ticker:** `{benchmark_ticker}`"))
    
# #     # --- Date Calculation using Master Calendar ---
# #     if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
# #         raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
# #     master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()

# #     start_date_raw = pd.to_datetime(start_date)
# #     start_idx = master_trading_days.searchsorted(start_date_raw)
# #     if start_idx >= len(master_trading_days):
# #         print(f"Error: Start date {start_date_raw.date()} is after the last available trading day."); return
# #     actual_start_date = master_trading_days[start_idx]
    
# #     calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
# #     fwd_end_idx = min(calc_end_idx + fwd_period, len(master_trading_days) - 1)
    
# #     actual_calc_end_date = master_trading_days[calc_end_idx]
# #     actual_fwd_end_date = master_trading_days[fwd_end_idx]
    
# #     display(Markdown(f"**Analysis Start:** `{actual_start_date.date()}` (Selected: `{start_date_raw.date()}`)\n"
# #                     f"**Calc End:** `{actual_calc_end_date.date()}` ({calc_period} trading days)\n"
# #                     f"**Fwd End:** `{actual_fwd_end_date.date()}` ({fwd_period} trading days)"))

# #     # --- Verification Logic ---
# #     df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
# #     portfolio_prices_raw_slice = df_close_full[tickers_to_verify].loc[actual_start_date:actual_fwd_end_date]
# #     portfolio_value_series = portfolio_prices_raw_slice.div(portfolio_prices_raw_slice.bfill().iloc[0]).mean(axis=1)
# #     benchmark_price_series = df_close_full.get(benchmark_ticker)

# #     def print_verification_steps(title, price_series):
# #         display(Markdown(f"#### Verification for: `{title}`"))
# #         if price_series.dropna().shape[0] < 2: print("  - Not enough data points."); return {'gain': np.nan, 'sharpe': np.nan}
# #         start_price, end_price = price_series.bfill().iloc[0], price_series.ffill().iloc[-1]
# #         gain = (end_price / start_price) - 1
# #         print(f"Start Value ({price_series.first_valid_index().date()}): {start_price:,.4f}\nEnd Value   ({price_series.last_valid_index().date()}): {end_price:,.4f}\nGain = {gain:.2%}")
# #         returns = price_series.pct_change()
# #         sharpe = calculate_sharpe(returns)
# #         print(f"Mean Daily Return: {returns.mean():.6f}\nStd Dev: {returns.std():.6f}\nSharpe = {sharpe:.2f}")
# #         return {'gain': gain, 'sharpe': sharpe}

# #     display(Markdown("### A. Calculation Period"))
# #     perf_calc_p = print_verification_steps("Group Portfolio", portfolio_value_series.loc[actual_start_date:actual_calc_end_date])
# #     perf_calc_b = print_verification_steps(f"Benchmark", benchmark_price_series.loc[actual_start_date:actual_calc_end_date])
# #     display(Markdown("### B. Forward Period"))
# #     perf_fwd_p = print_verification_steps("Group Portfolio", portfolio_value_series.loc[actual_calc_end_date:actual_fwd_end_date])
# #     perf_fwd_b = print_verification_steps(f"Benchmark", benchmark_price_series.loc[actual_calc_end_date:actual_fwd_end_date])

# # def verify_ticker_ranking_metrics(df_ohlcv, ticker, start_date, calc_period,
# #                                   master_calendar_ticker='VOO', export_csv=False):
# #     """
# #     Verifies ranking metrics for a single ticker using precise trading day periods,
# #     and displays detailed intermediate calculations for ATR and ATRP.
# #     """
# #     display(Markdown(f"## Verification Report for Ticker Ranking: `{ticker}`"))
    
# #     # --- Date Calculation using Master Calendar ---
# #     if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
# #         raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
# #     master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()

# #     start_date_raw = pd.to_datetime(start_date)
# #     start_idx = master_trading_days.searchsorted(start_date_raw)
# #     if start_idx >= len(master_trading_days):
# #         print(f"Error: Start date {start_date_raw.date()} is after the last available trading day."); return
# #     actual_start_date = master_trading_days[start_idx]
    
# #     calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
# #     actual_calc_end_date = master_trading_days[calc_end_idx]

# #     df_ticker = df_ohlcv.loc[ticker].sort_index()
# #     calc_df = df_ticker.loc[actual_start_date:actual_calc_end_date].copy()
# #     if calc_df.empty or len(calc_df) < 2: 
# #         print("No data or not enough data in calc period."); return

# #     display(Markdown("### A. Calculation Period (for Ranking Metrics)"))
# #     display(Markdown(f"**Period Start:** `{actual_start_date.date()}`\n"
# #                     f"**Period End:** `{actual_calc_end_date.date()}`\n"
# #                     f"**Total Trading Days:** `{len(calc_df)}` (Requested: `{calc_period}`)"))
    
# #     # --- Detailed Calculation Breakdown ---
# #     display(Markdown("#### Detailed Metric Calculation Data"))
    
# #     # Use a temporary DataFrame for clarity
# #     vdf = calc_df[['Adj High', 'Adj Low', 'Adj Close']].copy()
    
# #     # 1. Daily Return
# #     vdf['Daily_Return'] = vdf['Adj Close'].pct_change()
    
# #     # 2. True Range (TR)
# #     vdf['TR'] = np.maximum(vdf['Adj High'] - vdf['Adj Low'], 
# #                            abs(vdf['Adj High'] - vdf['Adj Close'].shift(1)),
# #                            abs(vdf['Adj Low'] - vdf['Adj Close'].shift(1)))
                           
# #     # 3. Average True Range (ATR) - 14 period exponential moving average
# #     vdf['ATR_14'] = vdf['TR'].ewm(alpha=1/14, adjust=False).mean()
    
# #     # 4. ATR Percent (ATRP)
# #     vdf['ATRP'] = vdf['ATR_14'] / vdf['Adj Close']
    
# #     # Display the first and last few rows of the calculation
# #     print("--- Start of Calculation Period ---")
# #     display(vdf.head())
# #     print("\n--- End of Calculation Period ---")
# #     display(vdf.tail())
    
# #     # --- Final Metric Summary ---
# #     display(Markdown("#### `MetricValue` Verification Summary:"))
    
# #     # 1. Price Metric
# #     calc_start_price = vdf['Adj Close'].bfill().iloc[0]
# #     calc_end_price = vdf['Adj Close'].ffill().iloc[-1]
# #     price_metric = (calc_end_price / calc_start_price)
# #     print(f"1. Price Metric: (Last Price / First Price) = ({calc_end_price:.2f} / {calc_start_price:.2f}) = {price_metric:.4f}")
    
# #     # 2. Sharpe Metric
# #     daily_returns = vdf['Daily_Return'].dropna()
# #     sharpe_ratio = calculate_sharpe(daily_returns)
# #     print(f"2. Sharpe Metric: (Mean Daily Return / Std Dev) * sqrt(252) = {sharpe_ratio:.4f}")

# #     # 3. Sharpe (ATR) Metric
# #     atrp_mean = vdf['ATRP'].mean()
# #     mean_daily_return = vdf['Daily_Return'].mean()
# #     sharpe_atr = (mean_daily_return / atrp_mean) if atrp_mean > 0 else 0
# #     print(f"3. Sharpe (ATR) Metric: (Mean Daily Return / Mean ATRP) = ({mean_daily_return:.6f} / {atrp_mean:.6f}) = {sharpe_atr:.4f}")


# def verify_group_tickers_walk_forward_calculation(df_ohlcv, tickers_to_verify, benchmark_ticker,
#                                                   start_date, calc_period, fwd_period,
#                                                   master_calendar_ticker='VOO', export_csv=False):
#     """
#     Verifies portfolio and benchmark performance using precise trading day periods.
#     """
#     display(Markdown(f"## Verification Report for Portfolio vs. Benchmark"))
#     display(Markdown(f"**Portfolio Tickers:** `{tickers_to_verify}`\n**Benchmark Ticker:** `{benchmark_ticker}`"))
    
#     # --- Date Calculation using Master Calendar ---
#     if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
#         raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
#     master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()

#     start_date_raw = pd.to_datetime(start_date)
#     start_idx = master_trading_days.searchsorted(start_date_raw)
#     if start_idx >= len(master_trading_days):
#         print(f"Error: Start date {start_date_raw.date()} is after the last available trading day."); return
#     actual_start_date = master_trading_days[start_idx]
    
#     calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
#     fwd_end_idx = min(calc_end_idx + fwd_period, len(master_trading_days) - 1)
    
#     actual_calc_end_date = master_trading_days[calc_end_idx]
#     actual_fwd_end_date = master_trading_days[fwd_end_idx]
    
#     display(Markdown(f"**Analysis Start:** `{actual_start_date.date()}` (Selected: `{start_date_raw.date()}`)\n"
#                     f"**Calc End:** `{actual_calc_end_date.date()}` ({calc_period} trading days)\n"
#                     f"**Fwd End:** `{actual_fwd_end_date.date()}` ({fwd_period} trading days)"))

#     # --- Verification Logic ---
#     df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
#     portfolio_prices_raw_slice = df_close_full[tickers_to_verify].loc[actual_start_date:actual_fwd_end_date]
#     portfolio_value_series = portfolio_prices_raw_slice.div(portfolio_prices_raw_slice.bfill().iloc[0]).mean(axis=1)
#     benchmark_price_series = df_close_full.get(benchmark_ticker)
    
#     # <<< NEW: CSV Export Logic Added Here ---
#     if export_csv:
#         # Prepare a detailed DataFrame for export
#         export_df = pd.DataFrame({
#             'Portfolio_Normalized_Price': portfolio_value_series,
#             'Portfolio_Daily_Return': portfolio_value_series.pct_change()
#         })
#         if benchmark_price_series is not None:
#             norm_bm = benchmark_price_series.loc[actual_start_date:actual_fwd_end_date]
#             norm_bm = norm_bm / norm_bm.bfill().iloc[0]
#             export_df['Benchmark_Normalized_Price'] = norm_bm
#             export_df['Benchmark_Daily_Return'] = norm_bm.pct_change()

#         # Create directory and save file
#         output_dir = 'export_csv'
#         os.makedirs(output_dir, exist_ok=True)
#         tickers_str = '_'.join(tickers_to_verify)
#         filename = f"verify_group_{actual_start_date.date()}_{tickers_str}.csv"
#         filepath = os.path.join(output_dir, filename)
#         export_df.to_csv(filepath)
#         print(f"\n✅ Data exported to: {filepath}")
#     # --- End of New CSV Logic ---

#     def print_verification_steps(title, price_series):
#         display(Markdown(f"#### Verification for: `{title}`"))
#         if price_series.dropna().shape[0] < 2: print("  - Not enough data points."); return {'gain': np.nan, 'sharpe': np.nan}
#         start_price, end_price = price_series.bfill().iloc[0], price_series.ffill().iloc[-1]
#         gain = (end_price / start_price) - 1
#         print(f"Start Value ({price_series.first_valid_index().date()}): {start_price:,.4f}\nEnd Value   ({price_series.last_valid_index().date()}): {end_price:,.4f}\nGain = {gain:.2%}")
#         returns = price_series.pct_change()
#         sharpe = calculate_sharpe(returns)
#         print(f"Mean Daily Return: {returns.mean():.6f}\nStd Dev: {returns.std():.6f}\nSharpe = {sharpe:.2f}")
#         return {'gain': gain, 'sharpe': sharpe}

#     display(Markdown("### A. Calculation Period"))
#     perf_calc_p = print_verification_steps("Group Portfolio", portfolio_value_series.loc[actual_start_date:actual_calc_end_date])
#     perf_calc_b = print_verification_steps(f"Benchmark", benchmark_price_series.loc[actual_start_date:actual_calc_end_date])
#     display(Markdown("### B. Forward Period"))
#     perf_fwd_p = print_verification_steps("Group Portfolio", portfolio_value_series.loc[actual_calc_end_date:actual_fwd_end_date])
#     perf_fwd_b = print_verification_steps(f"Benchmark", benchmark_price_series.loc[actual_calc_end_date:actual_fwd_end_date])


# def verify_ticker_ranking_metrics(df_ohlcv, ticker, start_date, calc_period,
#                                   master_calendar_ticker='VOO', export_csv=False):
#     """
#     Verifies ranking metrics for a single ticker using precise trading day periods,
#     and displays detailed intermediate calculations for ATR and ATRP.
#     """
#     display(Markdown(f"## Verification Report for Ticker Ranking: `{ticker}`"))
    
#     # --- Date Calculation using Master Calendar ---
#     if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
#         raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
#     master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()

#     start_date_raw = pd.to_datetime(start_date)
#     start_idx = master_trading_days.searchsorted(start_date_raw)
#     if start_idx >= len(master_trading_days):
#         print(f"Error: Start date {start_date_raw.date()} is after the last available trading day."); return
#     actual_start_date = master_trading_days[start_idx]
    
#     calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
#     actual_calc_end_date = master_trading_days[calc_end_idx]

#     df_ticker = df_ohlcv.loc[ticker].sort_index()
#     calc_df = df_ticker.loc[actual_start_date:actual_calc_end_date].copy()
#     if calc_df.empty or len(calc_df) < 2: 
#         print("No data or not enough data in calc period."); return

#     display(Markdown("### A. Calculation Period (for Ranking Metrics)"))
#     display(Markdown(f"**Period Start:** `{actual_start_date.date()}`\n"
#                     f"**Period End:** `{actual_calc_end_date.date()}`\n"
#                     f"**Total Trading Days:** `{len(calc_df)}` (Requested: `{calc_period}`)"))
    
#     # --- Detailed Calculation Breakdown ---
#     display(Markdown("#### Detailed Metric Calculation Data"))
    
#     vdf = calc_df[['Adj High', 'Adj Low', 'Adj Close']].copy()
#     vdf['Daily_Return'] = vdf['Adj Close'].pct_change()
#     # <<< REFACTORED: Replaced the block of TR logic with a single call to the new helper function.
#     vdf['TR'] = calculate_true_range(vdf['Adj High'], vdf['Adj Low'], vdf['Adj Close'])
#     vdf['ATR_14'] = vdf['TR'].ewm(alpha=1/14, adjust=False).mean()
#     vdf['ATRP'] = vdf['ATR_14'] / vdf['Adj Close']
    
#     print("--- Start of Calculation Period ---")
#     display(vdf.head())
#     print("\n--- End of Calculation Period ---")
#     display(vdf.tail())

#     # <<< NEW: CSV Export Logic Added Here ---
#     if export_csv:
#         output_dir = 'export_csv'
#         os.makedirs(output_dir, exist_ok=True)
#         filename = f"verify_ticker_{actual_start_date.date()}_{ticker}.csv"
#         filepath = os.path.join(output_dir, filename)
#         vdf.to_csv(filepath)
#         print(f"\n✅ Data exported to: {filepath}")
#     # --- End of New CSV Logic ---
    
#     # --- Final Metric Summary ---
#     display(Markdown("#### `MetricValue` Verification Summary:"))
    
#     calc_start_price = vdf['Adj Close'].bfill().iloc[0]
#     calc_end_price = vdf['Adj Close'].ffill().iloc[-1]
#     price_metric = (calc_end_price / calc_start_price)
#     print(f"1. Price Metric: (Last Price / First Price) = ({calc_end_price:.2f} / {calc_start_price:.2f}) = {price_metric:.4f}")
    
#     daily_returns = vdf['Daily_Return'].dropna()
#     sharpe_ratio = calculate_sharpe(daily_returns)
#     print(f"2. Sharpe Metric: (Mean Daily Return / Std Dev) * sqrt(252) = {sharpe_ratio:.4f}")

#     atrp_mean = vdf['ATRP'].mean()
#     mean_daily_return = vdf['Daily_Return'].mean()
#     sharpe_atr = (mean_daily_return / atrp_mean) if atrp_mean > 0 else 0
#     print(f"3. Sharpe (ATR) Metric: (Mean Daily Return / Mean ATRP) = ({mean_daily_return:.6f} / {atrp_mean:.6f}) = {sharpe_atr:.4f}")



In [7]:
# # ==============================================================================
# # FINAL PROJECT FUNCTIONS (Golden Copy - TR Calculation Fixed)
# # ==============================================================================

# import pandas as pd
# import numpy as np
# import plotly.graph_objects as go
# from datetime import datetime, date
# import ipywidgets as widgets
# from IPython.display import display, Markdown
# import pprint
# from tqdm.auto import tqdm
# import re
# import os

# # --- A. HELPER FUNCTIONS ---

# def calculate_gain(price_series: pd.Series):
#     if price_series.dropna().shape[0] < 2: return np.nan
#     return (price_series.ffill().iloc[-1] / price_series.bfill().iloc[0]) - 1

# def calculate_sharpe(return_series: pd.Series):
#     if return_series.dropna().shape[0] < 2: return np.nan
#     std_dev = return_series.std()
#     if std_dev > 0 and std_dev != np.inf:
#         return (return_series.mean() / std_dev) * np.sqrt(252)
#     return np.nan

# # --- B. THE CORE CALCULATION ENGINE ---

# def run_walk_forward_step(df_close_full, df_high_full, df_low_full,
#                           master_trading_days,
#                           start_date, calc_period, fwd_period,
#                           metric, rank_start, rank_end, benchmark_ticker,
#                           debug=False):
#     debug_data = {} if debug else None
    
#     try:
#         start_idx = master_trading_days.get_loc(start_date)
#     except KeyError:
#         return ({'error': f"Start date {start_date.date()} is not a valid trading day."}, None)
        
#     calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
#     viz_end_idx = min(calc_end_idx + fwd_period, len(master_trading_days) - 1)

#     safe_start_date = master_trading_days[start_idx]
#     safe_calc_end_date = master_trading_days[calc_end_idx]
#     safe_viz_end_date = master_trading_days[viz_end_idx]
    
#     if safe_start_date >= safe_calc_end_date:
#         return ({'error': "Invalid date range (calc period has zero or negative length)."}, None)

#     calc_close_raw = df_close_full.loc[safe_start_date:safe_calc_end_date]
#     calc_close = calc_close_raw.dropna(axis=1, how='all')
#     if calc_close.shape[1] == 0 or len(calc_close) < 2:
#         return ({'error': "Not enough data in calc period."}, None)

#     first_prices = calc_close.bfill().iloc[0]
#     last_prices = calc_close.ffill().iloc[-1]
#     daily_returns = calc_close.bfill().ffill().pct_change()
#     mean_returns = daily_returns.mean()
#     std_returns = daily_returns.std()
    
#     valid_tickers = calc_close.columns
#     calc_high = df_high_full[valid_tickers].loc[safe_start_date:safe_calc_end_date]
#     calc_low = df_low_full[valid_tickers].loc[safe_start_date:safe_calc_end_date]
    
#     # <<< BUG FIX: Reverted to a robust, in-place True Range calculation to prevent index errors.
#     # This logic is now self-contained and guaranteed to be aligned.
#     prev_close = df_close_full[valid_tickers].shift(1).loc[safe_start_date:safe_calc_end_date]
    
#     tr_df = pd.DataFrame({
#         'h_l': calc_high - calc_low,
#         'h_pc': abs(calc_high - prev_close),
#         'l_pc': abs(calc_low - prev_close)
#     })
#     tr = tr_df.max(axis=1)
    
#     atr = tr.ewm(alpha=1/14, adjust=False).mean()
#     atrp = (atr / calc_close).mean()

#     metric_values = {}
#     metric_values['Price'] = (last_prices / first_prices).dropna()
#     metric_values['Sharpe'] = (mean_returns / std_returns * np.sqrt(252)).fillna(0)
#     metric_values['Sharpe (ATR)'] = (mean_returns / atrp).fillna(0)
    
#     if debug:
#         df_ranking = pd.DataFrame({
#             'FirstPrice': first_prices, 'LastPrice': last_prices, 'MeanDailyReturn': mean_returns,
#             'StdDevDailyReturn': std_returns, 'MeanATRP': atrp, 'Metric_Price': metric_values['Price'],
#             'Metric_Sharpe': metric_values['Sharpe'], 'Metric_Sharpe (ATR)': metric_values['Sharpe (ATR)']
#         })
#         df_ranking.index.name = 'Ticker'
#         debug_data['ranking_metrics'] = df_ranking.sort_values(f'Metric_{metric}', ascending=False)

#     sorted_tickers = metric_values[metric].sort_values(ascending=False)
#     tickers_to_display = sorted_tickers.index[rank_start-1:rank_end].tolist()
#     if not tickers_to_display:
#         return ({'error': "No tickers found for the selected rank."}, None)

#     normalized_plot_data = df_close_full[tickers_to_display].loc[safe_start_date:safe_viz_end_date]
#     normalized_plot_data = normalized_plot_data.div(normalized_plot_data.bfill().iloc[0])
#     actual_calc_end_ts = calc_close.index.max()
#     portfolio_series = normalized_plot_data.mean(axis=1)
#     portfolio_return_series = portfolio_series.pct_change()
#     benchmark_price_series = df_close_full.get(benchmark_ticker)
#     benchmark_return_series = benchmark_price_series.loc[safe_start_date:safe_viz_end_date].bfill().ffill().pct_change() if benchmark_price_series is not None else pd.Series(dtype='float64')

#     try:
#         boundary_loc = portfolio_return_series.index.get_loc(actual_calc_end_ts)
#         calc_portfolio_returns = portfolio_return_series.iloc[:boundary_loc + 1]
#         fwd_portfolio_returns = portfolio_return_series.iloc[boundary_loc + 1:]
#         bm_boundary_loc = benchmark_return_series.index.get_loc(actual_calc_end_ts)
#         calc_benchmark_returns = benchmark_return_series.iloc[:bm_boundary_loc + 1]
#         fwd_benchmark_returns = benchmark_return_series.iloc[bm_boundary_loc + 1:]
#     except (KeyError, IndexError):
#         calc_portfolio_returns = portfolio_return_series.loc[:actual_calc_end_ts]
#         fwd_portfolio_returns = portfolio_return_series.loc[actual_calc_end_ts:].iloc[1:]
#         calc_benchmark_returns = benchmark_return_series.loc[:actual_calc_end_ts]
#         fwd_benchmark_returns = benchmark_return_series.loc[actual_calc_end_ts:].iloc[1:]

#     perf_data = {}
#     perf_data['calc_p_gain'] = calculate_gain(portfolio_series.loc[:actual_calc_end_ts])
#     perf_data['fwd_p_gain'] = calculate_gain(portfolio_series.loc[actual_calc_end_ts:])
#     perf_data['full_p_gain'] = calculate_gain(portfolio_series)
#     perf_data['calc_p_sharpe'] = calculate_sharpe(calc_portfolio_returns)
#     perf_data['fwd_p_sharpe'] = calculate_sharpe(fwd_portfolio_returns)
#     perf_data['full_p_sharpe'] = calculate_sharpe(portfolio_return_series)
#     perf_data['calc_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:actual_calc_end_ts]) if benchmark_price_series is not None else np.nan
#     perf_data['fwd_b_gain'] = calculate_gain(benchmark_price_series.loc[actual_calc_end_ts:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
#     perf_data['full_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
#     perf_data['calc_b_sharpe'] = calculate_sharpe(calc_benchmark_returns)
#     perf_data['fwd_b_sharpe'] = calculate_sharpe(fwd_benchmark_returns)
#     perf_data['full_b_sharpe'] = calculate_sharpe(benchmark_return_series)

#     calc_end_prices = calc_close.ffill().iloc[-1]
#     fwd_close_slice = df_close_full.loc[actual_calc_end_ts:safe_viz_end_date]
#     viz_end_prices = fwd_close_slice.ffill().iloc[-1] if not fwd_close_slice.empty and len(fwd_close_slice) >= 2 else calc_end_prices
#     calc_gains = (calc_end_prices / calc_close.bfill().iloc[0]) - 1
#     fwd_gains = (viz_end_prices / calc_end_prices) - 1
#     results_df = pd.DataFrame({'Rank': range(rank_start, rank_start + len(tickers_to_display)), 'Metric': metric, 'MetricValue': sorted_tickers.loc[tickers_to_display].values, 'CalcPrice': calc_end_prices.loc[tickers_to_display], 'CalcGain': calc_gains.loc[tickers_to_display], 'FwdGain': fwd_gains.loc[tickers_to_display]}, index=pd.Index(tickers_to_display, name='Ticker'))
#     if benchmark_price_series is not None and benchmark_ticker in calc_close.columns:
#         benchmark_df_row = pd.DataFrame({'Rank': np.nan, 'Metric': metric, 'MetricValue': metric_values[metric].get(benchmark_ticker, np.nan), 'CalcPrice': calc_end_prices[benchmark_ticker], 'CalcGain': calc_gains[benchmark_ticker], 'FwdGain': fwd_gains[benchmark_ticker]}, index=pd.Index([f"{benchmark_ticker} (BM)"], name='Ticker'))
#         results_df = pd.concat([results_df, benchmark_df_row])
    
#     if debug:
#         df_trace = normalized_plot_data.copy()
#         df_trace.columns = [f'Norm_Price_{c}' for c in df_trace.columns]
#         df_trace['Norm_Price_Portfolio'] = portfolio_series
#         if benchmark_price_series is not None and not benchmark_price_series.loc[safe_start_date:safe_viz_end_date].dropna().empty:
#             norm_bm = benchmark_price_series.loc[safe_start_date:safe_viz_end_date] / benchmark_price_series.loc[safe_start_date:].bfill().iloc[0]
#             df_trace[f'Norm_Price_Benchmark_{benchmark_ticker}'] = norm_bm
#         for col in df_trace.columns:
#             if 'Norm_Price' in col:
#                 df_trace[col.replace('Norm_Price', 'Return')] = df_trace[col].pct_change()
#         debug_data['portfolio_trace'] = df_trace

#     final_results = {
#         'tickers_to_display': tickers_to_display, 'normalized_plot_data': normalized_plot_data,
#         'portfolio_series': portfolio_series, 'benchmark_price_series': benchmark_price_series,
#         'performance_data': perf_data, 'results_df': results_df, 'actual_calc_end_ts': actual_calc_end_ts,
#         'safe_start_date': safe_start_date, 'safe_viz_end_date': safe_viz_end_date,
#         'error': None
#     }
#     return (final_results, debug_data)

# # --- C. DYNAMIC DATA QUALITY FILTER FUNCTIONS ---

# def calculate_rolling_quality_metrics(df_ohlcv, window=252, min_periods=126, debug=False):
#     print(f"--- Calculating Rolling Quality Metrics (Window: {window} days) ---")
#     df = df_ohlcv.copy()
#     if not df.index.is_monotonic_increasing:
#         df.sort_index(inplace=True)
#     df['IsStale'] = np.where((df['Volume'] == 0) | (df['Adj High'] == df['Adj Low']), 1, 0)
#     df['DollarVolume'] = df['Adj Close'] * df['Volume']
#     df['HasSameVolumeAsPrevDay'] = (df.groupby(level='Ticker')['Volume'].diff() == 0).astype(int)
#     grouped = df.groupby(level='Ticker')
#     stale_pct = grouped['IsStale'].rolling(window=window, min_periods=min_periods).mean()
#     median_vol = grouped['DollarVolume'].rolling(window=window, min_periods=min_periods).median()
#     same_vol_count = grouped['HasSameVolumeAsPrevDay'].rolling(window=window, min_periods=min_periods).sum()
#     quality_df = pd.concat([stale_pct, median_vol, same_vol_count], axis=1)
#     quality_df.columns = ['RollingStalePct', 'RollingMedianVolume', 'RollingSameVolCount']
#     quality_df.index = quality_df.index.droplevel(0)
#     print("✅ Rolling metrics calculation complete.")
#     return quality_df

# def get_eligible_universe(quality_metrics_df, filter_date, thresholds):
#     filter_date_ts = pd.to_datetime(filter_date)
#     date_index = quality_metrics_df.index.get_level_values('Date').unique().sort_values()
#     if filter_date_ts < date_index[0]:
#         print(f"Warning: Filter date {filter_date_ts.date()} is before the earliest data point. Returning empty universe.")
#         return []
#     valid_prior_dates = date_index[date_index <= filter_date_ts]
#     if valid_prior_dates.empty:
#         print(f"Warning: No available data found on or before {filter_date_ts.date()}. Returning empty universe.")
#         return []
#     actual_date_to_use = valid_prior_dates[-1]
#     if actual_date_to_use.date() != filter_date_ts.date():
#         print(f"ℹ️ Info: Filter date {filter_date_ts.date()} not found. Using previous available date {actual_date_to_use.date()}.")
#     metrics_on_date = quality_metrics_df.xs(actual_date_to_use, level='Date')
#     mask = ((metrics_on_date['RollingMedianVolume'] >= thresholds['min_median_dollar_volume']) &
#             (metrics_on_date['RollingStalePct'] <= thresholds['max_stale_pct']) &
#             (metrics_on_date['RollingSameVolCount'] <= thresholds['max_same_vol_count']))
#     eligible_tickers = metrics_on_date[mask].index.tolist()
#     all_tickers = metrics_on_date.index.tolist()
#     print(f"Dynamic Filter ({filter_date_ts.date()}): Kept {len(eligible_tickers)} of {len(all_tickers)} tickers.")
#     return eligible_tickers    

# # --- D. INTERACTIVE ANALYSIS & BACKTESTING TOOLS ---

# def plot_walk_forward_analyzer(df_ohlcv, 
#                                default_start_date=None, default_calc_period=126, default_fwd_period=63,
#                                default_metric='Sharpe (ATR)', default_rank_start=1, default_rank_end=10,
#                                default_benchmark_ticker='VOO', master_calendar_ticker='VOO',
#                                quality_thresholds={'min_median_dollar_volume': 1_000_000, 'max_stale_pct': 0.05, 'max_same_vol_count': 10},
#                                debug=False):
#     print("Initializing Walk-Forward Analyzer (using Trading Day Logic)...")
#     if not isinstance(df_ohlcv.index, pd.MultiIndex): raise ValueError("Input DataFrame must have a (Ticker, Date) MultiIndex.")
#     df_ohlcv = df_ohlcv.sort_index()

#     if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
#         raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
#     master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()
#     print(f"Master trading day calendar created from '{master_calendar_ticker}' ({len(master_trading_days)} days).")

#     print("Pre-calculating data quality metrics...")
#     quality_metrics_df = calculate_rolling_quality_metrics(df_ohlcv, window=252)
#     print("Pre-processing data (unstacking)...")
#     df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
#     df_high_full = df_ohlcv['Adj High'].unstack(level=0)
#     df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
    
#     start_date_picker = widgets.DatePicker(description='Start Date:', value=pd.to_datetime(default_start_date), disabled=False)
#     calc_period_input = widgets.IntText(value=default_calc_period, description='Calc Period (days):')
#     fwd_period_input = widgets.IntText(value=default_fwd_period, description='Fwd Period (days):')
#     metrics = ['Price', 'Sharpe', 'Sharpe (ATR)']
#     metric_dropdown = widgets.Dropdown(options=metrics, value=default_metric, description='Metric:')
#     rank_start_input = widgets.IntText(value=default_rank_start, description='Rank Start:')
#     rank_end_input = widgets.IntText(value=default_rank_end, description='Rank End:')
#     benchmark_ticker_input = widgets.Text(value=default_benchmark_ticker, description='Benchmark:', placeholder='Enter Ticker')
#     update_button = widgets.Button(description="Update Chart", button_style='primary')
#     ticker_list_output = widgets.Output()
#     results_container, debug_data_container = [None], [None]

#     fig = go.FigureWidget()
#     max_traces = 50
#     for i in range(max_traces): fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'placeholder_{i}', visible=False, showlegend=False))
#     fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Benchmark', visible=True, showlegend=True, line=dict(color='black', width=3, dash='dash')))
#     fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Group Portfolio', visible=True, showlegend=True, line=dict(color='green', width=3)))

#     def update_plot(button_click):
#         ticker_list_output.clear_output()
#         start_date_raw = pd.to_datetime(start_date_picker.value)
#         start_date_idx = master_trading_days.searchsorted(start_date_raw)
#         if start_date_idx >= len(master_trading_days):
#             with ticker_list_output: print(f"Error: Start date is after the last available trading day."); return
#         actual_start_date = master_trading_days[start_date_idx]
#         with ticker_list_output: 
#             if start_date_raw.date() != actual_start_date.date():
#                 print(f"ℹ️ Info: Start date {start_date_raw.date()} is not a trading day. Snapping forward to {actual_start_date.date()}.")

#         calc_period, fwd_period = calc_period_input.value, fwd_period_input.value
#         metric = metric_dropdown.value
#         rank_start, rank_end = rank_start_input.value, rank_end_input.value
#         benchmark_ticker = benchmark_ticker_input.value.strip().upper()
        
#         if rank_start > rank_end:
#             with ticker_list_output: print("Error: 'Rank Start' must be <= 'Rank End'."); return
#         if rank_start < 1 or calc_period < 2 or fwd_period < 1:
#             with ticker_list_output: print("Error: Ranks must be >= 1, Calc Period >= 2, Fwd Period >= 1."); return

#         eligible_tickers = get_eligible_universe(quality_metrics_df, actual_start_date, quality_thresholds)
#         if not eligible_tickers:
#             with ticker_list_output: print(f"Error: No eligible tickers found on {actual_start_date.date()} with the current quality filters."); return
#         df_close_step = df_close_full[eligible_tickers]; df_high_step = df_high_full[eligible_tickers]; df_low_step = df_low_full[eligible_tickers]

#         results, debug_output = run_walk_forward_step(
#             df_close_step, df_high_step, df_low_step, master_trading_days,
#             actual_start_date, calc_period, fwd_period, 
#             metric, rank_start, rank_end, benchmark_ticker, debug=debug
#         )
#         if results['error']:
#             with ticker_list_output: print(f"Error: {results['error']}"); return

#         with fig.batch_update():
#             for i in range(max_traces):
#                 trace = fig.data[i]
#                 if i < len(results['tickers_to_display']):
#                     ticker = results['tickers_to_display'][i]; plot_data_series = results['normalized_plot_data'][ticker]
#                     trace.x, trace.y, trace.name, trace.visible, trace.showlegend = plot_data_series.index, plot_data_series.values, ticker, True, True
#                 else: trace.visible, trace.showlegend = False, False
#             benchmark_trace = fig.data[max_traces]
#             if results['benchmark_price_series'] is not None and not results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']].dropna().empty:
#                 normalized_benchmark = results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']] / results['benchmark_price_series'].loc[results['safe_start_date']:].bfill().iloc[0]
#                 benchmark_trace.x, benchmark_trace.y, benchmark_trace.name, benchmark_trace.visible = normalized_benchmark.index, normalized_benchmark, f"Benchmark ({benchmark_ticker})", True
#             else: benchmark_trace.visible = False
#             portfolio_trace = fig.data[max_traces + 1]
#             portfolio_trace.x, portfolio_trace.y, portfolio_trace.name, portfolio_trace.visible = results['portfolio_series'].index, results['portfolio_series'], 'Group Portfolio', True
#             fig.layout.shapes = []; fig.add_shape(type="line", x0=results['actual_calc_end_ts'], y0=0, x1=results['actual_calc_end_ts'], y1=1, xref='x', yref='paper', line=dict(color="grey", width=2, dash="dash"))
            
#         results_container[0] = results; debug_data_container[0] = debug_output
        
#         with ticker_list_output:
#             print(f"Analysis Period: {results['safe_start_date'].date()} to {results['safe_viz_end_date'].date()}.")
#             pprint.pprint(results['tickers_to_display'])
#             p = results['performance_data']
#             rows = []
#             rows.append({'Metric': 'Group Portfolio Gain', 'Full': p['full_p_gain'], 'Calc': p['calc_p_gain'], 'Fwd': p['fwd_p_gain']})
#             if not np.isnan(p['full_b_gain']):
#                 rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Gain', 'Full': p['full_b_gain'], 'Calc': p['calc_b_gain'], 'Fwd': p['fwd_b_gain']})
#                 rows.append({'Metric': 'Gain Delta (vs Bm)', 'Full': p['full_p_gain'] - p['full_b_gain'], 'Calc': p['calc_p_gain'] - p['calc_b_gain'], 'Fwd': p['fwd_p_gain'] - p['fwd_b_gain']})
#             rows.append({'Metric': 'Group Portfolio Sharpe', 'Full': p['full_p_sharpe'], 'Calc': p['calc_p_sharpe'], 'Fwd': p['fwd_p_sharpe']})
#             if not np.isnan(p['full_b_sharpe']):
#                 rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Sharpe', 'Full': p['full_b_sharpe'], 'Calc': p['calc_b_sharpe'], 'Fwd': p['fwd_b_sharpe']})
#                 rows.append({'Metric': 'Sharpe Delta (vs Bm)', 'Full': p['full_p_sharpe'] - p['full_b_sharpe'], 'Calc': p['calc_p_sharpe'] - p['calc_b_sharpe'], 'Fwd': p['fwd_p_sharpe'] - p['fwd_b_sharpe']})
#             report_df = pd.DataFrame(rows).set_index('Metric')
#             gain_rows = [row for row in report_df.index if 'Gain' in row or 'Delta' in row]
#             sharpe_rows = [row for row in report_df.index if 'Sharpe' in row]
#             styled_df = report_df.style.format('{:+.2%}', na_rep='N/A', subset=(gain_rows, report_df.columns)).format('{:+.2f}', na_rep='N/A', subset=(sharpe_rows, report_df.columns)).set_properties(**{'text-align': 'right', 'width': '100px'}).set_table_styles([{'selector': 'th.col_heading', 'props': [('text-align', 'right')]}, {'selector': 'th.row_heading', 'props': [('text-align', 'left')]}])
#             print("\n--- Strategy Performance Summary ---")
#             display(styled_df)
            
#     fig.update_layout(title_text='Walk-Forward Performance Analysis', xaxis_title='Date', yaxis_title='Normalized Price (Start = 1)', hovermode='x unified', legend_title_text='Tickers (Ranked)', height=600, margin=dict(t=50))
#     fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="grey")
#     update_button.on_click(update_plot)
#     controls_row1 = widgets.HBox([start_date_picker, calc_period_input, fwd_period_input])
#     controls_row2 = widgets.HBox([metric_dropdown, rank_start_input, rank_end_input, benchmark_ticker_input, update_button])
#     ui_container = widgets.VBox([controls_row1, controls_row2, ticker_list_output], layout=widgets.Layout(margin='10px 0 20px 0'))
#     display(ui_container, fig)
#     update_plot(None)
#     return (results_container, debug_data_container)

# def run_full_backtest(df_ohlcv, strategy_params, quality_thresholds):
#     print(f"--- Running Full Forensic Backtest for Strategy: {strategy_params['metric']} (Top {strategy_params['rank_start']}-{strategy_params['rank_end']}) ---")
    
#     start_date, end_date = pd.to_datetime(strategy_params['start_date']), pd.to_datetime(strategy_params['end_date'])
#     calc_period, fwd_period = strategy_params['calc_period'], strategy_params['fwd_period']
#     metric, rank_start, rank_end = strategy_params['metric'], strategy_params['rank_start'], strategy_params['rank_end']
#     benchmark_ticker = strategy_params['benchmark_ticker']
#     master_calendar_ticker = strategy_params.get('master_calendar_ticker', 'VOO')
    
#     if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
#         raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
#     master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()
    
#     start_idx = master_trading_days.searchsorted(start_date)
#     end_idx = master_trading_days.searchsorted(end_date, side='right')
    
#     quality_metrics_df = calculate_rolling_quality_metrics(df_ohlcv, window=252)
#     df_close_full = df_ohlcv['Adj Close'].unstack(level=0); df_high_full = df_ohlcv['Adj High'].unstack(level=0); df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
    
#     step_indices = range(start_idx, end_idx, fwd_period)
#     all_fwd_gains, period_by_period_debug = [], {}

#     print(f"Simulating {len(step_indices)} periods from {master_trading_days[step_indices[0]].date()} to {master_trading_days[step_indices[-1]].date()}...")
#     for current_idx in tqdm(step_indices, desc="Backtest Progress"):
#         step_date = master_trading_days[current_idx]
#         eligible_tickers = get_eligible_universe(quality_metrics_df, step_date, quality_thresholds)
#         if not eligible_tickers: continue
        
#         df_close_step = df_close_full[eligible_tickers]; df_high_step = df_high_full[eligible_tickers]; df_low_step = df_low_full[eligible_tickers]
        
#         results, debug_output = run_walk_forward_step(
#             df_close_step, df_high_step, df_low_step, master_trading_days,
#             step_date, calc_period, fwd_period,
#             metric, rank_start, rank_end, benchmark_ticker, debug=True
#         )
#         if results['error'] is None:
#             fwd_series = results['portfolio_series'].loc[results['actual_calc_end_ts']:]
#             all_fwd_gains.append(fwd_series.pct_change().dropna())
#             period_by_period_debug[step_date.date().isoformat()] = debug_output
            
#     if not all_fwd_gains:
#         print("Error: No valid periods were simulated."); return None

#     strategy_returns = pd.concat(all_fwd_gains); strategy_equity_curve = (1 + strategy_returns).cumprod()
#     benchmark_returns = df_close_full[benchmark_ticker].pct_change().loc[strategy_equity_curve.index]; benchmark_equity_curve = (1 + benchmark_returns).cumprod()
#     cumulative_equity_df = pd.DataFrame({'Strategy_Equity': strategy_equity_curve, 'Benchmark_Equity': benchmark_equity_curve})
    
#     fig = go.Figure()
#     fig.add_trace(go.Scatter(x=cumulative_equity_df.index, y=cumulative_equity_df['Strategy_Equity'], name='Strategy', line=dict(color='green')))
#     fig.add_trace(go.Scatter(x=cumulative_equity_df.index, y=cumulative_equity_df['Benchmark_Equity'], name=f'Benchmark ({benchmark_ticker})', line=dict(color='black', dash='dash')))
#     fig.update_layout(title=f"Cumulative Performance: '{metric}' Strategy (Top {rank_start}-{rank_end})", xaxis_title="Date", yaxis_title="Cumulative Growth")
#     fig.show()

#     final_backtest_results = {'cumulative_performance': cumulative_equity_df, 'period_by_period_debug': period_by_period_debug}
#     print("\n✅ Full backtest complete. Results object is ready for forensic analysis.")
#     return final_backtest_results

# # --- E. VERIFICATION TOOLS (User Requested) ---

# def verify_group_tickers_walk_forward_calculation(df_ohlcv, tickers_to_verify, benchmark_ticker,
#                                                   start_date, calc_period, fwd_period,
#                                                   master_calendar_ticker='VOO', export_csv=False):
#     display(Markdown(f"## Verification Report for Portfolio vs. Benchmark"))
#     display(Markdown(f"**Portfolio Tickers:** `{tickers_to_verify}`\n**Benchmark Ticker:** `{benchmark_ticker}`"))
    
#     if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
#         raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
#     master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()

#     start_date_raw = pd.to_datetime(start_date)
#     start_idx = master_trading_days.searchsorted(start_date_raw)
#     if start_idx >= len(master_trading_days):
#         print(f"Error: Start date {start_date_raw.date()} is after the last available trading day."); return
#     actual_start_date = master_trading_days[start_idx]
    
#     calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
#     fwd_end_idx = min(calc_end_idx + fwd_period, len(master_trading_days) - 1)
    
#     actual_calc_end_date = master_trading_days[calc_end_idx]
#     actual_fwd_end_date = master_trading_days[fwd_end_idx]
    
#     display(Markdown(f"**Analysis Start:** `{actual_start_date.date()}` (Selected: `{start_date_raw.date()}`)\n"
#                     f"**Calc End:** `{actual_calc_end_date.date()}` ({calc_period} trading days)\n"
#                     f"**Fwd End:** `{actual_fwd_end_date.date()}` ({fwd_period} trading days)"))

#     df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
#     portfolio_prices_raw_slice = df_close_full[tickers_to_verify].loc[actual_start_date:actual_fwd_end_date]
#     portfolio_value_series = portfolio_prices_raw_slice.div(portfolio_prices_raw_slice.bfill().iloc[0]).mean(axis=1)
#     benchmark_price_series = df_close_full.get(benchmark_ticker)
    
#     if export_csv:
#         export_df = pd.DataFrame({
#             'Portfolio_Normalized_Price': portfolio_value_series,
#             'Portfolio_Daily_Return': portfolio_value_series.pct_change()
#         })
#         if benchmark_price_series is not None:
#             norm_bm = benchmark_price_series.loc[actual_start_date:actual_fwd_end_date]
#             norm_bm = norm_bm / norm_bm.bfill().iloc[0]
#             export_df['Benchmark_Normalized_Price'] = norm_bm
#             export_df['Benchmark_Daily_Return'] = norm_bm.pct_change()

#         output_dir = 'export_csv'
#         os.makedirs(output_dir, exist_ok=True)
#         tickers_str = '_'.join(tickers_to_verify)
#         filename = f"verify_group_{actual_start_date.date()}_{tickers_str}.csv"
#         filepath = os.path.join(output_dir, filename)
#         export_df.to_csv(filepath)
#         print(f"\n✅ Data exported to: {filepath}")

#     def print_verification_steps(title, price_series):
#         display(Markdown(f"#### Verification for: `{title}`"))
#         if price_series.dropna().shape[0] < 2: print("  - Not enough data points."); return {'gain': np.nan, 'sharpe': np.nan}
#         start_price, end_price = price_series.bfill().iloc[0], price_series.ffill().iloc[-1]
#         gain = (end_price / start_price) - 1
#         print(f"Start Value ({price_series.first_valid_index().date()}): {start_price:,.4f}\nEnd Value   ({price_series.last_valid_index().date()}): {end_price:,.4f}\nGain = {gain:.2%}")
#         returns = price_series.pct_change()
#         sharpe = calculate_sharpe(returns)
#         print(f"Mean Daily Return: {returns.mean():.6f}\nStd Dev: {returns.std():.6f}\nSharpe = {sharpe:.2f}")
#         return {'gain': gain, 'sharpe': sharpe}

#     display(Markdown("### A. Calculation Period"))
#     perf_calc_p = print_verification_steps("Group Portfolio", portfolio_value_series.loc[actual_start_date:actual_calc_end_date])
#     perf_calc_b = print_verification_steps(f"Benchmark", benchmark_price_series.loc[actual_start_date:actual_calc_end_date])
#     display(Markdown("### B. Forward Period"))
#     perf_fwd_p = print_verification_steps("Group Portfolio", portfolio_value_series.loc[actual_calc_end_date:actual_fwd_end_date])
#     perf_fwd_b = print_verification_steps(f"Benchmark", benchmark_price_series.loc[actual_calc_end_date:actual_fwd_end_date])
    

# def verify_ticker_ranking_metrics(df_ohlcv, ticker, start_date, calc_period,
#                                   master_calendar_ticker='VOO', export_csv=False):
#     display(Markdown(f"## Verification Report for Ticker Ranking: `{ticker}`"))
    
#     if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
#         raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
#     master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()

#     start_date_raw = pd.to_datetime(start_date)
#     start_idx = master_trading_days.searchsorted(start_date_raw)
#     if start_idx >= len(master_trading_days):
#         print(f"Error: Start date {start_date_raw.date()} is after the last available trading day."); return
#     actual_start_date = master_trading_days[start_idx]
    
#     calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
#     actual_calc_end_date = master_trading_days[calc_end_idx]

#     df_ticker = df_ohlcv.loc[ticker].sort_index()
#     calc_df = df_ticker.loc[actual_start_date:actual_calc_end_date].copy()
#     if calc_df.empty or len(calc_df) < 2: 
#         print("No data or not enough data in calc period."); return

#     display(Markdown("### A. Calculation Period (for Ranking Metrics)"))
#     display(Markdown(f"**Period Start:** `{actual_start_date.date()}`\n"
#                     f"**Period End:** `{actual_calc_end_date.date()}`\n"
#                     f"**Total Trading Days:** `{len(calc_df)}` (Requested: `{calc_period}`)"))
    
#     display(Markdown("#### Detailed Metric Calculation Data"))
    
#     vdf = calc_df[['Adj High', 'Adj Low', 'Adj Close']].copy()
#     vdf['Daily_Return'] = vdf['Adj Close'].pct_change()
    
#     # <<< BUG FIX: Reverted to a robust, in-place True Range calculation to prevent index errors.
#     prev_close = df_ticker['Adj Close'].shift(1).loc[actual_start_date:actual_calc_end_date]
#     tr_df = pd.DataFrame({
#         'h_l': vdf['Adj High'] - vdf['Adj Low'],
#         'h_pc': abs(vdf['Adj High'] - prev_close),
#         'l_pc': abs(vdf['Adj Low'] - prev_close)
#     })
#     vdf['TR'] = tr_df.max(axis=1)
    
#     vdf['ATR_14'] = vdf['TR'].ewm(alpha=1/14, adjust=False).mean()
#     vdf['ATRP'] = vdf['ATR_14'] / vdf['Adj Close']
    
#     print("--- Start of Calculation Period ---")
#     display(vdf.head())
#     print("\n--- End of Calculation Period ---")
#     display(vdf.tail())

#     if export_csv:
#         output_dir = 'export_csv'
#         os.makedirs(output_dir, exist_ok=True)
#         filename = f"verify_ticker_{actual_start_date.date()}_{ticker}.csv"
#         filepath = os.path.join(output_dir, filename)
#         vdf.to_csv(filepath)
#         print(f"\n✅ Data exported to: {filepath}")
    
#     display(Markdown("#### `MetricValue` Verification Summary:"))
    
#     calc_start_price = vdf['Adj Close'].bfill().iloc[0]
#     calc_end_price = vdf['Adj Close'].ffill().iloc[-1]
#     price_metric = (calc_end_price / calc_start_price)
#     print(f"1. Price Metric: (Last Price / First Price) = ({calc_end_price:.2f} / {calc_start_price:.2f}) = {price_metric:.4f}")
    
#     daily_returns = vdf['Daily_Return'].dropna()
#     sharpe_ratio = calculate_sharpe(daily_returns)
#     print(f"2. Sharpe Metric: (Mean Daily Return / Std Dev) * sqrt(252) = {sharpe_ratio:.4f}")

#     atrp_mean = vdf['ATRP'].mean()
#     mean_daily_return = vdf['Daily_Return'].mean()
#     sharpe_atr = (mean_daily_return / atrp_mean) if atrp_mean > 0 else 0
#     print(f"3. Sharpe (ATR) Metric: (Mean Daily Return / Mean ATRP) = ({mean_daily_return:.6f} / {atrp_mean:.6f}) = {sharpe_atr:.4f}")

In [8]:
# # ==============================================================================
# # FINAL PROJECT FUNCTIONS (Golden Copy - Definitive TR Bug Fix)
# # ==============================================================================

# import pandas as pd
# import numpy as np
# import plotly.graph_objects as go
# from datetime import datetime, date
# import ipywidgets as widgets
# from IPython.display import display, Markdown
# import pprint
# from tqdm.auto import tqdm
# import re
# import os

# # --- A. HELPER FUNCTIONS ---

# def calculate_gain(price_series: pd.Series):
#     if price_series.dropna().shape[0] < 2: return np.nan
#     return (price_series.ffill().iloc[-1] / price_series.bfill().iloc[0]) - 1

# def calculate_sharpe(return_series: pd.Series):
#     if return_series.dropna().shape[0] < 2: return np.nan
#     std_dev = return_series.std()
#     if std_dev > 0 and std_dev != np.inf:
#         return (return_series.mean() / std_dev) * np.sqrt(252)
#     return np.nan

# # --- B. THE CORE CALCULATION ENGINE ---

# def run_walk_forward_step(df_close_full, df_high_full, df_low_full,
#                           master_trading_days,
#                           start_date, calc_period, fwd_period,
#                           metric, rank_start, rank_end, benchmark_ticker,
#                           debug=False):
#     debug_data = {} if debug else None
    
#     try:
#         start_idx = master_trading_days.get_loc(start_date)
#     except KeyError:
#         return ({'error': f"Start date {start_date.date()} is not a valid trading day."}, None)
        
#     calc_end_idx = min(start_idx + calc_period -1, len(master_trading_days) - 1) # -1 because period is inclusive
#     viz_end_idx = min(calc_end_idx + fwd_period, len(master_trading_days) - 1)

#     safe_start_date = master_trading_days[start_idx]
#     safe_calc_end_date = master_trading_days[calc_end_idx]
#     safe_viz_end_date = master_trading_days[viz_end_idx]
    
#     if safe_start_date >= safe_calc_end_date:
#         return ({'error': "Invalid date range (calc period has zero or negative length)."}, None)

#     calc_close_raw = df_close_full.loc[safe_start_date:safe_calc_end_date]
#     calc_close = calc_close_raw.dropna(axis=1, how='all')
#     if calc_close.shape[1] == 0 or len(calc_close) < 2:
#         return ({'error': "Not enough data in calc period."}, None)

#     first_prices = calc_close.bfill().iloc[0]
#     last_prices = calc_close.ffill().iloc[-1]
#     daily_returns = calc_close.bfill().ffill().pct_change()
#     mean_returns = daily_returns.mean()
#     std_returns = daily_returns.std()
    
#     valid_tickers = calc_close.columns
#     calc_high = df_high_full[valid_tickers].loc[safe_start_date:safe_calc_end_date]
#     calc_low = df_low_full[valid_tickers].loc[safe_start_date:safe_calc_end_date]
    
#     # <<< DEFINITIVE BUG FIX: Robust, explicit slicing for previous close calculation.
#     # 1. Get the index position of the start date
#     prev_day_idx = max(0, start_idx - 1)
#     prev_day_date = master_trading_days[prev_day_idx]
#     # 2. Select close data over an extended range (one extra day at the start)
#     extended_close = df_close_full[valid_tickers].loc[prev_day_date:safe_calc_end_date]
#     # 3. Shift this smaller DataFrame, then slice it to the final exact window
#     prev_close = extended_close.shift(1).loc[safe_start_date:safe_calc_end_date]

#     tr_df = pd.DataFrame({
#         'h_l': calc_high - calc_low,
#         'h_pc': abs(calc_high - prev_close),
#         'l_pc': abs(calc_low - prev_close)
#     })
#     tr = tr_df.max(axis=1)
    
#     atr = tr.ewm(alpha=1/14, adjust=False).mean()
#     atrp = (atr / calc_close).mean()

#     metric_values = {}
#     metric_values['Price'] = (last_prices / first_prices).dropna()
#     metric_values['Sharpe'] = (mean_returns / std_returns * np.sqrt(252)).fillna(0)
#     metric_values['Sharpe (ATR)'] = (mean_returns / atrp).fillna(0)
    
#     if debug:
#         df_ranking = pd.DataFrame({
#             'FirstPrice': first_prices, 'LastPrice': last_prices, 'MeanDailyReturn': mean_returns,
#             'StdDevDailyReturn': std_returns, 'MeanATRP': atrp, 'Metric_Price': metric_values['Price'],
#             'Metric_Sharpe': metric_values['Sharpe'], 'Metric_Sharpe (ATR)': metric_values['Sharpe (ATR)']
#         })
#         df_ranking.index.name = 'Ticker'
#         debug_data['ranking_metrics'] = df_ranking.sort_values(f'Metric_{metric}', ascending=False)

#     sorted_tickers = metric_values[metric].sort_values(ascending=False)
#     tickers_to_display = sorted_tickers.index[rank_start-1:rank_end].tolist()
#     if not tickers_to_display:
#         return ({'error': "No tickers found for the selected rank."}, None)

#     normalized_plot_data = df_close_full[tickers_to_display].loc[safe_start_date:safe_viz_end_date]
#     normalized_plot_data = normalized_plot_data.div(normalized_plot_data.bfill().iloc[0])
#     actual_calc_end_ts = calc_close.index.max()
#     portfolio_series = normalized_plot_data.mean(axis=1)
#     portfolio_return_series = portfolio_series.pct_change()
#     benchmark_price_series = df_close_full.get(benchmark_ticker)
#     benchmark_return_series = benchmark_price_series.loc[safe_start_date:safe_viz_end_date].bfill().ffill().pct_change() if benchmark_price_series is not None else pd.Series(dtype='float64')

#     try:
#         boundary_loc = portfolio_return_series.index.get_loc(actual_calc_end_ts)
#         calc_portfolio_returns = portfolio_return_series.iloc[:boundary_loc + 1]
#         fwd_portfolio_returns = portfolio_return_series.iloc[boundary_loc + 1:]
#         bm_boundary_loc = benchmark_return_series.index.get_loc(actual_calc_end_ts)
#         calc_benchmark_returns = benchmark_return_series.iloc[:bm_boundary_loc + 1]
#         fwd_benchmark_returns = benchmark_return_series.iloc[bm_boundary_loc + 1:]
#     except (KeyError, IndexError):
#         calc_portfolio_returns = portfolio_return_series.loc[:actual_calc_end_ts]
#         fwd_portfolio_returns = portfolio_return_series.loc[actual_calc_end_ts:].iloc[1:]
#         calc_benchmark_returns = benchmark_return_series.loc[:actual_calc_end_ts]
#         fwd_benchmark_returns = benchmark_return_series.loc[actual_calc_end_ts:].iloc[1:]

#     perf_data = {}
#     perf_data['calc_p_gain'] = calculate_gain(portfolio_series.loc[:actual_calc_end_ts])
#     perf_data['fwd_p_gain'] = calculate_gain(portfolio_series.loc[actual_calc_end_ts:])
#     perf_data['full_p_gain'] = calculate_gain(portfolio_series)
#     perf_data['calc_p_sharpe'] = calculate_sharpe(calc_portfolio_returns)
#     perf_data['fwd_p_sharpe'] = calculate_sharpe(fwd_portfolio_returns)
#     perf_data['full_p_sharpe'] = calculate_sharpe(portfolio_return_series)
#     perf_data['calc_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:actual_calc_end_ts]) if benchmark_price_series is not None else np.nan
#     perf_data['fwd_b_gain'] = calculate_gain(benchmark_price_series.loc[actual_calc_end_ts:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
#     perf_data['full_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
#     perf_data['calc_b_sharpe'] = calculate_sharpe(calc_benchmark_returns)
#     perf_data['fwd_b_sharpe'] = calculate_sharpe(fwd_benchmark_returns)
#     perf_data['full_b_sharpe'] = calculate_sharpe(benchmark_return_series)

#     calc_end_prices = calc_close.ffill().iloc[-1]
#     fwd_close_slice = df_close_full.loc[actual_calc_end_ts:safe_viz_end_date]
#     viz_end_prices = fwd_close_slice.ffill().iloc[-1] if not fwd_close_slice.empty and len(fwd_close_slice) >= 2 else calc_end_prices
#     calc_gains = (calc_end_prices / calc_close.bfill().iloc[0]) - 1
#     fwd_gains = (viz_end_prices / calc_end_prices) - 1
#     results_df = pd.DataFrame({'Rank': range(rank_start, rank_start + len(tickers_to_display)), 'Metric': metric, 'MetricValue': sorted_tickers.loc[tickers_to_display].values, 'CalcPrice': calc_end_prices.loc[tickers_to_display], 'CalcGain': calc_gains.loc[tickers_to_display], 'FwdGain': fwd_gains.loc[tickers_to_display]}, index=pd.Index(tickers_to_display, name='Ticker'))
#     if benchmark_price_series is not None and benchmark_ticker in calc_close.columns:
#         benchmark_df_row = pd.DataFrame({'Rank': np.nan, 'Metric': metric, 'MetricValue': metric_values[metric].get(benchmark_ticker, np.nan), 'CalcPrice': calc_end_prices[benchmark_ticker], 'CalcGain': calc_gains[benchmark_ticker], 'FwdGain': fwd_gains[benchmark_ticker]}, index=pd.Index([f"{benchmark_ticker} (BM)"], name='Ticker'))
#         results_df = pd.concat([results_df, benchmark_df_row])
    
#     if debug:
#         df_trace = normalized_plot_data.copy()
#         df_trace.columns = [f'Norm_Price_{c}' for c in df_trace.columns]
#         df_trace['Norm_Price_Portfolio'] = portfolio_series
#         if benchmark_price_series is not None and not benchmark_price_series.loc[safe_start_date:safe_viz_end_date].dropna().empty:
#             norm_bm = benchmark_price_series.loc[safe_start_date:safe_viz_end_date] / benchmark_price_series.loc[safe_start_date:].bfill().iloc[0]
#             df_trace[f'Norm_Price_Benchmark_{benchmark_ticker}'] = norm_bm
#         for col in df_trace.columns:
#             if 'Norm_Price' in col:
#                 df_trace[col.replace('Norm_Price', 'Return')] = df_trace[col].pct_change()
#         debug_data['portfolio_trace'] = df_trace

#     final_results = {
#         'tickers_to_display': tickers_to_display, 'normalized_plot_data': normalized_plot_data,
#         'portfolio_series': portfolio_series, 'benchmark_price_series': benchmark_price_series,
#         'performance_data': perf_data, 'results_df': results_df, 'actual_calc_end_ts': actual_calc_end_ts,
#         'safe_start_date': safe_start_date, 'safe_viz_end_date': safe_viz_end_date,
#         'error': None
#     }
#     return (final_results, debug_data)

# # --- C. DYNAMIC DATA QUALITY FILTER FUNCTIONS ---

# def calculate_rolling_quality_metrics(df_ohlcv, window=252, min_periods=126, debug=False):
#     print(f"--- Calculating Rolling Quality Metrics (Window: {window} days) ---")
#     df = df_ohlcv.copy()
#     if not df.index.is_monotonic_increasing:
#         df.sort_index(inplace=True)
#     df['IsStale'] = np.where((df['Volume'] == 0) | (df['Adj High'] == df['Adj Low']), 1, 0)
#     df['DollarVolume'] = df['Adj Close'] * df['Volume']
#     df['HasSameVolumeAsPrevDay'] = (df.groupby(level='Ticker')['Volume'].diff() == 0).astype(int)
#     grouped = df.groupby(level='Ticker')
#     stale_pct = grouped['IsStale'].rolling(window=window, min_periods=min_periods).mean()
#     median_vol = grouped['DollarVolume'].rolling(window=window, min_periods=min_periods).median()
#     same_vol_count = grouped['HasSameVolumeAsPrevDay'].rolling(window=window, min_periods=min_periods).sum()
#     quality_df = pd.concat([stale_pct, median_vol, same_vol_count], axis=1)
#     quality_df.columns = ['RollingStalePct', 'RollingMedianVolume', 'RollingSameVolCount']
#     quality_df.index = quality_df.index.droplevel(0)
#     print("✅ Rolling metrics calculation complete.")
#     return quality_df

# def get_eligible_universe(quality_metrics_df, filter_date, thresholds):
#     filter_date_ts = pd.to_datetime(filter_date)
#     date_index = quality_metrics_df.index.get_level_values('Date').unique().sort_values()
#     if filter_date_ts < date_index[0]:
#         print(f"Warning: Filter date {filter_date_ts.date()} is before the earliest data point. Returning empty universe.")
#         return []
#     valid_prior_dates = date_index[date_index <= filter_date_ts]
#     if valid_prior_dates.empty:
#         print(f"Warning: No available data found on or before {filter_date_ts.date()}. Returning empty universe.")
#         return []
#     actual_date_to_use = valid_prior_dates[-1]
#     if actual_date_to_use.date() != filter_date_ts.date():
#         print(f"ℹ️ Info: Filter date {filter_date_ts.date()} not found. Using previous available date {actual_date_to_use.date()}.")
#     metrics_on_date = quality_metrics_df.xs(actual_date_to_use, level='Date')
#     mask = ((metrics_on_date['RollingMedianVolume'] >= thresholds['min_median_dollar_volume']) &
#             (metrics_on_date['RollingStalePct'] <= thresholds['max_stale_pct']) &
#             (metrics_on_date['RollingSameVolCount'] <= thresholds['max_same_vol_count']))
#     eligible_tickers = metrics_on_date[mask].index.tolist()
#     all_tickers = metrics_on_date.index.tolist()
#     print(f"Dynamic Filter ({filter_date_ts.date()}): Kept {len(eligible_tickers)} of {len(all_tickers)} tickers.")
#     return eligible_tickers    

# # --- D. INTERACTIVE ANALYSIS & BACKTESTING TOOLS ---

# def plot_walk_forward_analyzer(df_ohlcv, 
#                                default_start_date=None, default_calc_period=126, default_fwd_period=63,
#                                default_metric='Sharpe (ATR)', default_rank_start=1, default_rank_end=10,
#                                default_benchmark_ticker='VOO', master_calendar_ticker='VOO',
#                                quality_thresholds={'min_median_dollar_volume': 1_000_000, 'max_stale_pct': 0.05, 'max_same_vol_count': 10},
#                                debug=False):
#     print("Initializing Walk-Forward Analyzer (using Trading Day Logic)...")
#     if not isinstance(df_ohlcv.index, pd.MultiIndex): raise ValueError("Input DataFrame must have a (Ticker, Date) MultiIndex.")
#     df_ohlcv = df_ohlcv.sort_index()

#     if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
#         raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
#     master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()
#     print(f"Master trading day calendar created from '{master_calendar_ticker}' ({len(master_trading_days)} days).")

#     print("Pre-calculating data quality metrics...")
#     quality_metrics_df = calculate_rolling_quality_metrics(df_ohlcv, window=252)
#     print("Pre-processing data (unstacking)...")
#     df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
#     df_high_full = df_ohlcv['Adj High'].unstack(level=0)
#     df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
    
#     start_date_picker = widgets.DatePicker(description='Start Date:', value=pd.to_datetime(default_start_date), disabled=False)
#     calc_period_input = widgets.IntText(value=default_calc_period, description='Calc Period (days):')
#     fwd_period_input = widgets.IntText(value=default_fwd_period, description='Fwd Period (days):')
#     metrics = ['Price', 'Sharpe', 'Sharpe (ATR)']
#     metric_dropdown = widgets.Dropdown(options=metrics, value=default_metric, description='Metric:')
#     rank_start_input = widgets.IntText(value=default_rank_start, description='Rank Start:')
#     rank_end_input = widgets.IntText(value=default_rank_end, description='Rank End:')
#     benchmark_ticker_input = widgets.Text(value=default_benchmark_ticker, description='Benchmark:', placeholder='Enter Ticker')
#     update_button = widgets.Button(description="Update Chart", button_style='primary')
#     ticker_list_output = widgets.Output()
#     results_container, debug_data_container = [None], [None]

#     fig = go.FigureWidget()
#     max_traces = 50
#     for i in range(max_traces): fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'placeholder_{i}', visible=False, showlegend=False))
#     fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Benchmark', visible=True, showlegend=True, line=dict(color='black', width=3, dash='dash')))
#     fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Group Portfolio', visible=True, showlegend=True, line=dict(color='green', width=3)))

#     def update_plot(button_click):
#         ticker_list_output.clear_output()
#         start_date_raw = pd.to_datetime(start_date_picker.value)
#         start_date_idx = master_trading_days.searchsorted(start_date_raw)
#         if start_date_idx >= len(master_trading_days):
#             with ticker_list_output: print(f"Error: Start date is after the last available trading day."); return
#         actual_start_date = master_trading_days[start_date_idx]
#         with ticker_list_output: 
#             if start_date_raw.date() != actual_start_date.date():
#                 print(f"ℹ️ Info: Start date {start_date_raw.date()} is not a trading day. Snapping forward to {actual_start_date.date()}.")

#         calc_period, fwd_period = calc_period_input.value, fwd_period_input.value
#         metric = metric_dropdown.value
#         rank_start, rank_end = rank_start_input.value, rank_end_input.value
#         benchmark_ticker = benchmark_ticker_input.value.strip().upper()
        
#         if rank_start > rank_end:
#             with ticker_list_output: print("Error: 'Rank Start' must be <= 'Rank End'."); return
#         if rank_start < 1 or calc_period < 2 or fwd_period < 1:
#             with ticker_list_output: print("Error: Ranks must be >= 1, Calc Period >= 2, Fwd Period >= 1."); return

#         eligible_tickers = get_eligible_universe(quality_metrics_df, actual_start_date, quality_thresholds)
#         if not eligible_tickers:
#             with ticker_list_output: print(f"Error: No eligible tickers found on {actual_start_date.date()} with the current quality filters."); return
#         df_close_step = df_close_full[eligible_tickers]; df_high_step = df_high_full[eligible_tickers]; df_low_step = df_low_full[eligible_tickers]

#         results, debug_output = run_walk_forward_step(
#             df_close_step, df_high_step, df_low_step, master_trading_days,
#             actual_start_date, calc_period, fwd_period, 
#             metric, rank_start, rank_end, benchmark_ticker, debug=debug
#         )
#         if results['error']:
#             with ticker_list_output: print(f"Error: {results['error']}"); return

#         with fig.batch_update():
#             for i in range(max_traces):
#                 trace = fig.data[i]
#                 if i < len(results['tickers_to_display']):
#                     ticker = results['tickers_to_display'][i]; plot_data_series = results['normalized_plot_data'][ticker]
#                     trace.x, trace.y, trace.name, trace.visible, trace.showlegend = plot_data_series.index, plot_data_series.values, ticker, True, True
#                 else: trace.visible, trace.showlegend = False, False
#             benchmark_trace = fig.data[max_traces]
#             if results['benchmark_price_series'] is not None and not results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']].dropna().empty:
#                 normalized_benchmark = results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']] / results['benchmark_price_series'].loc[results['safe_start_date']:].bfill().iloc[0]
#                 benchmark_trace.x, benchmark_trace.y, benchmark_trace.name, benchmark_trace.visible = normalized_benchmark.index, normalized_benchmark, f"Benchmark ({benchmark_ticker})", True
#             else: benchmark_trace.visible = False
#             portfolio_trace = fig.data[max_traces + 1]
#             portfolio_trace.x, portfolio_trace.y, portfolio_trace.name, portfolio_trace.visible = results['portfolio_series'].index, results['portfolio_series'], 'Group Portfolio', True
#             fig.layout.shapes = []; fig.add_shape(type="line", x0=results['actual_calc_end_ts'], y0=0, x1=results['actual_calc_end_ts'], y1=1, xref='x', yref='paper', line=dict(color="grey", width=2, dash="dash"))
            
#         results_container[0] = results; debug_data_container[0] = debug_output
        
#         with ticker_list_output:
#             print(f"Analysis Period: {results['safe_start_date'].date()} to {results['safe_viz_end_date'].date()}.")
#             pprint.pprint(results['tickers_to_display'])
#             p = results['performance_data']
#             rows = []
#             rows.append({'Metric': 'Group Portfolio Gain', 'Full': p['full_p_gain'], 'Calc': p['calc_p_gain'], 'Fwd': p['fwd_p_gain']})
#             if not np.isnan(p['full_b_gain']):
#                 rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Gain', 'Full': p['full_b_gain'], 'Calc': p['calc_b_gain'], 'Fwd': p['fwd_b_gain']})
#                 rows.append({'Metric': 'Gain Delta (vs Bm)', 'Full': p['full_p_gain'] - p['full_b_gain'], 'Calc': p['calc_p_gain'] - p['calc_b_gain'], 'Fwd': p['fwd_p_gain'] - p['fwd_b_gain']})
#             rows.append({'Metric': 'Group Portfolio Sharpe', 'Full': p['full_p_sharpe'], 'Calc': p['calc_p_sharpe'], 'Fwd': p['fwd_p_sharpe']})
#             if not np.isnan(p['full_b_sharpe']):
#                 rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Sharpe', 'Full': p['full_b_sharpe'], 'Calc': p['calc_b_sharpe'], 'Fwd': p['fwd_b_sharpe']})
#                 rows.append({'Metric': 'Sharpe Delta (vs Bm)', 'Full': p['full_p_sharpe'] - p['full_b_sharpe'], 'Calc': p['calc_p_sharpe'] - p['calc_b_sharpe'], 'Fwd': p['fwd_p_sharpe'] - p['fwd_b_sharpe']})
#             report_df = pd.DataFrame(rows).set_index('Metric')
#             gain_rows = [row for row in report_df.index if 'Gain' in row or 'Delta' in row]
#             sharpe_rows = [row for row in report_df.index if 'Sharpe' in row]
#             styled_df = report_df.style.format('{:+.2%}', na_rep='N/A', subset=(gain_rows, report_df.columns)).format('{:+.2f}', na_rep='N/A', subset=(sharpe_rows, report_df.columns)).set_properties(**{'text-align': 'right', 'width': '100px'}).set_table_styles([{'selector': 'th.col_heading', 'props': [('text-align', 'right')]}, {'selector': 'th.row_heading', 'props': [('text-align', 'left')]}])
#             print("\n--- Strategy Performance Summary ---")
#             display(styled_df)
            
#     fig.update_layout(title_text='Walk-Forward Performance Analysis', xaxis_title='Date', yaxis_title='Normalized Price (Start = 1)', hovermode='x unified', legend_title_text='Tickers (Ranked)', height=600, margin=dict(t=50))
#     fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="grey")
#     update_button.on_click(update_plot)
#     controls_row1 = widgets.HBox([start_date_picker, calc_period_input, fwd_period_input])
#     controls_row2 = widgets.HBox([metric_dropdown, rank_start_input, rank_end_input, benchmark_ticker_input, update_button])
#     ui_container = widgets.VBox([controls_row1, controls_row2, ticker_list_output], layout=widgets.Layout(margin='10px 0 20px 0'))
#     display(ui_container, fig)
#     update_plot(None)
#     return (results_container, debug_data_container)

# def run_full_backtest(df_ohlcv, strategy_params, quality_thresholds):
#     print(f"--- Running Full Forensic Backtest for Strategy: {strategy_params['metric']} (Top {strategy_params['rank_start']}-{strategy_params['rank_end']}) ---")
    
#     start_date, end_date = pd.to_datetime(strategy_params['start_date']), pd.to_datetime(strategy_params['end_date'])
#     calc_period, fwd_period = strategy_params['calc_period'], strategy_params['fwd_period']
#     metric, rank_start, rank_end = strategy_params['metric'], strategy_params['rank_start'], strategy_params['rank_end']
#     benchmark_ticker = strategy_params['benchmark_ticker']
#     master_calendar_ticker = strategy_params.get('master_calendar_ticker', 'VOO')
    
#     if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
#         raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
#     master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()
    
#     start_idx = master_trading_days.searchsorted(start_date)
#     end_idx = master_trading_days.searchsorted(end_date, side='right')
    
#     quality_metrics_df = calculate_rolling_quality_metrics(df_ohlcv, window=252)
#     df_close_full = df_ohlcv['Adj Close'].unstack(level=0); df_high_full = df_ohlcv['Adj High'].unstack(level=0); df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
    
#     step_indices = range(start_idx, end_idx, fwd_period)
#     all_fwd_gains, period_by_period_debug = [], {}

#     print(f"Simulating {len(step_indices)} periods from {master_trading_days[step_indices[0]].date()} to {master_trading_days[step_indices[-1]].date()}...")
#     for current_idx in tqdm(step_indices, desc="Backtest Progress"):
#         step_date = master_trading_days[current_idx]
#         eligible_tickers = get_eligible_universe(quality_metrics_df, step_date, quality_thresholds)
#         if not eligible_tickers: continue
        
#         df_close_step = df_close_full[eligible_tickers]; df_high_step = df_high_full[eligible_tickers]; df_low_step = df_low_full[eligible_tickers]
        
#         results, debug_output = run_walk_forward_step(
#             df_close_step, df_high_step, df_low_step, master_trading_days,
#             step_date, calc_period, fwd_period,
#             metric, rank_start, rank_end, benchmark_ticker, debug=True
#         )
#         if results['error'] is None:
#             fwd_series = results['portfolio_series'].loc[results['actual_calc_end_ts']:]
#             all_fwd_gains.append(fwd_series.pct_change().dropna())
#             period_by_period_debug[step_date.date().isoformat()] = debug_output
            
#     if not all_fwd_gains:
#         print("Error: No valid periods were simulated."); return None

#     strategy_returns = pd.concat(all_fwd_gains); strategy_equity_curve = (1 + strategy_returns).cumprod()
#     benchmark_returns = df_close_full[benchmark_ticker].pct_change().loc[strategy_equity_curve.index]; benchmark_equity_curve = (1 + benchmark_returns).cumprod()
#     cumulative_equity_df = pd.DataFrame({'Strategy_Equity': strategy_equity_curve, 'Benchmark_Equity': benchmark_equity_curve})
    
#     fig = go.Figure()
#     fig.add_trace(go.Scatter(x=cumulative_equity_df.index, y=cumulative_equity_df['Strategy_Equity'], name='Strategy', line=dict(color='green')))
#     fig.add_trace(go.Scatter(x=cumulative_equity_df.index, y=cumulative_equity_df['Benchmark_Equity'], name=f'Benchmark ({benchmark_ticker})', line=dict(color='black', dash='dash')))
#     fig.update_layout(title=f"Cumulative Performance: '{metric}' Strategy (Top {rank_start}-{rank_end})", xaxis_title="Date", yaxis_title="Cumulative Growth")
#     fig.show()

#     final_backtest_results = {'cumulative_performance': cumulative_equity_df, 'period_by_period_debug': period_by_period_debug}
#     print("\n✅ Full backtest complete. Results object is ready for forensic analysis.")
#     return final_backtest_results

# # --- E. VERIFICATION TOOLS (User Requested) ---

# def verify_group_tickers_walk_forward_calculation(df_ohlcv, tickers_to_verify, benchmark_ticker,
#                                                   start_date, calc_period, fwd_period,
#                                                   master_calendar_ticker='VOO', export_csv=False):
#     display(Markdown(f"## Verification Report for Portfolio vs. Benchmark"))
#     display(Markdown(f"**Portfolio Tickers:** `{tickers_to_verify}`\n**Benchmark Ticker:** `{benchmark_ticker}`"))
    
#     if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
#         raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
#     master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()

#     start_date_raw = pd.to_datetime(start_date)
#     start_idx = master_trading_days.searchsorted(start_date_raw)
#     if start_idx >= len(master_trading_days):
#         print(f"Error: Start date {start_date_raw.date()} is after the last available trading day."); return
#     actual_start_date = master_trading_days[start_idx]
    
#     calc_end_idx = min(start_idx + calc_period - 1, len(master_trading_days) - 1)
#     fwd_end_idx = min(calc_end_idx + fwd_period, len(master_trading_days) - 1)
    
#     actual_calc_end_date = master_trading_days[calc_end_idx]
#     actual_fwd_end_date = master_trading_days[fwd_end_idx]
    
#     display(Markdown(f"**Analysis Start:** `{actual_start_date.date()}` (Selected: `{start_date_raw.date()}`)\n"
#                     f"**Calc End:** `{actual_calc_end_date.date()}` ({calc_period} trading days)\n"
#                     f"**Fwd End:** `{actual_fwd_end_date.date()}` ({fwd_period} trading days)"))

#     df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
#     portfolio_prices_raw_slice = df_close_full[tickers_to_verify].loc[actual_start_date:actual_fwd_end_date]
#     portfolio_value_series = portfolio_prices_raw_slice.div(portfolio_prices_raw_slice.bfill().iloc[0]).mean(axis=1)
#     benchmark_price_series = df_close_full.get(benchmark_ticker)
    
#     if export_csv:
#         export_df = pd.DataFrame({
#             'Portfolio_Normalized_Price': portfolio_value_series,
#             'Portfolio_Daily_Return': portfolio_value_series.pct_change()
#         })
#         if benchmark_price_series is not None:
#             norm_bm = benchmark_price_series.loc[actual_start_date:actual_fwd_end_date]
#             norm_bm = norm_bm / norm_bm.bfill().iloc[0]
#             export_df['Benchmark_Normalized_Price'] = norm_bm
#             export_df['Benchmark_Daily_Return'] = norm_bm.pct_change()

#         output_dir = 'export_csv'
#         os.makedirs(output_dir, exist_ok=True)
#         tickers_str = '_'.join(tickers_to_verify)
#         filename = f"verify_group_{actual_start_date.date()}_{tickers_str}.csv"
#         filepath = os.path.join(output_dir, filename)
#         export_df.to_csv(filepath)
#         print(f"\n✅ Data exported to: {filepath}")

#     def print_verification_steps(title, price_series):
#         display(Markdown(f"#### Verification for: `{title}`"))
#         if price_series.dropna().shape[0] < 2: print("  - Not enough data points."); return {'gain': np.nan, 'sharpe': np.nan}
#         start_price, end_price = price_series.bfill().iloc[0], price_series.ffill().iloc[-1]
#         gain = (end_price / start_price) - 1
#         print(f"Start Value ({price_series.first_valid_index().date()}): {start_price:,.4f}\nEnd Value   ({price_series.last_valid_index().date()}): {end_price:,.4f}\nGain = {gain:.2%}")
#         returns = price_series.pct_change()
#         sharpe = calculate_sharpe(returns)
#         print(f"Mean Daily Return: {returns.mean():.6f}\nStd Dev: {returns.std():.6f}\nSharpe = {sharpe:.2f}")
#         return {'gain': gain, 'sharpe': sharpe}

#     display(Markdown("### A. Calculation Period"))
#     perf_calc_p = print_verification_steps("Group Portfolio", portfolio_value_series.loc[actual_start_date:actual_calc_end_date])
#     perf_calc_b = print_verification_steps(f"Benchmark", benchmark_price_series.loc[actual_start_date:actual_calc_end_date])
#     display(Markdown("### B. Forward Period"))
#     perf_fwd_p = print_verification_steps("Group Portfolio", portfolio_value_series.loc[actual_calc_end_date:actual_fwd_end_date])
#     perf_fwd_b = print_verification_steps(f"Benchmark", benchmark_price_series.loc[actual_calc_end_date:actual_fwd_end_date])
    

# def verify_ticker_ranking_metrics(df_ohlcv, ticker, start_date, calc_period,
#                                   master_calendar_ticker='VOO', export_csv=False):
#     display(Markdown(f"## Verification Report for Ticker Ranking: `{ticker}`"))
    
#     if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
#         raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
#     master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()

#     start_date_raw = pd.to_datetime(start_date)
#     start_idx = master_trading_days.searchsorted(start_date_raw)
#     if start_idx >= len(master_trading_days):
#         print(f"Error: Start date {start_date_raw.date()} is after the last available trading day."); return
#     actual_start_date = master_trading_days[start_idx]
    
#     calc_end_idx = min(start_idx + calc_period - 1, len(master_trading_days) - 1)
#     actual_calc_end_date = master_trading_days[calc_end_idx]

#     df_ticker = df_ohlcv.loc[ticker].sort_index()
#     calc_df = df_ticker.loc[actual_start_date:actual_calc_end_date].copy()
#     if calc_df.empty or len(calc_df) < 2: 
#         print("No data or not enough data in calc period."); return

#     display(Markdown("### A. Calculation Period (for Ranking Metrics)"))
#     display(Markdown(f"**Period Start:** `{actual_start_date.date()}`\n"
#                     f"**Period End:** `{actual_calc_end_date.date()}`\n"
#                     f"**Total Trading Days:** `{len(calc_df)}` (Requested: `{calc_period}`)"))
    
#     display(Markdown("#### Detailed Metric Calculation Data"))
    
#     vdf = calc_df[['Adj High', 'Adj Low', 'Adj Close']].copy()
#     vdf['Daily_Return'] = vdf['Adj Close'].pct_change()
    
#     # <<< DEFINITIVE BUG FIX: Robust, explicit slicing for previous close calculation.
#     # 1. Get the index position of the start date in the single ticker's index
#     ticker_index = df_ticker.index
#     ticker_start_idx = ticker_index.get_loc(actual_start_date)
#     # 2. Go back one day
#     prev_day_idx = max(0, ticker_start_idx - 1)
#     prev_day_date = ticker_index[prev_day_idx]
#     # 3. Select close data over the extended range
#     extended_close = df_ticker['Adj Close'].loc[prev_day_date:actual_calc_end_date]
#     # 4. Shift and slice to get the perfectly aligned previous close
#     prev_close = extended_close.shift(1).loc[actual_start_date:actual_calc_end_date]

#     tr_df = pd.DataFrame({
#         'h_l': vdf['Adj High'] - vdf['Adj Low'],
#         'h_pc': abs(vdf['Adj High'] - prev_close),
#         'l_pc': abs(vdf['Adj Low'] - prev_close)
#     })
#     vdf['TR'] = tr_df.max(axis=1)
    
#     vdf['ATR_14'] = vdf['TR'].ewm(alpha=1/14, adjust=False).mean()
#     vdf['ATRP'] = vdf['ATR_14'] / vdf['Adj Close']
    
#     print("--- Start of Calculation Period ---")
#     display(vdf.head())
#     print("\n--- End of Calculation Period ---")
#     display(vdf.tail())

#     if export_csv:
#         output_dir = 'export_csv'
#         os.makedirs(output_dir, exist_ok=True)
#         filename = f"verify_ticker_{actual_start_date.date()}_{ticker}.csv"
#         filepath = os.path.join(output_dir, filename)
#         vdf.to_csv(filepath)
#         print(f"\n✅ Data exported to: {filepath}")
    
#     display(Markdown("#### `MetricValue` Verification Summary:"))
    
#     calc_start_price = vdf['Adj Close'].bfill().iloc[0]
#     calc_end_price = vdf['Adj Close'].ffill().iloc[-1]
#     price_metric = (calc_end_price / calc_start_price)
#     print(f"1. Price Metric: (Last Price / First Price) = ({calc_end_price:.2f} / {calc_start_price:.2f}) = {price_metric:.4f}")
    
#     daily_returns = vdf['Daily_Return'].dropna()
#     sharpe_ratio = calculate_sharpe(daily_returns)
#     print(f"2. Sharpe Metric: (Mean Daily Return / Std Dev) * sqrt(252) = {sharpe_ratio:.4f}")

#     atrp_mean = vdf['ATRP'].mean()
#     mean_daily_return = vdf['Daily_Return'].mean()
#     sharpe_atr = (mean_daily_return / atrp_mean) if atrp_mean > 0 else 0
#     print(f"3. Sharpe (ATR) Metric: (Mean Daily Return / Mean ATRP) = ({mean_daily_return:.6f} / {atrp_mean:.6f}) = {sharpe_atr:.4f}")

In [23]:
# ==============================================================================
# GOLDEN COPY - COMPLETE PROJECT CODE (All Fixes Included)
# Version: Trading Day Logic with Sharpe and TR Bug Fixes
# ==============================================================================

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime, date
import ipywidgets as widgets
from IPython.display import display, Markdown
import pprint
from tqdm.auto import tqdm
import re
import os

# --- A. HELPER FUNCTIONS ---

def calculate_gain(price_series: pd.Series):
    """Calculates the total gain over a series of prices."""
    if price_series.dropna().shape[0] < 2: return np.nan
    return (price_series.ffill().iloc[-1] / price_series.bfill().iloc[0]) - 1

def calculate_sharpe(return_series: pd.Series):
    """Calculates the annualized Sharpe ratio from a series of daily returns."""
    if return_series.dropna().shape[0] < 2: return np.nan
    std_dev = return_series.std()
    if std_dev > 0 and std_dev != np.inf:
        return (return_series.mean() / std_dev) * np.sqrt(252)
    return np.nan

# --- B. THE CORE CALCULATION ENGINE ---

# def run_walk_forward_step(df_close_full, df_high_full, df_low_full,
#                           master_trading_days,
#                           start_date, calc_period, fwd_period,
#                           metric, rank_start, rank_end, benchmark_ticker,
#                           debug=False):
#     """Runs a single step of the walk-forward analysis using precise trading days."""
#     debug_data = {} if debug else None
    
#     try:
#         start_idx = master_trading_days.get_loc(start_date)
#     except KeyError:
#         return ({'error': f"Start date {start_date.date()} is not a valid trading day."}, None)
        
#     calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
#     viz_end_idx = min(calc_end_idx + fwd_period, len(master_trading_days) - 1)

#     safe_start_date = master_trading_days[start_idx]
#     safe_calc_end_date = master_trading_days[calc_end_idx]
#     safe_viz_end_date = master_trading_days[viz_end_idx]
    
#     if safe_start_date >= safe_calc_end_date:
#         return ({'error': "Invalid date range (calc period has zero or negative length)."}, None)

#     calc_close_raw = df_close_full.loc[safe_start_date:safe_calc_end_date]
#     calc_close = calc_close_raw.dropna(axis=1, how='all')
#     if calc_close.shape[1] == 0 or len(calc_close) < 2:
#         return ({'error': "Not enough data in calc period."}, None)

#     first_prices = calc_close.bfill().iloc[0]
#     last_prices = calc_close.ffill().iloc[-1]
#     daily_returns = calc_close.bfill().ffill().pct_change()
#     mean_returns = daily_returns.mean()
#     std_returns = daily_returns.std()
    
#     valid_tickers = calc_close.columns
#     calc_high = df_high_full[valid_tickers].loc[safe_start_date:safe_calc_end_date]
#     calc_low = df_low_full[valid_tickers].loc[safe_start_date:safe_calc_end_date]
    
#     # Corrected True Range calculation
#     tr_df = pd.DataFrame({
#         'h_l': calc_high - calc_low,
#         'h_cp': abs(calc_high - df_close_full[valid_tickers].shift(1)),
#         'l_cp': abs(calc_low - df_close_full[valid_tickers].shift(1))
#     })
#     tr = tr_df.max(axis=1)
    
#     atr = tr.ewm(alpha=1/14, adjust=False).mean()
#     atrp = (atr / calc_close).mean()

#     metric_values = {}
#     metric_values['Price'] = (last_prices / first_prices).dropna()
#     metric_values['Sharpe'] = (mean_returns / std_returns * np.sqrt(252)).fillna(0)
#     metric_values['Sharpe (ATR)'] = (mean_returns / atrp).fillna(0)

#     if debug:
#         df_ranking = pd.DataFrame({
#             'FirstPrice': first_prices, 'LastPrice': last_prices, 'MeanDailyReturn': mean_returns,
#             'StdDevDailyReturn': std_returns, 'MeanATRP': atrp, 'Metric_Price': metric_values['Price'],
#             'Metric_Sharpe': metric_values['Sharpe'], 'Metric_Sharpe (ATR)': metric_values['Sharpe (ATR)']
#         })
#         df_ranking.index.name = 'Ticker'
#         debug_data['ranking_metrics'] = df_ranking.sort_values(f'Metric_{metric}', ascending=False)

#     sorted_tickers = metric_values[metric].sort_values(ascending=False)
#     tickers_to_display = sorted_tickers.index[rank_start-1:rank_end].tolist()
#     if not tickers_to_display:
#         return ({'error': "No tickers found for the selected rank."}, None)

#     normalized_plot_data = df_close_full[tickers_to_display].loc[safe_start_date:safe_viz_end_date]
#     normalized_plot_data = normalized_plot_data.div(normalized_plot_data.bfill().iloc[0])
#     actual_calc_end_ts = calc_close.index.max()
#     portfolio_series = normalized_plot_data.mean(axis=1)
#     portfolio_return_series = portfolio_series.pct_change()
#     benchmark_price_series = df_close_full.get(benchmark_ticker)
#     benchmark_return_series = benchmark_price_series.loc[safe_start_date:safe_viz_end_date].bfill().ffill().pct_change() if benchmark_price_series is not None else pd.Series(dtype='float64')

#     # Corrected slicing for Sharpe calculation to prevent overlap
#     try:
#         boundary_loc = portfolio_return_series.index.get_loc(actual_calc_end_ts)
#         calc_portfolio_returns = portfolio_return_series.iloc[:boundary_loc + 1]
#         fwd_portfolio_returns = portfolio_return_series.iloc[boundary_loc + 1:]
        
#         bm_boundary_loc = benchmark_return_series.index.get_loc(actual_calc_end_ts)
#         calc_benchmark_returns = benchmark_return_series.iloc[:bm_boundary_loc + 1]
#         fwd_benchmark_returns = benchmark_return_series.iloc[bm_boundary_loc + 1:]
#     except (KeyError, IndexError):
#         calc_portfolio_returns = portfolio_return_series.loc[:actual_calc_end_ts]
#         fwd_portfolio_returns = portfolio_return_series.loc[actual_calc_end_ts:].iloc[1:]
#         calc_benchmark_returns = benchmark_return_series.loc[:actual_calc_end_ts]
#         fwd_benchmark_returns = benchmark_return_series.loc[actual_calc_end_ts:].iloc[1:]

#     perf_data = {}
#     perf_data['calc_p_gain'] = calculate_gain(portfolio_series.loc[:actual_calc_end_ts])
#     perf_data['fwd_p_gain'] = calculate_gain(portfolio_series.loc[actual_calc_end_ts:])
#     perf_data['full_p_gain'] = calculate_gain(portfolio_series)
#     perf_data['calc_p_sharpe'] = calculate_sharpe(calc_portfolio_returns)
#     perf_data['fwd_p_sharpe'] = calculate_sharpe(fwd_portfolio_returns)
#     perf_data['full_p_sharpe'] = calculate_sharpe(portfolio_return_series)
    
#     perf_data['calc_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:actual_calc_end_ts]) if benchmark_price_series is not None else np.nan
#     perf_data['fwd_b_gain'] = calculate_gain(benchmark_price_series.loc[actual_calc_end_ts:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
#     perf_data['full_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
#     perf_data['calc_b_sharpe'] = calculate_sharpe(calc_benchmark_returns)
#     perf_data['fwd_b_sharpe'] = calculate_sharpe(fwd_benchmark_returns)
#     perf_data['full_b_sharpe'] = calculate_sharpe(benchmark_return_series)

#     calc_end_prices = calc_close.ffill().iloc[-1]
#     fwd_close_slice = df_close_full.loc[actual_calc_end_ts:safe_viz_end_date]
#     viz_end_prices = fwd_close_slice.ffill().iloc[-1] if not fwd_close_slice.empty and len(fwd_close_slice) >= 2 else calc_end_prices
#     calc_gains = (calc_end_prices / calc_close.bfill().iloc[0]) - 1
#     fwd_gains = (viz_end_prices / calc_end_prices) - 1
#     results_df = pd.DataFrame({'Rank': range(rank_start, rank_start + len(tickers_to_display)), 'Metric': metric, 'MetricValue': sorted_tickers.loc[tickers_to_display].values, 'CalcPrice': calc_end_prices.loc[tickers_to_display], 'CalcGain': calc_gains.loc[tickers_to_display], 'FwdGain': fwd_gains.loc[tickers_to_display]}, index=pd.Index(tickers_to_display, name='Ticker'))
#     if benchmark_price_series is not None and benchmark_ticker in calc_close.columns:
#         benchmark_df_row = pd.DataFrame({'Rank': np.nan, 'Metric': metric, 'MetricValue': metric_values[metric].get(benchmark_ticker, np.nan), 'CalcPrice': calc_end_prices[benchmark_ticker], 'CalcGain': calc_gains[benchmark_ticker], 'FwdGain': fwd_gains[benchmark_ticker]}, index=pd.Index([f"{benchmark_ticker} (BM)"], name='Ticker'))
#         results_df = pd.concat([results_df, benchmark_df_row])
    
#     if debug:
#         df_trace = normalized_plot_data.copy()
#         df_trace.columns = [f'Norm_Price_{c}' for c in df_trace.columns]
#         df_trace['Norm_Price_Portfolio'] = portfolio_series
#         if benchmark_price_series is not None and not benchmark_price_series.loc[safe_start_date:safe_viz_end_date].dropna().empty:
#             norm_bm = benchmark_price_series.loc[safe_start_date:safe_viz_end_date] / benchmark_price_series.loc[safe_start_date:].bfill().iloc[0]
#             df_trace[f'Norm_Price_Benchmark_{benchmark_ticker}'] = norm_bm
#         for col in df_trace.columns:
#             if 'Norm_Price' in col:
#                 df_trace[col.replace('Norm_Price', 'Return')] = df_trace[col].pct_change()
#         debug_data['portfolio_trace'] = df_trace

#     final_results = {
#         'tickers_to_display': tickers_to_display, 'normalized_plot_data': normalized_plot_data,
#         'portfolio_series': portfolio_series, 'benchmark_price_series': benchmark_price_series,
#         'performance_data': perf_data, 'results_df': results_df, 'actual_calc_end_ts': actual_calc_end_ts,
#         'safe_start_date': safe_start_date, 'safe_viz_end_date': safe_viz_end_date,
#         'error': None
#     }
#     return (final_results, debug_data)

# def run_walk_forward_step(df_close_full, df_high_full, df_low_full,
#                           master_trading_days,
#                           start_date, calc_period, fwd_period,
#                           metric, rank_start, rank_end, benchmark_ticker,
#                           debug=False):
#     """Runs a single step of the walk-forward analysis using precise trading days."""
#     debug_data = {} if debug else None
    
#     try:
#         start_idx = master_trading_days.get_loc(start_date)
#     except KeyError:
#         return ({'error': f"Start date {start_date.date()} is not a valid trading day."}, None)
        
#     calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
#     viz_end_idx = min(calc_end_idx + fwd_period, len(master_trading_days) - 1)

#     safe_start_date = master_trading_days[start_idx]
#     safe_calc_end_date = master_trading_days[calc_end_idx]
#     safe_viz_end_date = master_trading_days[viz_end_idx]
    
#     if safe_start_date >= safe_calc_end_date:
#         return ({'error': "Invalid date range (calc period has zero or negative length)."}, None)

#     calc_close_raw = df_close_full.loc[safe_start_date:safe_calc_end_date]
#     calc_close = calc_close_raw.dropna(axis=1, how='all')
#     if calc_close.shape[1] == 0 or len(calc_close) < 2:
#         return ({'error': "Not enough data in calc period."}, None)

#     first_prices = calc_close.bfill().iloc[0]
#     last_prices = calc_close.ffill().iloc[-1]
#     daily_returns = calc_close.bfill().ffill().pct_change()
#     mean_returns = daily_returns.mean()
#     std_returns = daily_returns.std()
    
#     valid_tickers = calc_close.columns
#     calc_high = df_high_full[valid_tickers].loc[safe_start_date:safe_calc_end_date]
#     calc_low = df_low_full[valid_tickers].loc[safe_start_date:safe_calc_end_date]
    
#     # ==============================================================================
#     # --- BUG FIX: Explicitly align previous day's close before subtraction ---
#     # ==============================================================================
#     # Create a DataFrame of previous day's close prices that EXACTLY matches the
#     # index of the calc_high and calc_low DataFrames.
#     prev_close = df_close_full[valid_tickers].shift(1).loc[safe_start_date:safe_calc_end_date]
    
#     # Now, all DataFrames in this calculation have the same shape and index.
#     tr_df = pd.DataFrame({
#         'h_l': calc_high - calc_low,
#         'h_cp': abs(calc_high - prev_close),
#         'l_cp': abs(calc_low - prev_close)
#     })
#     # ==============================================================================
#     # --- END OF FIX ---
#     # ==============================================================================
    
#     tr = tr_df.max(axis=1)
    
#     atr = tr.ewm(alpha=1/14, adjust=False).mean()
#     atrp = (atr / calc_close).mean()

#     metric_values = {}
#     metric_values['Price'] = (last_prices / first_prices).dropna()
#     metric_values['Sharpe'] = (mean_returns / std_returns * np.sqrt(252)).fillna(0)
#     metric_values['Sharpe (ATR)'] = (mean_returns / atrp).fillna(0)

#     if debug:
#         df_ranking = pd.DataFrame({
#             'FirstPrice': first_prices, 'LastPrice': last_prices, 'MeanDailyReturn': mean_returns,
#             'StdDevDailyReturn': std_returns, 'MeanATRP': atrp, 'Metric_Price': metric_values['Price'],
#             'Metric_Sharpe': metric_values['Sharpe'], 'Metric_Sharpe (ATR)': metric_values['Sharpe (ATR)']
#         })
#         df_ranking.index.name = 'Ticker'
#         debug_data['ranking_metrics'] = df_ranking.sort_values(f'Metric_{metric}', ascending=False)

#     sorted_tickers = metric_values[metric].sort_values(ascending=False)
#     tickers_to_display = sorted_tickers.index[rank_start-1:rank_end].tolist()
#     if not tickers_to_display:
#         return ({'error': "No tickers found for the selected rank."}, None)

#     normalized_plot_data = df_close_full[tickers_to_display].loc[safe_start_date:safe_viz_end_date]
#     normalized_plot_data = normalized_plot_data.div(normalized_plot_data.bfill().iloc[0])
#     actual_calc_end_ts = calc_close.index.max()
#     portfolio_series = normalized_plot_data.mean(axis=1)
#     portfolio_return_series = portfolio_series.pct_change()
#     benchmark_price_series = df_close_full.get(benchmark_ticker)
#     benchmark_return_series = benchmark_price_series.loc[safe_start_date:safe_viz_end_date].bfill().ffill().pct_change() if benchmark_price_series is not None else pd.Series(dtype='float64')

#     # Corrected slicing for Sharpe calculation to prevent overlap
#     try:
#         boundary_loc = portfolio_return_series.index.get_loc(actual_calc_end_ts)
#         calc_portfolio_returns = portfolio_return_series.iloc[:boundary_loc + 1]
#         fwd_portfolio_returns = portfolio_return_series.iloc[boundary_loc + 1:]
        
#         bm_boundary_loc = benchmark_return_series.index.get_loc(actual_calc_end_ts)
#         calc_benchmark_returns = benchmark_return_series.iloc[:bm_boundary_loc + 1]
#         fwd_benchmark_returns = benchmark_return_series.iloc[bm_boundary_loc + 1:]
#     except (KeyError, IndexError):
#         calc_portfolio_returns = portfolio_return_series.loc[:actual_calc_end_ts]
#         fwd_portfolio_returns = portfolio_return_series.loc[actual_calc_end_ts:].iloc[1:]
#         calc_benchmark_returns = benchmark_return_series.loc[:actual_calc_end_ts]
#         fwd_benchmark_returns = benchmark_return_series.loc[actual_calc_end_ts:].iloc[1:]

#     perf_data = {}
#     perf_data['calc_p_gain'] = calculate_gain(portfolio_series.loc[:actual_calc_end_ts])
#     perf_data['fwd_p_gain'] = calculate_gain(portfolio_series.loc[actual_calc_end_ts:])
#     perf_data['full_p_gain'] = calculate_gain(portfolio_series)
#     perf_data['calc_p_sharpe'] = calculate_sharpe(calc_portfolio_returns)
#     perf_data['fwd_p_sharpe'] = calculate_sharpe(fwd_portfolio_returns)
#     perf_data['full_p_sharpe'] = calculate_sharpe(portfolio_return_series)
    
#     perf_data['calc_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:actual_calc_end_ts]) if benchmark_price_series is not None else np.nan
#     perf_data['fwd_b_gain'] = calculate_gain(benchmark_price_series.loc[actual_calc_end_ts:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
#     perf_data['full_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
#     perf_data['calc_b_sharpe'] = calculate_sharpe(calc_benchmark_returns)
#     perf_data['fwd_b_sharpe'] = calculate_sharpe(fwd_benchmark_returns)
#     perf_data['full_b_sharpe'] = calculate_sharpe(benchmark_return_series)

#     calc_end_prices = calc_close.ffill().iloc[-1]
#     fwd_close_slice = df_close_full.loc[actual_calc_end_ts:safe_viz_end_date]
#     viz_end_prices = fwd_close_slice.ffill().iloc[-1] if not fwd_close_slice.empty and len(fwd_close_slice) >= 2 else calc_end_prices
#     calc_gains = (calc_end_prices / calc_close.bfill().iloc[0]) - 1
#     fwd_gains = (viz_end_prices / calc_end_prices) - 1
#     results_df = pd.DataFrame({'Rank': range(rank_start, rank_start + len(tickers_to_display)), 'Metric': metric, 'MetricValue': sorted_tickers.loc[tickers_to_display].values, 'CalcPrice': calc_end_prices.loc[tickers_to_display], 'CalcGain': calc_gains.loc[tickers_to_display], 'FwdGain': fwd_gains.loc[tickers_to_display]}, index=pd.Index(tickers_to_display, name='Ticker'))
#     if benchmark_price_series is not None and benchmark_ticker in calc_close.columns:
#         benchmark_df_row = pd.DataFrame({'Rank': np.nan, 'Metric': metric, 'MetricValue': metric_values[metric].get(benchmark_ticker, np.nan), 'CalcPrice': calc_end_prices[benchmark_ticker], 'CalcGain': calc_gains[benchmark_ticker], 'FwdGain': fwd_gains[benchmark_ticker]}, index=pd.Index([f"{benchmark_ticker} (BM)"], name='Ticker'))
#         results_df = pd.concat([results_df, benchmark_df_row])
    
#     if debug:
#         df_trace = normalized_plot_data.copy()
#         df_trace.columns = [f'Norm_Price_{c}' for c in df_trace.columns]
#         df_trace['Norm_Price_Portfolio'] = portfolio_series
#         if benchmark_price_series is not None and not benchmark_price_series.loc[safe_start_date:safe_viz_end_date].dropna().empty:
#             norm_bm = benchmark_price_series.loc[safe_start_date:safe_viz_end_date] / benchmark_price_series.loc[safe_start_date:].bfill().iloc[0]
#             df_trace[f'Norm_Price_Benchmark_{benchmark_ticker}'] = norm_bm
#         for col in df_trace.columns:
#             if 'Norm_Price' in col:
#                 df_trace[col.replace('Norm_Price', 'Return')] = df_trace[col].pct_change()
#         debug_data['portfolio_trace'] = df_trace

#     final_results = {
#         'tickers_to_display': tickers_to_display, 'normalized_plot_data': normalized_plot_data,
#         'portfolio_series': portfolio_series, 'benchmark_price_series': benchmark_price_series,
#         'performance_data': perf_data, 'results_df': results_df, 'actual_calc_end_ts': actual_calc_end_ts,
#         'safe_start_date': safe_start_date, 'safe_viz_end_date': safe_viz_end_date,
#         'error': None
#     }
#     return (final_results, debug_data)


def run_walk_forward_step(df_close_full, df_high_full, df_low_full,
                          master_trading_days,
                          start_date, calc_period, fwd_period,
                          metric, rank_start, rank_end, benchmark_ticker,
                          debug=False):
    """Runs a single step of the walk-forward analysis using precise trading days."""
    debug_data = {} if debug else None
    
    try:
        start_idx = master_trading_days.get_loc(start_date)
    except KeyError:
        return ({'error': f"Start date {start_date.date()} is not a valid trading day."}, None)
        
    calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
    viz_end_idx = min(calc_end_idx + fwd_period, len(master_trading_days) - 1)

    safe_start_date = master_trading_days[start_idx]
    safe_calc_end_date = master_trading_days[calc_end_idx]
    safe_viz_end_date = master_trading_days[viz_end_idx]
    
    if safe_start_date >= safe_calc_end_date:
        return ({'error': "Invalid date range (calc period has zero or negative length)."}, None)

    calc_close_raw = df_close_full.loc[safe_start_date:safe_calc_end_date]
    calc_close = calc_close_raw.dropna(axis=1, how='all')
    if calc_close.shape[1] == 0 or len(calc_close) < 2:
        return ({'error': "Not enough data in calc period."}, None)

    first_prices = calc_close.bfill().iloc[0]
    last_prices = calc_close.ffill().iloc[-1]
    daily_returns = calc_close.bfill().ffill().pct_change()
    mean_returns = daily_returns.mean()
    std_returns = daily_returns.std()
    
    valid_tickers = calc_close.columns
    calc_high = df_high_full[valid_tickers].loc[safe_start_date:safe_calc_end_date]
    calc_low = df_low_full[valid_tickers].loc[safe_start_date:safe_calc_end_date]
    
    # ==============================================================================
    # --- FINAL BUG FIX: Correctly calculate element-wise True Range ---
    # ==============================================================================
    # Create a DataFrame of previous day's close prices that EXACTLY matches the
    # index and columns of the calc_high and calc_low DataFrames.
    prev_close = df_close_full[valid_tickers].shift(1).loc[safe_start_date:safe_calc_end_date]
    
    # Calculate the three components of True Range. Each of these is a DataFrame
    # with the same shape (days x tickers).
    component1 = calc_high - calc_low
    component2 = abs(calc_high - prev_close)
    component3 = abs(calc_low - prev_close)

    # Find the element-wise maximum across the three component DataFrames.
    # np.maximum is efficient and preserves the DataFrame structure.
    # We chain it to handle the three components.
    tr = np.maximum(component1, np.maximum(component2, component3))
    # ==============================================================================
    # --- END OF FIX ---
    # ==============================================================================
    
    atr = tr.ewm(alpha=1/14, adjust=False).mean()
    atrp = (atr / calc_close).mean()

    metric_values = {}
    metric_values['Price'] = (last_prices / first_prices).dropna()
    metric_values['Sharpe'] = (mean_returns / std_returns * np.sqrt(252)).fillna(0)
    metric_values['Sharpe (ATR)'] = (mean_returns / atrp).fillna(0)

    if debug:
        df_ranking = pd.DataFrame({
            'FirstPrice': first_prices, 'LastPrice': last_prices, 'MeanDailyReturn': mean_returns,
            'StdDevDailyReturn': std_returns, 'MeanATRP': atrp, 'Metric_Price': metric_values['Price'],
            'Metric_Sharpe': metric_values['Sharpe'], 'Metric_Sharpe (ATR)': metric_values['Sharpe (ATR)']
        })
        df_ranking.index.name = 'Ticker'
        debug_data['ranking_metrics'] = df_ranking.sort_values(f'Metric_{metric}', ascending=False)

    sorted_tickers = metric_values[metric].sort_values(ascending=False)
    tickers_to_display = sorted_tickers.index[rank_start-1:rank_end].tolist()
    if not tickers_to_display:
        return ({'error': "No tickers found for the selected rank."}, None)

    normalized_plot_data = df_close_full[tickers_to_display].loc[safe_start_date:safe_viz_end_date]
    normalized_plot_data = normalized_plot_data.div(normalized_plot_data.bfill().iloc[0])
    actual_calc_end_ts = calc_close.index.max()
    portfolio_series = normalized_plot_data.mean(axis=1)
    portfolio_return_series = portfolio_series.pct_change()
    benchmark_price_series = df_close_full.get(benchmark_ticker)
    benchmark_return_series = benchmark_price_series.loc[safe_start_date:safe_viz_end_date].bfill().ffill().pct_change() if benchmark_price_series is not None else pd.Series(dtype='float64')

    # Corrected slicing for Sharpe calculation to prevent overlap
    try:
        boundary_loc = portfolio_return_series.index.get_loc(actual_calc_end_ts)
        calc_portfolio_returns = portfolio_return_series.iloc[:boundary_loc + 1]
        fwd_portfolio_returns = portfolio_return_series.iloc[boundary_loc + 1:]
        
        bm_boundary_loc = benchmark_return_series.index.get_loc(actual_calc_end_ts)
        calc_benchmark_returns = benchmark_return_series.iloc[:bm_boundary_loc + 1]
        fwd_benchmark_returns = benchmark_return_series.iloc[bm_boundary_loc + 1:]
    except (KeyError, IndexError):
        calc_portfolio_returns = portfolio_return_series.loc[:actual_calc_end_ts]
        fwd_portfolio_returns = portfolio_return_series.loc[actual_calc_end_ts:].iloc[1:]
        calc_benchmark_returns = benchmark_return_series.loc[:actual_calc_end_ts]
        fwd_benchmark_returns = benchmark_return_series.loc[actual_calc_end_ts:].iloc[1:]

    perf_data = {}
    perf_data['calc_p_gain'] = calculate_gain(portfolio_series.loc[:actual_calc_end_ts])
    perf_data['fwd_p_gain'] = calculate_gain(portfolio_series.loc[actual_calc_end_ts:])
    perf_data['full_p_gain'] = calculate_gain(portfolio_series)
    perf_data['calc_p_sharpe'] = calculate_sharpe(calc_portfolio_returns)
    perf_data['fwd_p_sharpe'] = calculate_sharpe(fwd_portfolio_returns)
    perf_data['full_p_sharpe'] = calculate_sharpe(portfolio_return_series)
    
    perf_data['calc_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:actual_calc_end_ts]) if benchmark_price_series is not None else np.nan
    perf_data['fwd_b_gain'] = calculate_gain(benchmark_price_series.loc[actual_calc_end_ts:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
    perf_data['full_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
    perf_data['calc_b_sharpe'] = calculate_sharpe(calc_benchmark_returns)
    perf_data['fwd_b_sharpe'] = calculate_sharpe(fwd_benchmark_returns)
    perf_data['full_b_sharpe'] = calculate_sharpe(benchmark_return_series)

    calc_end_prices = calc_close.ffill().iloc[-1]
    fwd_close_slice = df_close_full.loc[actual_calc_end_ts:safe_viz_end_date]
    viz_end_prices = fwd_close_slice.ffill().iloc[-1] if not fwd_close_slice.empty and len(fwd_close_slice) >= 2 else calc_end_prices
    calc_gains = (calc_end_prices / calc_close.bfill().iloc[0]) - 1
    fwd_gains = (viz_end_prices / calc_end_prices) - 1
    results_df = pd.DataFrame({'Rank': range(rank_start, rank_start + len(tickers_to_display)), 'Metric': metric, 'MetricValue': sorted_tickers.loc[tickers_to_display].values, 'CalcPrice': calc_end_prices.loc[tickers_to_display], 'CalcGain': calc_gains.loc[tickers_to_display], 'FwdGain': fwd_gains.loc[tickers_to_display]}, index=pd.Index(tickers_to_display, name='Ticker'))
    if benchmark_price_series is not None and benchmark_ticker in calc_close.columns:
        benchmark_df_row = pd.DataFrame({'Rank': np.nan, 'Metric': metric, 'MetricValue': metric_values[metric].get(benchmark_ticker, np.nan), 'CalcPrice': calc_end_prices[benchmark_ticker], 'CalcGain': calc_gains[benchmark_ticker], 'FwdGain': fwd_gains[benchmark_ticker]}, index=pd.Index([f"{benchmark_ticker} (BM)"], name='Ticker'))
        results_df = pd.concat([results_df, benchmark_df_row])
    
    if debug:
        df_trace = normalized_plot_data.copy()
        df_trace.columns = [f'Norm_Price_{c}' for c in df_trace.columns]
        df_trace['Norm_Price_Portfolio'] = portfolio_series
        if benchmark_price_series is not None and not benchmark_price_series.loc[safe_start_date:safe_viz_end_date].dropna().empty:
            norm_bm = benchmark_price_series.loc[safe_start_date:safe_viz_end_date] / benchmark_price_series.loc[safe_start_date:].bfill().iloc[0]
            df_trace[f'Norm_Price_Benchmark_{benchmark_ticker}'] = norm_bm
        for col in df_trace.columns:
            if 'Norm_Price' in col:
                df_trace[col.replace('Norm_Price', 'Return')] = df_trace[col].pct_change()
        debug_data['portfolio_trace'] = df_trace

    final_results = {
        'tickers_to_display': tickers_to_display, 'normalized_plot_data': normalized_plot_data,
        'portfolio_series': portfolio_series, 'benchmark_price_series': benchmark_price_series,
        'performance_data': perf_data, 'results_df': results_df, 'actual_calc_end_ts': actual_calc_end_ts,
        'safe_start_date': safe_start_date, 'safe_viz_end_date': safe_viz_end_date,
        'error': None
    }
    return (final_results, debug_data)

# --- C. DYNAMIC DATA QUALITY FILTER FUNCTIONS ---

def calculate_rolling_quality_metrics(df_ohlcv, window=252, min_periods=126, debug=False):
    """Calculates rolling data quality metrics for the entire dataset."""
    print(f"--- Calculating Rolling Quality Metrics (Window: {window} days) ---")
    df = df_ohlcv.copy()
    if not df.index.is_monotonic_increasing:
        df.sort_index(inplace=True)
    df['IsStale'] = np.where((df['Volume'] == 0) | (df['Adj High'] == df['Adj Low']), 1, 0)
    df['DollarVolume'] = df['Adj Close'] * df['Volume']
    df['HasSameVolumeAsPrevDay'] = (df.groupby(level='Ticker')['Volume'].diff() == 0).astype(int)
    grouped = df.groupby(level='Ticker')
    stale_pct = grouped['IsStale'].rolling(window=window, min_periods=min_periods).mean()
    median_vol = grouped['DollarVolume'].rolling(window=window, min_periods=min_periods).median()
    same_vol_count = grouped['HasSameVolumeAsPrevDay'].rolling(window=window, min_periods=min_periods).sum()
    quality_df = pd.concat([stale_pct, median_vol, same_vol_count], axis=1)
    quality_df.columns = ['RollingStalePct', 'RollingMedianVolume', 'RollingSameVolCount']
    quality_df.index = quality_df.index.droplevel(0)
    print("✅ Rolling metrics calculation complete.")
    return quality_df

def get_eligible_universe(quality_metrics_df, filter_date, thresholds):
    """Filters the universe of tickers based on quality metrics for a given date."""
    filter_date_ts = pd.to_datetime(filter_date)
    date_index = quality_metrics_df.index.get_level_values('Date').unique().sort_values()
    if filter_date_ts < date_index[0]:
        print(f"Warning: Filter date {filter_date_ts.date()} is before the earliest data point. Returning empty universe.")
        return []
    valid_prior_dates = date_index[date_index <= filter_date_ts]
    if valid_prior_dates.empty:
        print(f"Warning: No available data found on or before {filter_date_ts.date()}. Returning empty universe.")
        return []
    actual_date_to_use = valid_prior_dates[-1]
    if actual_date_to_use.date() != filter_date_ts.date():
        print(f"ℹ️ Info: Filter date {filter_date_ts.date()} not found. Using previous available date {actual_date_to_use.date()}.")
    metrics_on_date = quality_metrics_df.xs(actual_date_to_use, level='Date')
    mask = ((metrics_on_date['RollingMedianVolume'] >= thresholds['min_median_dollar_volume']) &
            (metrics_on_date['RollingStalePct'] <= thresholds['max_stale_pct']) &
            (metrics_on_date['RollingSameVolCount'] <= thresholds['max_same_vol_count']))
    eligible_tickers = metrics_on_date[mask].index.tolist()
    all_tickers = metrics_on_date.index.tolist()
    print(f"Dynamic Filter ({filter_date_ts.date()}): Kept {len(eligible_tickers)} of {len(all_tickers)} tickers.")
    return eligible_tickers    

# --- D. INTERACTIVE ANALYSIS & BACKTESTING TOOLS ---

def plot_walk_forward_analyzer(df_ohlcv, 
                               default_start_date=None, default_calc_period=126, default_fwd_period=63,
                               default_metric='Sharpe (ATR)', default_rank_start=1, default_rank_end=10,
                               default_benchmark_ticker='VOO', master_calendar_ticker='VOO',
                               quality_thresholds={'min_median_dollar_volume': 1_000_000, 'max_stale_pct': 0.05, 'max_same_vol_count': 10},
                               debug=False):
    """Creates an interactive widget for single-period walk-forward analysis."""
    print("Initializing Walk-Forward Analyzer (using Trading Day Logic)...")
    if not isinstance(df_ohlcv.index, pd.MultiIndex): raise ValueError("Input DataFrame must have a (Ticker, Date) MultiIndex.")
    df_ohlcv = df_ohlcv.sort_index()

    if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
        raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
    master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()
    print(f"Master trading day calendar created from '{master_calendar_ticker}' ({len(master_trading_days)} days).")

    print("Pre-calculating data quality metrics...")
    quality_metrics_df = calculate_rolling_quality_metrics(df_ohlcv, window=252)
    print("Pre-processing data (unstacking)...")
    df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
    df_high_full = df_ohlcv['Adj High'].unstack(level=0)
    df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
    
    start_date_picker = widgets.DatePicker(description='Start Date:', value=pd.to_datetime(default_start_date), disabled=False)
    calc_period_input = widgets.IntText(value=default_calc_period, description='Calc Period (days):')
    fwd_period_input = widgets.IntText(value=default_fwd_period, description='Fwd Period (days):')
    metrics = ['Price', 'Sharpe', 'Sharpe (ATR)']
    metric_dropdown = widgets.Dropdown(options=metrics, value=default_metric, description='Metric:')
    rank_start_input = widgets.IntText(value=default_rank_start, description='Rank Start:')
    rank_end_input = widgets.IntText(value=default_rank_end, description='Rank End:')
    benchmark_ticker_input = widgets.Text(value=default_benchmark_ticker, description='Benchmark:', placeholder='Enter Ticker')
    update_button = widgets.Button(description="Update Chart", button_style='primary')
    ticker_list_output = widgets.Output()
    results_container, debug_data_container = [None], [None]

    fig = go.FigureWidget()
    max_traces = 50
    for i in range(max_traces): fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'placeholder_{i}', visible=False, showlegend=False))
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Benchmark', visible=True, showlegend=True, line=dict(color='black', width=3, dash='dash')))
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Group Portfolio', visible=True, showlegend=True, line=dict(color='green', width=3)))

    def update_plot(button_click):
        ticker_list_output.clear_output()
        
        start_date_raw = pd.to_datetime(start_date_picker.value)
        start_date_idx = master_trading_days.searchsorted(start_date_raw)
        if start_date_idx >= len(master_trading_days):
            with ticker_list_output: print(f"Error: Start date is after the last available trading day."); return
        actual_start_date = master_trading_days[start_date_idx]
        with ticker_list_output: 
            if start_date_raw.date() != actual_start_date.date():
                print(f"ℹ️ Info: Start date {start_date_raw.date()} is not a trading day. Snapping forward to {actual_start_date.date()}.")

        calc_period, fwd_period = calc_period_input.value, fwd_period_input.value
        metric = metric_dropdown.value
        rank_start, rank_end = rank_start_input.value, rank_end_input.value
        benchmark_ticker = benchmark_ticker_input.value.strip().upper()
        
        if rank_start > rank_end:
            with ticker_list_output: print("Error: 'Rank Start' must be <= 'Rank End'."); return
        if rank_start < 1 or calc_period < 2 or fwd_period < 1:
            with ticker_list_output: print("Error: Ranks must be >= 1, Calc Period >= 2, Fwd Period >= 1."); return

        eligible_tickers = get_eligible_universe(quality_metrics_df, actual_start_date, quality_thresholds)
        if not eligible_tickers:
            with ticker_list_output: print(f"Error: No eligible tickers found on {actual_start_date.date()} with the current quality filters."); return
        df_close_step = df_close_full[eligible_tickers]; df_high_step = df_high_full[eligible_tickers]; df_low_step = df_low_full[eligible_tickers]

        results, debug_output = run_walk_forward_step(
            df_close_step, df_high_step, df_low_step, master_trading_days,
            actual_start_date, calc_period, fwd_period, 
            metric, rank_start, rank_end, benchmark_ticker, debug=debug
        )
        if results['error']:
            with ticker_list_output: print(f"Error: {results['error']}"); return

        with fig.batch_update():
            for i in range(max_traces):
                trace = fig.data[i]
                if i < len(results['tickers_to_display']):
                    ticker = results['tickers_to_display'][i]; plot_data_series = results['normalized_plot_data'][ticker]
                    trace.x, trace.y, trace.name, trace.visible, trace.showlegend = plot_data_series.index, plot_data_series.values, ticker, True, True
                else: trace.visible, trace.showlegend = False, False
            benchmark_trace = fig.data[max_traces]
            if results['benchmark_price_series'] is not None and not results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']].dropna().empty:
                normalized_benchmark = results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']] / results['benchmark_price_series'].loc[results['safe_start_date']:].bfill().iloc[0]
                benchmark_trace.x, benchmark_trace.y, benchmark_trace.name, benchmark_trace.visible = normalized_benchmark.index, normalized_benchmark, f"Benchmark ({benchmark_ticker})", True
            else: benchmark_trace.visible = False
            portfolio_trace = fig.data[max_traces + 1]
            portfolio_trace.x, portfolio_trace.y, portfolio_trace.name, portfolio_trace.visible = results['portfolio_series'].index, results['portfolio_series'], 'Group Portfolio', True
            fig.layout.shapes = []; fig.add_shape(type="line", x0=results['actual_calc_end_ts'], y0=0, x1=results['actual_calc_end_ts'], y1=1, xref='x', yref='paper', line=dict(color="grey", width=2, dash="dash"))
            
        results_container[0] = results; debug_data_container[0] = debug_output
        
        with ticker_list_output:
            print(f"Analysis Period: {results['safe_start_date'].date()} to {results['safe_viz_end_date'].date()}.")
            pprint.pprint(results['tickers_to_display'])
            p = results['performance_data']
            rows = []
            rows.append({'Metric': 'Group Portfolio Gain', 'Full': p['full_p_gain'], 'Calc': p['calc_p_gain'], 'Fwd': p['fwd_p_gain']})
            if not np.isnan(p['full_b_gain']):
                rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Gain', 'Full': p['full_b_gain'], 'Calc': p['calc_b_gain'], 'Fwd': p['fwd_b_gain']})
                rows.append({'Metric': 'Gain Delta (vs Bm)', 'Full': p['full_p_gain'] - p['full_b_gain'], 'Calc': p['calc_p_gain'] - p['calc_b_gain'], 'Fwd': p['fwd_p_gain'] - p['fwd_b_gain']})
            rows.append({'Metric': 'Group Portfolio Sharpe', 'Full': p['full_p_sharpe'], 'Calc': p['calc_p_sharpe'], 'Fwd': p['fwd_p_sharpe']})
            if not np.isnan(p['full_b_sharpe']):
                rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Sharpe', 'Full': p['full_b_sharpe'], 'Calc': p['calc_b_sharpe'], 'Fwd': p['fwd_b_sharpe']})
                rows.append({'Metric': 'Sharpe Delta (vs Bm)', 'Full': p['full_p_sharpe'] - p['full_b_sharpe'], 'Calc': p['calc_p_sharpe'] - p['calc_b_sharpe'], 'Fwd': p['fwd_p_sharpe'] - p['fwd_b_sharpe']})
            report_df = pd.DataFrame(rows).set_index('Metric')
            gain_rows = [row for row in report_df.index if 'Gain' in row or 'Delta' in row]
            sharpe_rows = [row for row in report_df.index if 'Sharpe' in row]
            styled_df = report_df.style.format('{:+.2%}', na_rep='N/A', subset=(gain_rows, report_df.columns)).format('{:+.2f}', na_rep='N/A', subset=(sharpe_rows, report_df.columns)).set_properties(**{'text-align': 'right', 'width': '100px'}).set_table_styles([{'selector': 'th.col_heading', 'props': [('text-align', 'right')]}, {'selector': 'th.row_heading', 'props': [('text-align', 'left')]}])
            print("\n--- Strategy Performance Summary ---")
            display(styled_df)
            
    fig.update_layout(title_text='Walk-Forward Performance Analysis', xaxis_title='Date', yaxis_title='Normalized Price (Start = 1)', hovermode='x unified', legend_title_text='Tickers (Ranked)', height=600, margin=dict(t=50))
    fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="grey")
    update_button.on_click(update_plot)
    controls_row1 = widgets.HBox([start_date_picker, calc_period_input, fwd_period_input])
    controls_row2 = widgets.HBox([metric_dropdown, rank_start_input, rank_end_input, benchmark_ticker_input, update_button])
    ui_container = widgets.VBox([controls_row1, controls_row2, ticker_list_output], layout=widgets.Layout(margin='10px 0 20px 0'))
    display(ui_container, fig)
    update_plot(None)
    return (results_container, debug_data_container)

def run_full_backtest(df_ohlcv, strategy_params, quality_thresholds):
    """Runs a full backtest of a strategy over a specified date range."""
    print(f"--- Running Full Forensic Backtest for Strategy: {strategy_params['metric']} (Top {strategy_params['rank_start']}-{strategy_params['rank_end']}) ---")
    
    start_date, end_date = pd.to_datetime(strategy_params['start_date']), pd.to_datetime(strategy_params['end_date'])
    calc_period, fwd_period = strategy_params['calc_period'], strategy_params['fwd_period']
    metric, rank_start, rank_end = strategy_params['metric'], strategy_params['rank_start'], strategy_params['rank_end']
    benchmark_ticker = strategy_params['benchmark_ticker']
    master_calendar_ticker = strategy_params.get('master_calendar_ticker', 'VOO')
    
    if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
        raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
    master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()
    
    start_idx = master_trading_days.searchsorted(start_date)
    end_idx = master_trading_days.searchsorted(end_date, side='right')
    
    quality_metrics_df = calculate_rolling_quality_metrics(df_ohlcv, window=252)
    df_close_full = df_ohlcv['Adj Close'].unstack(level=0); df_high_full = df_ohlcv['Adj High'].unstack(level=0); df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
    
    step_indices = range(start_idx, end_idx, fwd_period)
    all_fwd_gains, period_by_period_debug = [], {}

    print(f"Simulating {len(step_indices)} periods from {master_trading_days[step_indices[0]].date()} to {master_trading_days[step_indices[-1]].date()}...")
    for current_idx in tqdm(step_indices, desc="Backtest Progress"):
        step_date = master_trading_days[current_idx]
        eligible_tickers = get_eligible_universe(quality_metrics_df, step_date, quality_thresholds)
        if not eligible_tickers: continue
        
        df_close_step = df_close_full[eligible_tickers]; df_high_step = df_high_full[eligible_tickers]; df_low_step = df_low_full[eligible_tickers]
        
        results, debug_output = run_walk_forward_step(
            df_close_step, df_high_step, df_low_step, master_trading_days,
            step_date, calc_period, fwd_period,
            metric, rank_start, rank_end, benchmark_ticker, debug=True
        )
        if results['error'] is None:
            fwd_series = results['portfolio_series'].loc[results['actual_calc_end_ts']:]
            all_fwd_gains.append(fwd_series.pct_change().dropna())
            period_by_period_debug[step_date.date().isoformat()] = debug_output
            
    if not all_fwd_gains:
        print("Error: No valid periods were simulated."); return None

    strategy_returns = pd.concat(all_fwd_gains); strategy_equity_curve = (1 + strategy_returns).cumprod()
    benchmark_returns = df_close_full[benchmark_ticker].pct_change().loc[strategy_equity_curve.index]; benchmark_equity_curve = (1 + benchmark_returns).cumprod()
    cumulative_equity_df = pd.DataFrame({'Strategy_Equity': strategy_equity_curve, 'Benchmark_Equity': benchmark_equity_curve})
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=cumulative_equity_df.index, y=cumulative_equity_df['Strategy_Equity'], name='Strategy', line=dict(color='green')))
    fig.add_trace(go.Scatter(x=cumulative_equity_df.index, y=cumulative_equity_df['Benchmark_Equity'], name=f'Benchmark ({benchmark_ticker})', line=dict(color='black', dash='dash')))
    fig.update_layout(title=f"Cumulative Performance: '{metric}' Strategy (Top {rank_start}-{rank_end})", xaxis_title="Date", yaxis_title="Cumulative Growth")
    fig.show()

    final_backtest_results = {'cumulative_performance': cumulative_equity_df, 'period_by_period_debug': period_by_period_debug}
    print("\n✅ Full backtest complete. Results object is ready for forensic analysis.")
    return final_backtest_results

# --- E. VERIFICATION TOOLS (User Requested) ---

def verify_group_tickers_walk_forward_calculation(df_ohlcv, tickers_to_verify, benchmark_ticker,
                                                  start_date, calc_period, fwd_period,
                                                  master_calendar_ticker='VOO', export_csv=False):
    """Verifies portfolio and benchmark performance and optionally exports the data."""
    display(Markdown(f"## Verification Report for Portfolio vs. Benchmark"))
    display(Markdown(f"**Portfolio Tickers:** `{tickers_to_verify}`\n**Benchmark Ticker:** `{benchmark_ticker}`"))
    
    if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
        raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
    master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()

    start_date_raw = pd.to_datetime(start_date)
    start_idx = master_trading_days.searchsorted(start_date_raw)
    if start_idx >= len(master_trading_days):
        print(f"Error: Start date {start_date_raw.date()} is after the last available trading day."); return
    actual_start_date = master_trading_days[start_idx]
    
    calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
    fwd_end_idx = min(calc_end_idx + fwd_period, len(master_trading_days) - 1)
    
    actual_calc_end_date = master_trading_days[calc_end_idx]
    actual_fwd_end_date = master_trading_days[fwd_end_idx]
    
    display(Markdown(f"**Analysis Start:** `{actual_start_date.date()}` (Selected: `{start_date_raw.date()}`)\n"
                    f"**Calc End:** `{actual_calc_end_date.date()}` ({calc_period} trading days)\n"
                    f"**Fwd End:** `{actual_fwd_end_date.date()}` ({fwd_period} trading days)"))

    df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
    portfolio_prices_raw_slice = df_close_full[tickers_to_verify].loc[actual_start_date:actual_fwd_end_date]
    portfolio_value_series = portfolio_prices_raw_slice.div(portfolio_prices_raw_slice.bfill().iloc[0]).mean(axis=1)
    benchmark_price_series = df_close_full.get(benchmark_ticker)
    
    if export_csv:
        export_df = pd.DataFrame({
            'Portfolio_Normalized_Price': portfolio_value_series,
            'Portfolio_Daily_Return': portfolio_value_series.pct_change()
        })
        if benchmark_price_series is not None:
            norm_bm = benchmark_price_series.loc[actual_start_date:actual_fwd_end_date]
            norm_bm = norm_bm / norm_bm.bfill().iloc[0]
            export_df['Benchmark_Normalized_Price'] = norm_bm
            export_df['Benchmark_Daily_Return'] = norm_bm.pct_change()

        output_dir = 'export_csv'
        os.makedirs(output_dir, exist_ok=True)
        tickers_str = '_'.join(tickers_to_verify)
        filename = f"verify_group_{actual_start_date.date()}_{tickers_str}.csv"
        filepath = os.path.join(output_dir, filename)
        export_df.to_csv(filepath)
        print(f"\n✅ Data exported to: {filepath}")

    def print_verification_steps(title, price_series):
        display(Markdown(f"#### Verification for: `{title}`"))
        if price_series.dropna().shape[0] < 2: print("  - Not enough data points."); return {'gain': np.nan, 'sharpe': np.nan}
        start_price, end_price = price_series.bfill().iloc[0], price_series.ffill().iloc[-1]
        gain = (end_price / start_price) - 1
        print(f"Start Value ({price_series.first_valid_index().date()}): {start_price:,.4f}\nEnd Value   ({price_series.last_valid_index().date()}): {end_price:,.4f}\nGain = {gain:.2%}")
        returns = price_series.pct_change()
        sharpe = calculate_sharpe(returns)
        print(f"Mean Daily Return: {returns.mean():.6f}\nStd Dev: {returns.std():.6f}\nSharpe = {sharpe:.2f}")
        return {'gain': gain, 'sharpe': sharpe}

    display(Markdown("### A. Calculation Period"))
    perf_calc_p = print_verification_steps("Group Portfolio", portfolio_value_series.loc[actual_start_date:actual_calc_end_date])
    perf_calc_b = print_verification_steps(f"Benchmark", benchmark_price_series.loc[actual_start_date:actual_calc_end_date])
    display(Markdown("### B. Forward Period"))
    perf_fwd_p = print_verification_steps("Group Portfolio", portfolio_value_series.loc[actual_calc_end_date:actual_fwd_end_date])
    perf_fwd_b = print_verification_steps(f"Benchmark", benchmark_price_series.loc[actual_calc_end_date:actual_fwd_end_date])

def verify_ticker_ranking_metrics(df_ohlcv, ticker, start_date, calc_period,
                                  master_calendar_ticker='VOO', export_csv=False):
    """Verifies ranking metrics for a single ticker and optionally exports the data."""
    display(Markdown(f"## Verification Report for Ticker Ranking: `{ticker}`"))
    
    if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
        raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
    master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()

    start_date_raw = pd.to_datetime(start_date)
    start_idx = master_trading_days.searchsorted(start_date_raw)
    if start_idx >= len(master_trading_days):
        print(f"Error: Start date {start_date_raw.date()} is after the last available trading day."); return
    actual_start_date = master_trading_days[start_idx]
    
    calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
    actual_calc_end_date = master_trading_days[calc_end_idx]

    df_ticker = df_ohlcv.loc[ticker].sort_index()
    calc_df = df_ticker.loc[actual_start_date:actual_calc_end_date].copy()
    if calc_df.empty or len(calc_df) < 2: 
        print("No data or not enough data in calc period."); return

    display(Markdown("### A. Calculation Period (for Ranking Metrics)"))
    display(Markdown(f"**Period Start:** `{actual_start_date.date()}`\n"
                    f"**Period End:** `{actual_calc_end_date.date()}`\n"
                    f"**Total Trading Days:** `{len(calc_df)}` (Requested: `{calc_period}`)"))
    
    display(Markdown("#### Detailed Metric Calculation Data"))
    
    vdf = calc_df[['Adj High', 'Adj Low', 'Adj Close']].copy()
    vdf['Daily_Return'] = vdf['Adj Close'].pct_change()
    
    # Corrected True Range calculation
    tr_df = pd.DataFrame({
        'h_l': vdf['Adj High'] - vdf['Adj Low'],
        'h_cp': abs(vdf['Adj High'] - vdf['Adj Close'].shift(1)),
        'l_cp': abs(vdf['Adj Low'] - vdf['Adj Close'].shift(1))
    })
    vdf['TR'] = tr_df.max(axis=1)
    
    vdf['ATR_14'] = vdf['TR'].ewm(alpha=1/14, adjust=False).mean()
    vdf['ATRP'] = vdf['ATR_14'] / vdf['Adj Close']
    
    print("--- Start of Calculation Period ---")
    display(vdf.head())
    print("\n--- End of Calculation Period ---")
    display(vdf.tail())

    if export_csv:
        output_dir = 'export_csv'
        os.makedirs(output_dir, exist_ok=True)
        filename = f"verify_ticker_{actual_start_date.date()}_{ticker}.csv"
        filepath = os.path.join(output_dir, filename)
        vdf.to_csv(filepath)
        print(f"\n✅ Data exported to: {filepath}")
    
    display(Markdown("#### `MetricValue` Verification Summary:"))
    
    calc_start_price = vdf['Adj Close'].bfill().iloc[0]
    calc_end_price = vdf['Adj Close'].ffill().iloc[-1]
    price_metric = (calc_end_price / calc_start_price)
    print(f"1. Price Metric: (Last Price / First Price) = ({calc_end_price:.2f} / {calc_start_price:.2f}) = {price_metric:.4f}")
    
    daily_returns = vdf['Daily_Return'].dropna()
    sharpe_ratio = calculate_sharpe(daily_returns)
    print(f"2. Sharpe Metric: (Mean Daily Return / Std Dev) * sqrt(252) = {sharpe_ratio:.4f}")

    atrp_mean = vdf['ATRP'].mean()
    mean_daily_return = vdf['Daily_Return'].mean()
    sharpe_atr = (mean_daily_return / atrp_mean) if atrp_mean > 0 else 0
    print(f"3. Sharpe (ATR) Metric: (Mean Daily Return / Mean ATRP) = ({mean_daily_return:.6f} / {atrp_mean:.6f}) = {sharpe_atr:.4f}")

In [10]:
download_path = Path.home() / "Downloads"  
# OHLCV_file_path = r'c:\Users\ping\Files_win10\python\py311\stocks\data\df_OHLCV_clean_stocks_etfs.parquet'
OHLCV_file_path = r'c:\Users\ping\Files_win10\python\py311\stocks\data\df_OHLCV_stocks_etfs.parquet'

df_OHLCV = pd.read_parquet(OHLCV_file_path, engine='pyarrow')
print(f'df_OHLCV.info() :\n{df_OHLCV.info()}')
print(f'\ndf_OHLCV.head():\n{df_OHLCV.head()}')
print(f'\ndf_OHLCV.tail():\n{df_OHLCV.tail()}')

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4036635 entries, ('A', Timestamp('1999-11-18 00:00:00')) to ('ZWS', Timestamp('2025-10-06 00:00:00'))
Data columns (total 5 columns):
 #   Column     Dtype  
---  ------     -----  
 0   Adj Open   float64
 1   Adj High   float64
 2   Adj Low    float64
 3   Adj Close  float64
 4   Volume     int64  
dtypes: float64(4), int64(1)
memory usage: 170.0+ MB
df_OHLCV.info() :
None

df_OHLCV.head():
                   Adj Open  Adj High  Adj Low  Adj Close    Volume
Ticker Date                                                        
A      1999-11-18   27.2452   29.9398  23.9518    26.3470  74716422
       1999-11-19   25.7108   25.7482  23.8396    24.1764  18198351
       1999-11-22   24.7378   26.3470  23.9893    26.3470   7857767
       1999-11-23   25.4488   26.1225  23.9518    23.9518   7138322
       1999-11-24   24.0267   25.1120  23.9518    24.5881   5785607

df_OHLCV.tail():
                   Adj Open  Adj High  Adj Low  Adj Close  V

In [11]:
# --- 2. DEFINE THE SPLIT DATE ---
# This is the last day of our In-Sample (IS) "Discovery Zone".
split_date = pd.to_datetime('2018-12-31')

print(f"Splitting data on: {split_date.date()}")
print("="*40)


# --- 3. PERFORM THE SPLIT ---
# We access the 'Date' level of the MultiIndex to create our boolean masks.

# In-Sample (IS) DataFrame: Data for discovery and training the bot.
df_IS = df_OHLCV[df_OHLCV.index.get_level_values('Date') <= split_date].copy()

# Out-of-Sample (OOS) DataFrame: Data held back for final validation.
df_OOS = df_OHLCV[df_OHLCV.index.get_level_values('Date') > split_date].copy()

# Using .copy() is good practice to avoid SettingWithCopyWarning later on.


# --- 4. VERIFY THE SPLIT ---
# Always check your work to ensure the split was done correctly.

print("\n--- In-Sample (IS) 'Discovery Zone' Info ---")
df_IS.info(verbose=False, memory_usage='deep') # Use verbose=False for a cleaner summary
print(f"IS Date Range: {df_IS.index.get_level_values('Date').min().date()} to {df_IS.index.get_level_values('Date').max().date()}")
print(f"IS Shape: {df_IS.shape}")
print(f"Percentage of IS data: {df_IS.shape[0] / df_OHLCV.shape[0]:.2%}")

print("\n--- Out-of-Sample (OOS) 'Validation Zone' Info ---")
df_OOS.info(verbose=False, memory_usage='deep')
print(f"OOS Date Range: {df_OOS.index.get_level_values('Date').min().date()} to {df_OOS.index.get_level_values('Date').max().date()}")
print(f"OOS Shape: {df_OOS.shape}")
print(f"Percentage of OOS data: {df_OOS.shape[0] / df_OHLCV.shape[0]:.2%}")

# Final check
total_rows = df_IS.shape[0] + df_OOS.shape[0]
print(f"\nVerification: {df_IS.shape[0]} (IS) + {df_OOS.shape[0]} (OOS) = {total_rows} rows.")
print(f"Original total rows: {df_OHLCV.shape[0]} rows. Match: {total_rows == df_OHLCV.shape[0]}")

Splitting data on: 2018-12-31

--- In-Sample (IS) 'Discovery Zone' Info ---
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2702881 entries, ('A', Timestamp('1999-11-18 00:00:00')) to ('ZWS', Timestamp('2018-12-31 00:00:00'))
Columns: 5 entries, Adj Open to Volume
dtypes: float64(4), int64(1)
memory usage: 114.1 MB
IS Date Range: 1962-01-02 to 2018-12-31
IS Shape: (2702881, 5)
Percentage of IS data: 66.96%

--- Out-of-Sample (OOS) 'Validation Zone' Info ---
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1333754 entries, ('A', Timestamp('2019-01-02 00:00:00')) to ('ZWS', Timestamp('2025-10-06 00:00:00'))
Columns: 5 entries, Adj Open to Volume
dtypes: float64(4), int64(1)
memory usage: 56.7 MB
OOS Date Range: 2019-01-02 to 2025-10-06
OOS Shape: (1333754, 5)
Percentage of OOS data: 33.04%

Verification: 2702881 (IS) + 1333754 (OOS) = 4036635 rows.
Original total rows: 4036635 rows. Match: True


In [12]:
# This code REPLACES the previous df_dev creation snippet.
# It should still be placed after df_IS and df_OOS are created.

# --- 5. (CORRECTED) CREATE A SMALLER "DEVELOPMENT SANDBOX" DATAFRAME ---
# We use a RECENT 5-year slice of our In-Sample data for rapid development.
# This is much faster and more representative of modern data.

dev_start_date = pd.to_datetime('2014-01-01')
dev_end_date = pd.to_datetime('2018-12-31') # This is the end of our df_IS period

print("\n--- Creating Development Sandbox DataFrame (Corrected) ---")
print(f"Slicing df_IS from {dev_start_date.date()} to {dev_end_date.date()}")
print("="*60)

# Create the development dataframe by slicing the main In-Sample data
df_dev = df_IS[(df_IS.index.get_level_values('Date') >= dev_start_date) &
               (df_IS.index.get_level_values('Date') <= dev_end_date)].copy()


# --- 6. VERIFY THE (CORRECTED) DEVELOPMENT DATAFRAME ---
print("\n--- Development Sandbox (df_dev) Info ---")
df_dev.info(verbose=False, memory_usage='deep')
print(f"df_dev Date Range: {df_dev.index.get_level_values('Date').min().date()} to {df_dev.index.get_level_values('Date').max().date()}")
print(f"df_dev Shape: {df_dev.shape}")
print(f"df_dev as percentage of IS data: {df_dev.shape[0] / df_IS.shape[0]:.2%}")

# --- Get unique tickers from the 'Ticker' level of the MultiIndex ---

# Get the 'Ticker' level of the index
ticker_index = df_dev.index.get_level_values('Ticker')

# Get the unique values from that level
unique_tickers = ticker_index.unique()

# Print the results
print("="*60)
print(f"\nFound {len(unique_tickers)} unique tickers in df_dev.")
print("First 10 unique tickers:")
print(unique_tickers[:10].tolist()) # .tolist() gives a cleaner printout for a slice
print("\nLast 10 unique tickers:")
print(unique_tickers[-10:].tolist())


--- Creating Development Sandbox DataFrame (Corrected) ---
Slicing df_IS from 2014-01-01 to 2018-12-31

--- Development Sandbox (df_dev) Info ---
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 778000 entries, ('A', Timestamp('2014-01-02 00:00:00')) to ('ZWS', Timestamp('2018-12-31 00:00:00'))
Columns: 5 entries, Adj Open to Volume
dtypes: float64(4), int64(1)
memory usage: 33.4 MB
df_dev Date Range: 2014-01-02 to 2018-12-31
df_dev Shape: (778000, 5)
df_dev as percentage of IS data: 28.78%

Found 668 unique tickers in df_dev.
First 10 unique tickers:
['A', 'AAL', 'ABBV', 'ABT', 'ACM', 'ACN', 'ACWI', 'ACWX', 'ADBE', 'ADI']

Last 10 unique tickers:
['XYL', 'XYZ', 'YUM', 'Z', 'ZBH', 'ZBRA', 'ZG', 'ZS', 'ZTS', 'ZWS']


In [13]:
# --- 7. BOT CONFIGURATION MANAGER (UPDATED) ---

bot_config = {
    # --- Time Parameters ---
    'search_start_date': '2014-01-01',
    'search_end_date': '2018-12-31',
    # MODIFICATION: Changed '3M' to '3ME' for Month-End frequency to resolve deprecation warning.
    'step_frequency': '3ME',

    # --- Strategy Parameters (The Search Grid) ---
    'calc_periods': ['6M', '1Y'],
    'fwd_periods': ['3M'], 
    'metrics': ['Sharpe', 'Sharpe (ATR)'],
    'rank_slices': [
        (1, 10),
        (11, 30),
    ],

    # --- Data Quality Filter Parameters within Rolling Window---
    'quality_thresholds': {
        'min_median_dollar_volume': 10_000_000,  # $10 mil trading volume
        'max_stale_pct': 0.05,  # 5% either Volume = 0, or High = Low
        'max_same_vol_count': 1,  # max 1 day with consecutive Volume
    },

    # --- General Parameters ---
    'benchmark_ticker': 'VOO',
    'results_output_path': './export_csv/dev_strategy_search_results.csv'
}


# --- Let's quickly calculate how many simulations this configuration will run ---
from itertools import product

num_param_sets = len(list(product(
    bot_config['calc_periods'],
    bot_config['fwd_periods'],
    bot_config['metrics'],
    bot_config['rank_slices']
)))

# Estimate the number of time steps
time_steps = pd.date_range(
    start=bot_config['search_start_date'],
    end=bot_config['search_end_date'],
    freq=bot_config['step_frequency']
)

print("\n--- Bot Configuration Initialized ---")
print(f"Number of unique parameter combinations: {num_param_sets}")
print(f"Estimated number of time steps: {len(time_steps)}")
print(f"Total simulations to run: {num_param_sets * len(time_steps)}")


--- Bot Configuration Initialized ---
Number of unique parameter combinations: 8
Estimated number of time steps: 20
Total simulations to run: 160


In [14]:
# plot_walk_forward_analyzer_original_v0(
#     df_ohlcv=df_OHLCV,  # CRITICAL: Use the same df_dev as the bot
#     default_start_date='2017-04-30',
#     default_calc_period='6M',
#     default_fwd_period='3M',
#     default_metric='Sharpe (ATR)',
#     default_rank_start=1,
#     default_rank_end=10,
#     default_benchmark_ticker='VOO'
# )

In [15]:
# plot_walk_forward_analyzer_v1(
#     df_ohlcv=df_OHLCV,  # CRITICAL: Use the same df_dev as the bot
#     default_start_date='2017-04-30',
#     default_calc_period='6M',
#     default_fwd_period='3M',
#     default_metric='Sharpe (ATR)',
#     default_rank_start=1,
#     default_rank_end=10,
#     default_benchmark_ticker='VOO'
# )

In [16]:
# plot_walk_forward_analyzer(
#     df_ohlcv=df_OHLCV,  # CRITICAL: Use the same df_dev as the bot
#     default_start_date='2017-04-30',
#     default_calc_period='6M',
#     default_fwd_period='3M',
#     default_metric='Sharpe (ATR)',
#     default_rank_start=1,
#     default_rank_end=10,
#     default_benchmark_ticker='VOO'
# )

In [24]:
results_container, debug_container = plot_walk_forward_analyzer(
    df_ohlcv=df_dev,
    default_start_date='2017-04-30',
    default_calc_period=20,
    default_fwd_period=5,
    default_metric='Sharpe (ATR)',
    default_rank_start=1,
    default_rank_end=5,
    default_benchmark_ticker='VOO',
    quality_thresholds=bot_config['quality_thresholds'],
    debug=True  # <-- Activate the new mode!
)

Initializing Walk-Forward Analyzer (using Trading Day Logic)...
Master trading day calendar created from 'VOO' (1258 days).
Pre-calculating data quality metrics...
--- Calculating Rolling Quality Metrics (Window: 252 days) ---
✅ Rolling metrics calculation complete.
Pre-processing data (unstacking)...


FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'placeholder_0',
              'showlegend': False,
              'type': 'scatter',
              'uid': 'dd0dd86a-eee7-4ff8-8551-6d0ec53a6de2',
              'visible': False,
              'x': [None],
              'y': [None]},
             {'mode': 'lines',
              'name': 'placeholder_1',
              'showlegend': False,
              'type': 'scatter',
              'uid': '59d0ec59-ad04-4b96-bcde-3ddfb8bbde7b',
              'visible': False,
              'x': [None],
              'y': [None]},
             {'mode': 'lines',
              'name': 'placeholder_2',
              'showlegend': False,
              'type': 'scatter',
              'uid': '971c939c-8575-467a-83a8-9b83be0bbec2',
              'visible': False,
              'x': [None],
              'y': [None]},
             {'mode': 'lines',
              'name': 'placeholder_3',
              'showlegend': False,
              'type': 

Dynamic Filter (2017-05-01): Kept 521 of 633 tickers.


In [25]:
# --- THE "BULLETPROOF" VERIFICATION SCRIPT (DEFINITIVELY CORRECTED) ---

# (Run this cell AFTER clicking "Update Chart" in the cell above)

# 2. Access the debug and results data
debug_results = debug_container[0]
results_dict = results_container[0] # The container holds the full dictionary

if debug_results and results_dict:
    print("--- Debug Data Successfully Captured ---")

    # Verify the Ranking ("The Report Card")
    print("\n--- [1] Ranking Metrics for all eligible stocks (Top 15 shown) ---")
    display(debug_results['ranking_metrics'].head(15))

    # Verify the Daily Performance ("The Daily Journal")
    print("\n\n--- [2] Daily Portfolio Trace for the Forward Period ---")
    
    # --- THIS IS THE FINAL FIX ---
    
    # The correct start date for the forward period is stored in our main results dictionary.
    forward_period_start_date = results_dict['actual_calc_end_ts']
    
    # Now we correctly slice the portfolio_trace DataFrame using a proper date.
    fwd_trace = debug_results['portfolio_trace'].loc[forward_period_start_date:]

    # --- END OF FIX ---
    
    print(f"Showing forward trace starting from: {forward_period_start_date.date()}")
    display(fwd_trace.head())
else:
    print("Debug data not found. Did you click 'Update Chart'?")

--- Debug Data Successfully Captured ---

--- [1] Ranking Metrics for all eligible stocks (Top 15 shown) ---


,FirstPrice,LastPrice,MeanDailyReturn,StdDevDailyReturn,MeanATRP,Metric_Price,Metric_Sharpe,Metric_Sharpe (ATR)
Ticker,,,,,,,,
VOD,14.59920,16.63220,0.006593,0.010672,0.008555,1.139254,9.807357,0.770707
INTU,118.32200,132.01800,0.005610,0.016061,0.009419,1.115752,5.544830,0.595607
YUM,56.63230,62.36870,0.004868,0.008219,0.008865,1.101292,9.401147,0.549064
NVDA,2.62969,3.57619,0.016445,0.046372,0.030067,1.359928,5.629597,0.546952
DPZ,165.93100,189.91500,0.006841,0.011984,0.012532,1.144542,9.062244,0.545867
BBY,38.19850,44.00320,0.008176,0.050486,0.015149,1.151961,2.570896,0.539721
PM,71.46520,77.54190,0.004108,0.006353,0.007754,1.085030,10.264892,0.529758
MCK,129.33000,151.41000,0.008115,0.021043,0.015902,1.170726,6.121860,0.510313
TTD,3.75900,5.34900,0.020118,0.074669,0.039954,1.422985,4.277000,0.503521




--- [2] Daily Portfolio Trace for the Forward Period ---
Showing forward trace starting from: 2017-05-30


,Norm_Price_VOD,Norm_Price_INTU,Norm_Price_YUM,Norm_Price_NVDA,Norm_Price_DPZ,Norm_Price_Portfolio,Norm_Price_Benchmark_VOO,Return_VOD,Return_INTU,Return_YUM,Return_NVDA,Return_DPZ,Return_Portfolio,Return_Benchmark_VOO
Date,,,,,,,,,,,,,,
2017-05-30,1.139254,1.115752,1.101292,1.359928,1.144542,1.172154,1.011818,-0.004006,0.010749,0.004438,0.021363,0.006951,0.008344,-0.000949
2017-05-31,1.154515,1.120459,1.104790,1.355046,1.162272,1.179417,1.011592,0.013396,0.004219,0.003176,-0.003590,0.015491,0.006196,-0.000223
2017-06-01,1.147652,1.125953,1.122889,1.355141,1.182148,1.186757,1.019484,-0.005945,0.004903,0.016382,0.000070,0.017101,0.006223,0.007802
2017-06-02,1.151851,1.137743,1.125322,1.348379,1.197028,1.192065,1.022679,0.003659,0.010471,0.002167,-0.004989,0.012587,0.004473,0.003134
2017-06-05,1.151467,1.136467,1.128517,1.389403,1.191809,1.199533,1.021996,-0.000333,-0.001122,0.002839,0.030425,-0.004360,0.006265,-0.000668


In [26]:
def print_nested_dict(d, prefix=""):
    """Recursively print every key→value in a (possibly) nested dict."""
    for k, v in d.items():
        full_key = f"{prefix}.{k}" if prefix else str(k)
        if isinstance(v, dict):
            print_nested_dict(v, full_key)          # dive deeper
        else:
            print(f'{full_key}:\n{v}\n')

# demo
nested = {
    'user': {
        'name': 'Alice',
        'age': 30,
        'address': {
            'city': 'Paris',
            'zip': 75001
        }
    },
    'server': {
        'host': 'localhost',
        'port': 8080
    }
}

print_nested_dict(nested)

user.name:
Alice

user.age:
30

user.address.city:
Paris

user.address.zip:
75001

server.host:
localhost

server.port:
8080



In [27]:
print_nested_dict(results_dict)

tickers_to_display:
['VOD', 'INTU', 'YUM', 'NVDA', 'DPZ']

normalized_plot_data:
Ticker           VOD      INTU       YUM      NVDA       DPZ
Date                                                        
2017-05-01  1.000000  1.000000  1.000000  1.000000  1.000000
2017-05-02  1.011062  1.003102  1.008972  0.970369  1.012355
2017-05-03  1.012590  1.000634  1.038022  0.977587  1.026957
2017-05-04  1.020604  1.003423  1.049124  0.973837  1.045181
2017-05-05  1.030522  1.004936  1.053992  0.973932  1.060170
2017-05-08  1.028611  1.005172  1.051559  0.963711  1.055613
2017-05-09  1.028234  1.009635  1.045780  0.965304  1.072464
2017-05-10  1.035098  1.012508  1.047604  1.137377  1.074163
2017-05-11  1.027851  1.010835  1.051407  1.186235  1.072572
2017-05-12  1.052270  1.007885  1.047907  1.199271  1.072024
2017-05-15  1.052270  1.016810  1.052776  1.259472  1.078834
2017-05-16  1.094615  1.015771  1.054753  1.282915  1.077243
2017-05-17  1.100341  0.998639  1.051863  1.197673  1.051057
2017

In [28]:
print_nested_dict(debug_results)

ranking_metrics:
        FirstPrice  LastPrice  MeanDailyReturn  StdDevDailyReturn  MeanATRP  Metric_Price  Metric_Sharpe  Metric_Sharpe (ATR)
Ticker                                                                                                                       
VOD       14.59920   16.63220         0.006593           0.010672  0.008555      1.139254       9.807357             0.770707
INTU     118.32200  132.01800         0.005610           0.016061  0.009419      1.115752       5.544830             0.595607
YUM       56.63230   62.36870         0.004868           0.008219  0.008865      1.101292       9.401147             0.549064
NVDA       2.62969    3.57619         0.016445           0.046372  0.030067      1.359928       5.629597             0.546952
DPZ      165.93100  189.91500         0.006841           0.011984  0.012532      1.144542       9.062244             0.545867
...            ...        ...              ...                ...       ...           ...            

In [29]:
results_dict['benchmark_price_series'].name

'VOO'

In [30]:
tickers_to_verify = results_dict['tickers_to_display']
benchmark_ticker = results_dict['benchmark_price_series'].name
start_date = results_dict['safe_start_date']

print(f'tickers_to_verify: {tickers_to_verify}')
print(f'benchmark_ticker: {benchmark_ticker}')
print(f'start_date: {start_date}')

tickers_to_verify: ['VOD', 'INTU', 'YUM', 'NVDA', 'DPZ']
benchmark_ticker: VOO
start_date: 2017-05-01 00:00:00


In [31]:
verify_group_tickers_walk_forward_calculation(df_ohlcv=df_dev, 
                                              tickers_to_verify=tickers_to_verify, 
                                              benchmark_ticker=benchmark_ticker,
                                              start_date=start_date, 
                                              calc_period=20, 
                                              fwd_period=5,
                                              master_calendar_ticker=benchmark_ticker, 
                                              export_csv=True)

## Verification Report for Portfolio vs. Benchmark

**Portfolio Tickers:** `['VOD', 'INTU', 'YUM', 'NVDA', 'DPZ']`
**Benchmark Ticker:** `VOO`

**Analysis Start:** `2017-05-01` (Selected: `2017-05-01`)
**Calc End:** `2017-05-30` (20 trading days)
**Fwd End:** `2017-06-06` (5 trading days)


✅ Data exported to: export_csv\verify_group_2017-05-01_VOD_INTU_YUM_NVDA_DPZ.csv


### A. Calculation Period

...

### B. Forward Period

#### Verification for: `Group Portfolio`

Start Value (2017-05-30): 1.1722
End Value   (2017-06-06): 1.1975
Gain = 2.16%
Mean Daily Return: 0.004285
Std Dev: 0.003448
Sharpe = 19.73


#### Verification for: `Benchmark`

Start Value (2017-05-30): 192.5550
End Value   (2017-06-06): 193.9360
Gain = 0.72%
Mean Daily Return: 0.001437
Std Dev: 0.004154
Sharpe = 5.49


In [32]:
verify_ticker_ranking_metrics(df_ohlcv=df_dev, 
                              ticker='VOD', 
                              start_date=start_date, 
                              calc_period=20,
                              master_calendar_ticker='VOO', 
                              export_csv=True)

## Verification Report for Ticker Ranking: `VOD`

### A. Calculation Period (for Ranking Metrics)

**Period Start:** `2017-05-01`
**Period End:** `2017-05-30`
**Total Trading Days:** `21` (Requested: `20`)

#### Detailed Metric Calculation Data

--- Start of Calculation Period ---


...


--- End of Calculation Period ---


,Adj High,Adj Low,Adj Close,Daily_Return,TR,ATR_14,ATRP
Date,,,,,,,
2017-05-23,16.5208,16.4206,16.4317,-0.004055,0.1002,0.180192,0.010966
2017-05-24,16.5264,16.4429,16.5041,0.004406,0.0947,0.174085,0.010548
2017-05-25,16.6991,16.6155,16.6601,0.009452,0.1950,0.175579,0.010539
2017-05-26,16.7047,16.5431,16.6991,0.002341,0.1616,0.174580,0.010454
2017-05-30,16.6879,16.5765,16.6322,-0.004006,0.1226,0.170868,0.010273



✅ Data exported to: export_csv\verify_ticker_2017-05-01_VOD.csv


#### `MetricValue` Verification Summary:

1. Price Metric: (Last Price / First Price) = (16.63 / 14.60) = 1.1393
2. Sharpe Metric: (Mean Daily Return / Std Dev) * sqrt(252) = 9.8074
3. Sharpe (ATR) Metric: (Mean Daily Return / Mean ATRP) = (0.006593 / 0.008555) = 0.7707


In [ ]:
df_dev.loc['VOD']["2017-05-01":"2017-05-30"]

In [ ]:
debug_container

In [ ]:
print(*debug_container[0]) 

In [ ]:
# debug_container[0]['ranking_metrics'].head(50)
debug_container[0]['ranking_metrics'].columns

# debug_container[0]['portfolio_trace'].head(50)
# debug_container[0]['portfolio_trace'].columns

In [ ]:
import pandas as pd
import numpy as np
from itertools import product
from tqdm.auto import tqdm
import time
from IPython.display import display

# --- Assume all functions from our project context are already defined ---
# (calculate_rolling_quality_metrics, get_eligible_universe, run_walk_forward_step, etc.)

# --- Assume df_dev and bot_config are already defined from previous steps ---

def run_strategy_search(df_ohlcv, config):
    """
    Runs the main backtesting loop based on a provided configuration dictionary.
    """
    start_time = time.time()
    
    # --- 1. PRE-PROCESSING (Run once for efficiency) ---
    print("--- Phase 1: Pre-processing Data ---")
    
    # Calculate quality metrics for the entire development period
    quality_metrics_df = calculate_rolling_quality_metrics(df_ohlcv, window=252)

    # Unstack the data for fast slicing later. This is a major optimization.
    print("Unstacking data for performance...")
    df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
    df_high_full = df_ohlcv['Adj High'].unstack(level=0)
    df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
    print("✅ Pre-processing complete.\n")
    
    # --- 2. SETUP THE MAIN LOOP ---
    print("--- Phase 2: Setting up Simulation Loops ---")
    
    # Create all parameter combinations to test
    param_combinations = list(product(
        config['calc_periods'],
        config['fwd_periods'],
        config['metrics'],
        config['rank_slices']
    ))
    
    # Create the list of dates where the bot will re-evaluate the strategy
    step_dates = pd.date_range(
        start=config['search_start_date'],
        end=config['search_end_date'],
        freq=config['step_frequency']
    )
    
    # Map string periods to pandas DateOffset objects for our core function
    period_options = {
        '1M': pd.DateOffset(months=1), '3M': pd.DateOffset(months=3),
        '6M': pd.DateOffset(months=6), '1Y': pd.DateOffset(years=1)
    }
    
    results_log = []
    total_sims = len(param_combinations) * len(step_dates)
    print(f"Found {len(param_combinations)} parameter sets and {len(step_dates)} time steps.")
    print(f"Total simulations to run: {total_sims}")
    print("✅ Setup complete. Starting main loop...\n")

    # --- 3. RUN THE MAIN LOOP ---
    print("--- Phase 3: Running Simulations ---")
    
    # Use tqdm for a progress bar on the outer loop
    pbar = tqdm(param_combinations, desc="Parameter Sets")
    for params in pbar:
        # Unpack parameters for this run
        calc_period_str, fwd_period_str, metric, rank_slice = params
        calc_period = period_options[calc_period_str]
        fwd_period = period_options[fwd_period_str]
        rank_start, rank_end = rank_slice

        # Inner loop for stepping through time
        for step_date in step_dates:
            # 3a. DYNAMIC UNIVERSE SELECTION
            eligible_tickers = get_eligible_universe(
                quality_metrics_df,
                filter_date=step_date,
                thresholds=config['quality_thresholds']
            )

            if not eligible_tickers:
                # print(f"Warning: No eligible tickers on {step_date.date()}. Skipping.")
                continue
            
            # 3b. FILTER DATA FOR THIS STEP
            df_close_step = df_close_full[eligible_tickers]
            df_high_step = df_high_full[eligible_tickers]
            df_low_step = df_low_full[eligible_tickers]

            # 3c. RUN THE CORE ANALYSIS
            step_result = run_walk_forward_step(
                df_close_full=df_close_step,
                df_high_full=df_high_step,
                df_low_full=df_low_step,
                start_date=step_date,
                calc_period=calc_period,
                fwd_period=fwd_period,
                metric=metric,
                rank_start=rank_start,
                rank_end=rank_end,
                benchmark_ticker=config['benchmark_ticker']
            )
            
            # 3d. LOG THE RESULTS
            if step_result['error'] is None:
                p = step_result['performance_data']
                log_entry = {
                    'step_date': step_date.date(),
                    'calc_period': calc_period_str,
                    'fwd_period': fwd_period_str,
                    'metric': metric,
                    'rank_start': rank_start,
                    'rank_end': rank_end,
                    'num_universe': len(eligible_tickers),
                    'num_portfolio': len(step_result['tickers_to_display']),
                    'fwd_p_gain': p['fwd_p_gain'],
                    'fwd_b_gain': p['fwd_b_gain'],
                    'fwd_gain_delta': p['fwd_p_gain'] - p['fwd_b_gain'] if not np.isnan(p['fwd_b_gain']) else np.nan,
                    'fwd_p_sharpe': p['fwd_p_sharpe'],
                }
                results_log.append(log_entry)

    print("✅ Main loop finished.\n")

    # --- 4. SAVE THE RESULTS ---
    print("--- Phase 4: Saving Results ---")
    if not results_log:
        print("Warning: No results were generated. The output file will be empty.")
        return None

    final_df = pd.DataFrame(results_log)
    output_path = config['results_output_path']
    final_df.to_csv(output_path, index=False, float_format='%.4f')

    end_time = time.time()
    print(f"✅ Results saved to '{output_path}'")
    print(f"Total execution time: {end_time - start_time:.2f} seconds.")
    
    return final_df

# --- Execute the Bot ---
dev_results_df = run_strategy_search(df_dev, bot_config)

# --- Display a sample of the results ---
if dev_results_df is not None:
    print("\n--- Sample of Generated Results ---")
    display(dev_results_df.head())

In [ ]:
dev_results_df.to_csv('./export_csv/dev_results_df.csv')

In [ ]:
# Our inspection rules inside the rolling window
ticker_thresholds = {
    'min_median_dollar_volume': 10_000_000, # $10 million median daily trade volume
    'max_stale_pct': 0.1,                   # Allow 10% stale days (i.e. Volume=0 or High=Low)
    'max_same_vol_count': 1                 # Allow at most 1 suspicious volume event (i.e. same Volume on consecutive day)
}

In [ ]:
# --- Verification for the first row of bot results ---

print("--- Replicating the scenario from the first CSV row ---")
print("Start Date: 2014-07-31")
print("Calc Period: 6M, Fwd Period: 3M")
print("Metric: Sharpe, Ranks: 1 to 10")
print("--------------------------------------------------")
print("\nInstructions: The UI will load with the correct defaults. Simply click the 'Update Chart' button.")

# Call the plotter using the parameters from the CSV row as defaults
plot_walk_forward_analyzer(
    df_ohlcv=df_dev,  # CRITICAL: Use the same df_dev as the bot
    default_start_date='2017-04-30',
    default_calc_period='6M',
    default_fwd_period='3M',
    default_metric='Sharpe (ATR)',
    default_rank_start=1,
    default_rank_end=10,
    default_benchmark_ticker='VOO',
    quality_thresholds=ticker_thresholds,
)

### Compare bot results vs verified plot_walk_forward_analyzer_original

In [ ]:
# pick the row
row_index = 43
row = dev_results_df.iloc[row_index]

# convert to dict and reformat step_date
row_dict = row.to_dict()
row_dict['step_date'] = row_dict['step_date'].strftime('%Y-%m-%d')
print(f'row no: {row_index}')
print(f'row_dict: {row_dict}')

_start_date=row_dict['step_date']
_calc_period=row_dict['calc_period']
_fwd_period=row_dict['fwd_period']
_metric=row_dict['metric']
_rank_start=row_dict['rank_start']
_rank_end=row_dict['rank_end']
_benchmark_ticker='VOO'

In [ ]:
plot_walk_forward_analyzer(
    df_ohlcv=df_dev,  # CRITICAL: Use the same df_dev as the bot
    default_start_date=_start_date,
    default_calc_period=_calc_period,
    default_fwd_period=_fwd_period,
    default_metric=_metric,
    default_rank_start=_rank_start,
    default_rank_end=_rank_end,
    default_benchmark_ticker=_benchmark_ticker,
)

In [ ]:
quality_df = calculate_rolling_quality_metrics(
    df_ohlcv=df_OHLCV,
    window=252,
    min_periods=126,
    debug=False,  # <-- The key to our new, improved workflow    
)

In [ ]:
# 1. make sure the whole frame is sorted
df_tmp = quality_df.sort_index()

# 2a. pick the exact date if it exists, otherwise the previous one
date_needed = pd.Timestamp(_start_date)
dates = df_tmp.index.get_level_values('Date').unique().sort_values()   # ← sorted
# first date >= date_needed  (later)
pos = dates.searchsorted(date_needed, side='left')
if pos == len(dates):          # date_needed is after the last date
    later_date = pd.NaT
else:
    later_date = dates[pos]

print(f'_start_date: {_start_date}')
print(f'real start date: {later_date}')

# 3. slice
# quality_df = df_tmp.xs(later_date, level='Date')
# print(quality_df)
print(f"--- Quality Metrics for {later_date} ---")
# print(quality_df.xs('2025-10-03', level='Date'))
print(quality_df.xs(later_date, level='Date'))

In [ ]:
eligible_tickers = get_eligible_universe(quality_metrics_df=quality_df, 
                                        #  filter_date=_start_date, 
                                         filter_date=later_date,                                         
                                         thresholds=ticker_thresholds)

In [ ]:
_df = df_OHLCV.loc[eligible_tickers].copy()
_df.info()

In [ ]:
plot_walk_forward_analyzer_original(
    df_ohlcv=_df,  # CRITICAL: Use the same df_dev as the bot
    default_start_date=_start_date,
    default_calc_period=_calc_period,
    default_fwd_period=_fwd_period,
    default_metric=_metric,
    default_rank_start=_rank_start,
    default_rank_end=_rank_end,
    default_benchmark_ticker=_benchmark_ticker,
)

In [ ]:
import pandas as pd
import numpy as np

# --- 1. Load the Bot's Output ---
try:
    results_df = pd.read_csv(bot_config['results_output_path'])
    print(f"Successfully loaded '{bot_config['results_output_path']}'. Shape: {results_df.shape}")
except FileNotFoundError:
    print("Error: The results file was not found. Please run the bot first.")
    # In a real script, you'd exit here. For a notebook, we'll stop.
    results_df = None

if results_df is not None:
    # --- 2. Define the Strategy Parameters for Grouping ---
    # These are the columns that uniquely identify one strategy configuration.
    # strategy_params = ['calc_period', 'metric', 'rank_start', 'rank_end']
    strategy_params = ['calc_period', 'fwd_period', 'metric', 'rank_start', 'rank_end']    

    # --- 3. Group and Aggregate the Results ---
    # We group by the strategy parameters and calculate key performance metrics for each group.
    summary_df = results_df.groupby(strategy_params).agg(
        avg_fwd_p_gain=('fwd_p_gain', 'mean'),
        std_fwd_p_gain=('fwd_p_gain', 'std'),
        avg_fwd_gain_delta=('fwd_gain_delta', 'mean'),
        # Calculate Win Rate: The percentage of periods with positive forward gain.
        win_rate=('fwd_p_gain', lambda x: (x > 0).sum() / len(x) if len(x) > 0 else 0),
        # Count the number of periods tested for this strategy
        num_periods=('step_date', 'count')
    ).sort_values(by='avg_fwd_gain_delta', ascending=False) # Sort by outperformance vs benchmark

    # --- 4. Format and Display the Summary Table ---
    print("\n--- Strategy Performance Summary (2014-2018 Development Run) ---")
    
    # Apply formatting for better readability
    formatted_summary = summary_df.style.format({
        'avg_fwd_p_gain': '{:+.2%}',
        'std_fwd_p_gain': '{:.2%}',
        'avg_fwd_gain_delta': '{:+.2%}',
        'win_rate': '{:.1%}',
    }).set_properties(**{'text-align': 'right'})

    display(formatted_summary)

In [ ]:
import plotly.graph_objects as go

def plot_cumulative_performance(df_ohlcv, strategy_params, quality_thresholds):
    """
    Plots the cumulative performance of a SINGLE strategy over a specified time range.

    This function simulates rebalancing a portfolio at a fixed frequency and charts
    the resulting equity curve against a benchmark.
    """
    print("--- Running Cumulative Performance Simulation for the Winning Strategy ---")
    
    # --- 1. Setup and Pre-processing ---
    # Unpack strategy parameters from the dictionary
    start_date = strategy_params['start_date']
    end_date = strategy_params['end_date']
    calc_period_str = strategy_params['calc_period']
    fwd_period_str = strategy_params['fwd_period']
    metric = strategy_params['metric']
    rank_start = strategy_params['rank_start']
    rank_end = strategy_params['rank_end']
    benchmark_ticker = strategy_params['benchmark_ticker']
    
    # Pre-calculate quality metrics and unstack data once for efficiency
    quality_metrics_df = calculate_rolling_quality_metrics(df_ohlcv, window=252)
    df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
    df_high_full = df_ohlcv['Adj High'].unstack(level=0)
    df_low_full = df_ohlcv['Adj Low'].unstack(level=0)

    # Map string periods to pandas DateOffset objects
    period_options = {'3M': pd.DateOffset(months=3), '6M': pd.DateOffset(months=6), '1Y': pd.DateOffset(years=1)}
    calc_period = period_options[calc_period_str]
    fwd_period = period_options[fwd_period_str] # This also defines our rebalancing frequency
    
    # --- 2. Main Simulation Loop ---
    # Create the rebalancing dates
    step_dates = pd.date_range(start=start_date, end=end_date, freq=f'{fwd_period.months}ME')
    
    all_fwd_gains = []
    
    print(f"Simulating from {step_dates[0].date()} to {step_dates[-1].date()}...")
    
    for step_date in step_dates:
        # Get the eligible universe for this specific rebalancing date
        eligible_tickers = get_eligible_universe(quality_metrics_df, step_date, quality_thresholds)
        if not eligible_tickers: continue
            
        df_close_step = df_close_full[eligible_tickers]
        df_high_step = df_high_full[eligible_tickers]
        df_low_step = df_low_full[eligible_tickers]
        
        # Run the core calculation for this single step in time
        step_result = run_walk_forward_step(
            df_close_step, df_high_step, df_low_step,
            step_date, calc_period, fwd_period,
            metric, rank_start, rank_end, benchmark_ticker
        )
        
        if step_result['error'] is None:
            # Extract the forward portion of the portfolio's performance
            fwd_series = step_result['portfolio_series'].loc[step_result['actual_calc_end_ts']:]
            all_fwd_gains.append(fwd_series.pct_change().dropna())
            
    # --- 3. Stitch Together Results & Plot ---
    if not all_fwd_gains:
        print("Error: No valid periods were simulated. Cannot plot.")
        return

    # Concatenate all the forward period returns into one long series
    strategy_returns = pd.concat(all_fwd_gains)
    
    # Create the equity curve (cumulative performance)
    # (1 + returns).cumprod() is the standard way to calculate this
    strategy_equity_curve = (1 + strategy_returns).cumprod()
    
    # Get the benchmark returns for the same period
    benchmark_returns = df_close_full[benchmark_ticker].pct_change()
    benchmark_returns_filtered = benchmark_returns.loc[strategy_equity_curve.index]
    benchmark_equity_curve = (1 + benchmark_returns_filtered).cumprod()
    
    # Create the plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=strategy_equity_curve.index, y=strategy_equity_curve,
                             mode='lines', name='Winning Strategy', line=dict(color='green', width=3)))
    fig.add_trace(go.Scatter(x=benchmark_equity_curve.index, y=benchmark_equity_curve,
                             mode='lines', name=f'Benchmark ({benchmark_ticker})', line=dict(color='black', dash='dash')))
    
    fig.update_layout(
        title=f"Cumulative Performance: '{metric}' Strategy (Top {rank_start}-{rank_end}) vs. Benchmark",
        xaxis_title="Date",
        yaxis_title="Cumulative Growth (Normalized to 1)",
        legend_title="Portfolio",
        hovermode='x unified',
        height=600
    )
    fig.show()

In [ ]:
# --- Define the parameters for our winning strategy ---
winning_strategy_params = {
    'start_date': '2014-01-31',
    'end_date': '2018-12-31',
    'calc_period': '6M',
    'fwd_period': '3M',
    'metric': 'Sharpe (ATR)',
    'rank_start': 1,
    'rank_end': 10,
    'benchmark_ticker': 'VOO'
}

# Get the quality thresholds from the bot's configuration
quality_thresholds_from_bot = bot_config['quality_thresholds']

# --- Run the simulation and generate the plot ---
plot_cumulative_performance(
    df_ohlcv=df_dev,  # Run on our development dataset
    strategy_params=winning_strategy_params,
    quality_thresholds=quality_thresholds_from_bot
)